<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#IV-Check-function" data-toc-modified-id="IV-Check-function-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>IV Check function</a></span></li><li><span><a href="#Train-and-test-file" data-toc-modified-id="Train-and-test-file-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Train and test file</a></span></li><li><span><a href="#Bureau-file" data-toc-modified-id="Bureau-file-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Bureau file</a></span></li><li><span><a href="#Prevapplication" data-toc-modified-id="Prevapplication-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Prevapplication</a></span></li><li><span><a href="#Pos-cash" data-toc-modified-id="Pos-cash-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Pos cash</a></span></li><li><span><a href="#Inst-Payments" data-toc-modified-id="Inst-Payments-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Inst Payments</a></span></li><li><span><a href="#CCbalance" data-toc-modified-id="CCbalance-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>CCbalance</a></span></li><li><span><a href="#Getting-important-columns" data-toc-modified-id="Getting-important-columns-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Getting important columns</a></span></li><li><span><a href="#Merging-the-train-files" data-toc-modified-id="Merging-the-train-files-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Merging the train files</a></span></li><li><span><a href="#Substituting-WOE" data-toc-modified-id="Substituting-WOE-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Substituting WOE</a></span></li><li><span><a href="#Model" data-toc-modified-id="Model-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Model</a></span></li><li><span><a href="#Kaggle-test-prediction" data-toc-modified-id="Kaggle-test-prediction-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Kaggle test prediction</a></span><ul class="toc-item"><li><span><a href="#Prevapp" data-toc-modified-id="Prevapp-12.1"><span class="toc-item-num">12.1&nbsp;&nbsp;</span>Prevapp</a></span></li><li><span><a href="#Poscash" data-toc-modified-id="Poscash-12.2"><span class="toc-item-num">12.2&nbsp;&nbsp;</span>Poscash</a></span></li><li><span><a href="#CCbalance" data-toc-modified-id="CCbalance-12.3"><span class="toc-item-num">12.3&nbsp;&nbsp;</span>CCbalance</a></span></li><li><span><a href="#Inst-payments" data-toc-modified-id="Inst-payments-12.4"><span class="toc-item-num">12.4&nbsp;&nbsp;</span>Inst payments</a></span></li><li><span><a href="#Creating-required-columns-dataset" data-toc-modified-id="Creating-required-columns-dataset-12.5"><span class="toc-item-num">12.5&nbsp;&nbsp;</span>Creating required columns dataset</a></span></li></ul></li><li><span><a href="#Metrics" data-toc-modified-id="Metrics-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>Metrics</a></span></li><li><span><a href="#Dropping-multicollinear-columns-using-VIF" data-toc-modified-id="Dropping-multicollinear-columns-using-VIF-14"><span class="toc-item-num">14&nbsp;&nbsp;</span>Dropping multicollinear columns using VIF</a></span></li></ul></div>

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.preprocessing import MinMaxScaler
from scipy import stats

In [2]:
os.chdir(r'D:\DS\BSFA\home-credit-default-risk')

In [3]:
cred=pd.read_csv('application_train.csv')

In [4]:
pd.set_option('display.max_columns', 125)
pd.set_option('display.max_rows', 125)

### IV Check function

In [5]:
def woe(df,targ,col):
    if((df[col].dtype!=object) and (df[col].nunique()>10)):
        df1=df[[col,targ]].sort_values(col)
        totalgood=sum(df[targ]==0)
        totalbad=sum(df[targ]==1)
        df1['bin']=pd.qcut(df1[col],20,duplicates='drop',precision=20,)
        df2=df1.drop(col,axis=1)
        ct=pd.crosstab(df2['bin'],df2[targ],colnames=[None])
        ct.reset_index(0,inplace=True)
        for i in range(len(ct)):
            if ct[1][i]==0:
                ct[1][i]=1
        for i in range(len(ct)):
            if ct[0][i]==0:
                ct[0][i]=1
        ct['% Good']=ct[0]/totalgood
        ct['% Bad']=ct[1]/totalbad
        ct['WOE']=np.log(ct['% Good']/ct['% Bad'])
        ct['IV']=(ct['% Good']-ct['% Bad'])*ct['WOE']
        if sum(df[col].isna())>1:
            missing=df.loc[df[col].isna(),]
            ones=sum(missing['TARGET']==1)
            zeroes=sum(missing['TARGET']==0)
            good=(zeroes/totalgood)
            if ones==0:
                bad=(ones+1/totalbad)
            else:
                bad=(ones/totalbad)
            woev=np.log(good/bad)
            iv=(good-bad)*woev
            ct.loc[max(ct.index)+1]=['Missing Values',zeroes,ones,good,bad,woev,iv]
        return(ct)
    else:
        ct=pd.crosstab(df[col],df[targ],colnames=[None])
        totalgood=sum(df[targ]==0)
        totalbad=sum(df[targ]==1)
        ct.reset_index(0,inplace=True)
        for i in range(len(ct)):
            if ct[1][i]==0:
                ct[1][i]=1
        for i in range(len(ct)):
            if ct[0][i]==0:
                ct[0][i]=1
        ct['% Good']=ct[0]/totalgood
        ct['% Bad']=ct[1]/totalbad
        ct['WOE']=np.log(ct['% Good']/ct['% Bad'])
        ct['IV']=(ct['% Good']-ct['% Bad'])*ct['WOE']
        if sum(df[col].isna())>1:
            missing=df.loc[df[col].isna(),]
            ones=sum(missing['TARGET']==1)
            zeroes=sum(missing['TARGET']==0)
            if ones==0:
                bad=(ones+1/totalbad)
            else:
                bad=(ones/totalbad)
            good=zeroes/totalgood
            woev=np.log(good/bad)
            iv=(good-bad)*woev
            ct.loc[max(ct.index)+1]=['Missing Values',zeroes,ones,good,bad,woev,iv]
        return(ct)

### Train and test file

In [6]:
y=cred['TARGET']
x=cred.drop('TARGET',axis=1)

In [7]:
credit,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.1,random_state=42)

In [8]:
credit['TARGET']=ytrain

D:\Software\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
credit.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET
131683,252724,Cash loans,F,N,Y,2,135000.0,1078200.0,31653.0,900000.0,Family,Working,Higher education,Single / not married,House / apartment,0.010643,-11141,-546,-2842.0,-3728,NaN,1,1,0,1,0,0,Core staff,3.0,2,2,FRIDAY,15,0,0,0,0,0,0,Kindergarten,0.234972,0.674088,0.617826,0.1278,NaN,0.9771,0.6872,0.0130,0.00,0.0345,0.1667,0.2083,0.0373,0.0740,0.0240,0.1390,0.0326,0.1303,NaN,0.9772,0.6994,0.0131,0.0000,0.0345,0.1667,0.2083,0.0381,0.0808,0.0250,0.1401,0.0346,0.1291,NaN,0.9771,0.6914,0.0131,0.00,0.0345,0.1667,0.2083,0.0379,0.0752,0.0245,0.1397,0.0333,reg oper account,block of flats,0.0260,"Stone, brick",No,0.0,0.0,0.0,0.0,-1402.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0
235547,372834,Cash loans,M,N,Y,2,337500.0,1288350.0,37800.0,1125000.0,Unaccompanied,Commercial associate,Secondary / secondary special,Separated,House / apartment,0.022625,-14564,-1720,-8557.0,-4080,NaN,1,1,0,1,0,0,Managers,3.0,2,2,TUESDAY,11,0,0,0,0,0,0,Transport: type 4,NaN,0.719454,0.492060,0.2247,0.0000,0.9806,0.7348,0.0507,0.24,0.2069,0.3333,0.3750,0.1609,0.1816,0.2209,0.0077,0.1351,0.2290,0.0000,0.9806,0.7452,0.0511,0.2417,0.2069,0.3333,0.3750,0.1646,0.1983,0.2301,0.0078,0.1431,0.2269,0.0000,0.9806,0.7383,0.0510,0.24,0.2069,0.3333,0.3750,0.1637,0.1847,0.2248,0.0078,0.1380,reg oper account,block of flats,0.2031,Panel,No,5.0,2.0,5.0,2.0,-852.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0,0
195189,326336,Cash loans,F,Y,N,0,135000.0,521280.0,28408.5,450000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.005144,-15313,-398,-1334.0,-1346,5.0,1,1,0,1,0,0,Sales staff,2.0,2,2,SATURDAY,16,0,0,0,0,1,1,Other,0.624646,0.505922,0.646330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [98]:
iv={}
for i in credit.drop('TARGET',axis=1).columns:
    ct=woe(credit,'TARGET',i)
    iv[i]=sum(ct['IV'])

D:\Software\Anaconda\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [99]:
imptrain1=[]
for i,j in iv.items():
    if j>=0.03:
        imptrain1.append(i)
imptrain1

['CODE_GENDER',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_ID_PUBLISH',
 'FLAG_EMP_PHONE',
 'OCCUPATION_TYPE',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'ORGANIZATION_TYPE',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'ELEVATORS_AVG',
 'FLOORSMAX_AVG',
 'LIVINGAREA_AVG',
 'APARTMENTS_MODE',
 'ELEVATORS_MODE',
 'FLOORSMAX_MODE',
 'LIVINGAREA_MODE',
 'APARTMENTS_MEDI',
 'ELEVATORS_MEDI',
 'FLOORSMAX_MEDI',
 'LIVINGAREA_MEDI',
 'TOTALAREA_MODE',
 'DAYS_LAST_PHONE_CHANGE']

In [100]:
imptrain2=[]
for i,j in iv.items():
    if j>=0.05:
        imptrain2.append(i)
imptrain2

['AMT_CREDIT',
 'AMT_GOODS_PRICE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'OCCUPATION_TYPE',
 'REGION_RATING_CLIENT_W_CITY',
 'ORGANIZATION_TYPE',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3']

### Bureau file

In [13]:
bureau=pd.read_csv('bureau.csv')
bureaubal=pd.read_csv('bureau_balance.csv')

In [14]:
bureaubal=pd.get_dummies(bureaubal)
bureaubal.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X
0,5715448,0,0,0,0,0,0,0,1,0
1,5715448,-1,0,0,0,0,0,0,1,0
2,5715448,-2,0,0,0,0,0,0,1,0
3,5715448,-3,0,0,0,0,0,0,1,0
4,5715448,-4,0,0,0,0,0,0,1,0


In [15]:
bureaubalagg=bureaubal.groupby('SK_ID_BUREAU')
bureaubalagg=bureaubalagg.agg(['sum','count','min','max','mean'])
bureaubalagg.reset_index(inplace=True)
bureaubalagg.head()

SK_ID_BUREAU MONTHS_BALANCE                     STATUS_0                \
                          sum count min max  mean      sum count min max   
0      5001709          -4656    97 -96   0 -48.0        0    97   0   0   
1      5001710          -3403    83 -82   0 -41.0        5    83   0   1   
2      5001711             -6     4  -3   0  -1.5        3     4   0   1   
3      5001712           -171    19 -18   0  -9.0       10    19   0   1   
4      5001713           -231    22 -21   0 -10.5        0    22   0   0   

            STATUS_1                    STATUS_2                    STATUS_3  \
       mean      sum count min max mean      sum count min max mean      sum   
0  0.000000        0    97   0   0  0.0        0    97   0   0  0.0        0   
1  0.060241        0    83   0   0  0.0        0    83   0   0  0.0        0   
2  0.750000        0     4   0   0  0.0        0     4   0   0  0.0        0   
3  0.526316        0    19   0   0  0.0        0    19   0   0  0.0        0   
4  0.000000        0    22   0   0  0.0        0    22   0   0  0.0        0   

                     STATUS_4                    STATUS_5                     \
  count min max mean      sum count min max mean      sum count min max mean   
0    97   0   0  0.0        0    97   0   0  0.0        0    97   0   0  0.0   
1    83   0   0  0.0        0    83   0   0  0.0        0    83   0   0  0.0   
2     4   0   0  0.0        0     4   0   0  0.0        0     4   0   0  0.0   
3    19   0   0  0.0        0    19   0   0  0.0        0    19   0   0  0.0   
4    22   0   0  0.0        0    22   0   0  0.0        0    22   0   0  0.0   

  STATUS_C                         STATUS_X                          
       sum count min max      mean      sum count min max      mean  
0       86    97   0   1  0.886598       11    97   0   1  0.113402  
1       48    83   0   1  0.578313       30    83   0   1  0.361446  
2        0     4   0   0  0.000000        1     4   0   1  0.250000  
3        9    19   0   1  0.473684        0    19   0   0  0.000000  
4        0    22   0   0  0.000000       22    22   1   1  1.000000

In [16]:
bureau=bureau.merge(bureaubalagg,'left','SK_ID_BUREAU')

D:\Software\Anaconda\lib\site-packages\pandas\core\reshape\merge.py:617: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
D:\Software\Anaconda\lib\site-packages\pandas\core\generic.py:3946: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


In [17]:
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,"(MONTHS_BALANCE, sum)","(MONTHS_BALANCE, count)","(MONTHS_BALANCE, min)","(MONTHS_BALANCE, max)","(MONTHS_BALANCE, mean)","(STATUS_0, sum)","(STATUS_0, count)","(STATUS_0, min)","(STATUS_0, max)","(STATUS_0, mean)","(STATUS_1, sum)","(STATUS_1, count)","(STATUS_1, min)","(STATUS_1, max)","(STATUS_1, mean)","(STATUS_2, sum)","(STATUS_2, count)","(STATUS_2, min)","(STATUS_2, max)","(STATUS_2, mean)","(STATUS_3, sum)","(STATUS_3, count)","(STATUS_3, min)","(STATUS_3, max)","(STATUS_3, mean)","(STATUS_4, sum)","(STATUS_4, count)","(STATUS_4, min)","(STATUS_4, max)","(STATUS_4, mean)","(STATUS_5, sum)","(STATUS_5, count)","(STATUS_5, min)","(STATUS_5, max)","(STATUS_5, mean)","(STATUS_C, sum)","(STATUS_C, count)","(STATUS_C, min)","(STATUS_C, max)","(STATUS_C, mean)","(STATUS_X, sum)","(STATUS_X, count)","(STATUS_X, min)","(STATUS_X, max)","(STATUS_X, mean)"
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
targ=credit[['TARGET','SK_ID_CURR']]
targ.head()

,TARGET,SK_ID_CURR
131683,0,252724
235547,0,372834
195189,1,326336
243952,0,382390
198817,0,330511


In [19]:
xtest['TARGET']=ytest
test=xtest[['TARGET','SK_ID_CURR']]
test.head()

D:\Software\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,TARGET,SK_ID_CURR
245895,0,384575
98194,0,214010
36463,0,142232
249923,0,389171
158389,0,283617


In [20]:
bureau=pd.get_dummies(bureau.drop('SK_ID_BUREAU',axis=1))
bureau=bureau.groupby('SK_ID_CURR').agg(['sum','count','min','max','mean'])
bureau.reset_index(inplace=True)
bureau.head()

SK_ID_CURR DAYS_CREDIT                               CREDIT_DAY_OVERDUE  \
                     sum count   min  max         mean                sum   
0     100001       -5145     7 -1572  -49  -735.000000                  0   
1     100002       -6992     8 -1437 -103  -874.000000                  0   
2     100003       -5603     4 -2586 -606 -1400.750000                  0   
3     100004       -1734     2 -1326 -408  -867.000000                  0   
4     100005        -572     3  -373  -62  -190.666667                  0   

                     DAYS_CREDIT_ENDDATE                                    \
  count min max mean                 sum count     min     max        mean   
0     7   0   0  0.0               577.0     7 -1329.0  1778.0   82.428571   
1     8   0   0  0.0             -2094.0     6 -1072.0   780.0 -349.000000   
2     4   0   0  0.0             -2178.0     4 -2434.0  1216.0 -544.500000   
3     2   0   0  0.0              -977.0     2  -595.0  -382.0 -488.500000   
4     3   0   0  0.0              1318.0     3  -128.0  1324.0  439.333333   

  DAYS_ENDDATE_FACT                                   AMT_CREDIT_MAX_OVERDUE  \
                sum count     min    max         mean                    sum   
0           -3302.0     4 -1328.0 -544.0  -825.500000                  0.000   
1           -4185.0     6 -1185.0  -36.0  -697.500000               8405.145   
2           -3292.0     3 -2131.0 -540.0 -1097.333333                  0.000   
3           -1065.0     2  -683.0 -382.0  -532.500000                  0.000   
4            -123.0     1  -123.0 -123.0  -123.000000                  0.000   

                                 CNT_CREDIT_PROLONG                     \
  count  min       max      mean                sum count min max mean   
0     0  NaN       NaN       NaN                  0     7   0   0  0.0   
1     5  0.0  5043.645  1681.029                  0     8   0   0  0.0   
2     4  0.0     0.000     0.000                  0     4   0   0  0.0   
3     1  0.0     0.000     0.000                  0     2   0   0  0.0   
4     1  0.0     0.000     0.000                  0     3   0   0  0.0   

  AMT_CREDIT_SUM                                         AMT_CREDIT_SUM_DEBT  \
             sum count      min       max           mean                 sum   
0    1453365.000     7  85500.0  378000.0  207623.571429            596686.5   
1     865055.565     8      0.0  450000.0  108131.945625            245781.0   
2    1017400.500     4  22248.0  810000.0  254350.125000                 0.0   
3     189037.800     2  94500.0   94537.8   94518.900000                 0.0   
4     657126.000     3  29826.0  568800.0  219042.000000            568408.5   

                                      AMT_CREDIT_SUM_LIMIT             \
  count  min       max           mean                  sum count  min   
0     7  0.0  373239.0   85240.928571                0.000     6  0.0   
1     5  0.0  245781.0   49156.200000            31988.565     4  0.0   
2     4  0.0       0.0       0.000000           810000.000     4  0.0   
3     2  0.0       0.0       0.000000                0.000     2  0.0   
4     3  0.0  543087.0  189469.500000                0.000     3  0.0   

                            AMT_CREDIT_SUM_OVERDUE                       \
          max          mean                    sum count  min  max mean   
0       0.000       0.00000                    0.0     7  0.0  0.0  0.0   
1   31988.565    7997.14125                    0.0     8  0.0  0.0  0.0   
2  810000.000  202500.00000                    0.0     4  0.0  0.0  0.0   
3       0.000       0.00000                    0.0     2  0.0  0.0  0.0   
4       0.000       0.00000                    0.0     3  0.0  0.0  0.0   

  DAYS_CREDIT_UPDATE                              AMT_ANNUITY             \
                 sum count   min  max        mean         sum count  min   
0               -652     7  -155   -6  -93.142857     24817.5     7  0.0   
1       

In [21]:
bureautest=test.merge(bureau,'left','SK_ID_CURR')
bureautest.dropna(subset=['TARGET'],inplace=True)
bureautest.head()

,TARGET,SK_ID_CURR,"(DAYS_CREDIT, sum)","(DAYS_CREDIT, count)","(DAYS_CREDIT, min)","(DAYS_CREDIT, max)","(DAYS_CREDIT, mean)","(CREDIT_DAY_OVERDUE, sum)","(CREDIT_DAY_OVERDUE, count)","(CREDIT_DAY_OVERDUE, min)","(CREDIT_DAY_OVERDUE, max)","(CREDIT_DAY_OVERDUE, mean)","(DAYS_CREDIT_ENDDATE, sum)","(DAYS_CREDIT_ENDDATE, count)","(DAYS_CREDIT_ENDDATE, min)","(DAYS_CREDIT_ENDDATE, max)","(DAYS_CREDIT_ENDDATE, mean)","(DAYS_ENDDATE_FACT, sum)","(DAYS_ENDDATE_FACT, count)","(DAYS_ENDDATE_FACT, min)","(DAYS_ENDDATE_FACT, max)","(DAYS_ENDDATE_FACT, mean)","(AMT_CREDIT_MAX_OVERDUE, sum)","(AMT_CREDIT_MAX_OVERDUE, count)","(AMT_CREDIT_MAX_OVERDUE, min)","(AMT_CREDIT_MAX_OVERDUE, max)","(AMT_CREDIT_MAX_OVERDUE, mean)","(CNT_CREDIT_PROLONG, sum)","(CNT_CREDIT_PROLONG, count)","(CNT_CREDIT_PROLONG, min)","(CNT_CREDIT_PROLONG, max)","(CNT_CREDIT_PROLONG, mean)","(AMT_CREDIT_SUM, sum)","(AMT_CREDIT_SUM, count)","(AMT_CREDIT_SUM, min)","(AMT_CREDIT_SUM, max)","(AMT_CREDIT_SUM, mean)","(AMT_CREDIT_SUM_DEBT, sum)","(AMT_CREDIT_SUM_DEBT, count)","(AMT_CREDIT_SUM_DEBT, min)","(AMT_CREDIT_SUM_DEBT, max)","(AMT_CREDIT_SUM_DEBT, mean)","(AMT_CREDIT_SUM_LIMIT, sum)","(AMT_CREDIT_SUM_LIMIT, count)","(AMT_CREDIT_SUM_LIMIT, min)","(AMT_CREDIT_SUM_LIMIT, max)","(AMT_CREDIT_SUM_LIMIT, mean)","(AMT_CREDIT_SUM_OVERDUE, sum)","(AMT_CREDIT_SUM_OVERDUE, count)","(AMT_CREDIT_SUM_OVERDUE, min)","(AMT_CREDIT_SUM_OVERDUE, max)","(AMT_CREDIT_SUM_OVERDUE, mean)","(DAYS_CREDIT_UPDATE, sum)","(DAYS_CREDIT_UPDATE, count)","(DAYS_CREDIT_UPDATE, min)","(DAYS_CREDIT_UPDATE, max)","(DAYS_CREDIT_UPDATE, mean)","(AMT_ANNUITY, sum)","(AMT_ANNUITY, count)","(AMT_ANNUITY, min)","(AMT_ANNUITY, max)","(AMT_ANNUITY, mean)",...,"(CREDIT_TYPE_Cash loan (non-earmarked), max)","(CREDIT_TYPE_Cash loan (non-earmarked), mean)","(CREDIT_TYPE_Consumer credit, sum)","(CREDIT_TYPE_Consumer credit, count)","(CREDIT_TYPE_Consumer credit, min)","(CREDIT_TYPE_Consumer credit, max)","(CREDIT_TYPE_Consumer credit, mean)","(CREDIT_TYPE_Credit card, sum)","(CREDIT_TYPE_Credit card, count)","(CREDIT_TYPE_Credit card, min)","(CREDIT_TYPE_Credit card, max)","(CREDIT_TYPE_Credit card, mean)","(CREDIT_TYPE_Interbank credit, sum)","(CREDIT_TYPE_Interbank credit, count)","(CREDIT_TYPE_Interbank credit, min)","(CREDIT_TYPE_Interbank credit, max)","(CREDIT_TYPE_Interbank credit, mean)","(CREDIT_TYPE_Loan for business development, sum)","(CREDIT_TYPE_Loan for business development, count)","(CREDIT_TYPE_Loan for business development, min)","(CREDIT_TYPE_Loan for business development, max)","(CREDIT_TYPE_Loan for business development, mean)","(CREDIT_TYPE_Loan for purchase of shares (margin lending), sum)","(CREDIT_TYPE_Loan for purchase of shares (margin lending), count)","(CREDIT_TYPE_Loan for purchase of shares (margin lending), min)","(CREDIT_TYPE_Loan for purchase of shares (margin lending), max)","(CREDIT_TYPE_Loan for purchase of shares (margin lending), mean)","(CREDIT_TYPE_Loan for the purchase of equipment, sum)","(CREDIT_TYPE_Loan for the purchase of equipment, count)","(CREDIT_TYPE_Loan for the purchase of equipment, min)","(CREDIT_TYPE_Loan for the purchase of equipment, max)","(CREDIT_TYPE_Loan for the purchase of equipment, mean)","(CREDIT_TYPE_Loan for working capital replenishment, sum)","(CREDIT_TYPE_Loan for working capital replenishment, count)","(CREDIT_TYPE_Loan for working capital replenishment, min)","(CREDIT_TYPE_Loan for working capital replenishment, max)","(CREDIT_TYPE_Loan for working capital replenishment, mean)","(CREDIT_TYPE_Microloan, sum)","(CREDIT_TYPE_Microloan, count)","(CREDIT_TYPE_Microloan, min)","(CREDIT_TYPE_Microloan, max)","(CREDIT_TYPE_Microloan, mean)","(CREDIT_TYPE_Mobile operator loan, sum)","(CREDIT_TYPE_Mobile operator loan, count)","(CREDIT_TYPE_Mobile operator loan, min)","(CREDIT_TYPE_Mobile operator loan, max)","(CREDIT_TYPE_Mobile operator loan, mean)","(CREDIT_TYPE_Mortgage, sum)","(CREDIT_TYPE_Mortgage, count)","(CREDIT_TYPE_Mortgage, min)","(CREDIT_TYP

In [25]:
bureau=targ.merge(bureau,'left','SK_ID_CURR')
bureau.dropna(subset=['TARGET'],inplace=True)
bureau.head()

,TARGET,SK_ID_CURR,"(DAYS_CREDIT, sum)","(DAYS_CREDIT, count)","(DAYS_CREDIT, min)","(DAYS_CREDIT, max)","(DAYS_CREDIT, mean)","(CREDIT_DAY_OVERDUE, sum)","(CREDIT_DAY_OVERDUE, count)","(CREDIT_DAY_OVERDUE, min)","(CREDIT_DAY_OVERDUE, max)","(CREDIT_DAY_OVERDUE, mean)","(DAYS_CREDIT_ENDDATE, sum)","(DAYS_CREDIT_ENDDATE, count)","(DAYS_CREDIT_ENDDATE, min)","(DAYS_CREDIT_ENDDATE, max)","(DAYS_CREDIT_ENDDATE, mean)","(DAYS_ENDDATE_FACT, sum)","(DAYS_ENDDATE_FACT, count)","(DAYS_ENDDATE_FACT, min)","(DAYS_ENDDATE_FACT, max)","(DAYS_ENDDATE_FACT, mean)","(AMT_CREDIT_MAX_OVERDUE, sum)","(AMT_CREDIT_MAX_OVERDUE, count)","(AMT_CREDIT_MAX_OVERDUE, min)","(AMT_CREDIT_MAX_OVERDUE, max)","(AMT_CREDIT_MAX_OVERDUE, mean)","(CNT_CREDIT_PROLONG, sum)","(CNT_CREDIT_PROLONG, count)","(CNT_CREDIT_PROLONG, min)","(CNT_CREDIT_PROLONG, max)","(CNT_CREDIT_PROLONG, mean)","(AMT_CREDIT_SUM, sum)","(AMT_CREDIT_SUM, count)","(AMT_CREDIT_SUM, min)","(AMT_CREDIT_SUM, max)","(AMT_CREDIT_SUM, mean)","(AMT_CREDIT_SUM_DEBT, sum)","(AMT_CREDIT_SUM_DEBT, count)","(AMT_CREDIT_SUM_DEBT, min)","(AMT_CREDIT_SUM_DEBT, max)","(AMT_CREDIT_SUM_DEBT, mean)","(AMT_CREDIT_SUM_LIMIT, sum)","(AMT_CREDIT_SUM_LIMIT, count)","(AMT_CREDIT_SUM_LIMIT, min)","(AMT_CREDIT_SUM_LIMIT, max)","(AMT_CREDIT_SUM_LIMIT, mean)","(AMT_CREDIT_SUM_OVERDUE, sum)","(AMT_CREDIT_SUM_OVERDUE, count)","(AMT_CREDIT_SUM_OVERDUE, min)","(AMT_CREDIT_SUM_OVERDUE, max)","(AMT_CREDIT_SUM_OVERDUE, mean)","(DAYS_CREDIT_UPDATE, sum)","(DAYS_CREDIT_UPDATE, count)","(DAYS_CREDIT_UPDATE, min)","(DAYS_CREDIT_UPDATE, max)","(DAYS_CREDIT_UPDATE, mean)","(AMT_ANNUITY, sum)","(AMT_ANNUITY, count)","(AMT_ANNUITY, min)","(AMT_ANNUITY, max)","(AMT_ANNUITY, mean)",...,"(CREDIT_TYPE_Cash loan (non-earmarked), max)","(CREDIT_TYPE_Cash loan (non-earmarked), mean)","(CREDIT_TYPE_Consumer credit, sum)","(CREDIT_TYPE_Consumer credit, count)","(CREDIT_TYPE_Consumer credit, min)","(CREDIT_TYPE_Consumer credit, max)","(CREDIT_TYPE_Consumer credit, mean)","(CREDIT_TYPE_Credit card, sum)","(CREDIT_TYPE_Credit card, count)","(CREDIT_TYPE_Credit card, min)","(CREDIT_TYPE_Credit card, max)","(CREDIT_TYPE_Credit card, mean)","(CREDIT_TYPE_Interbank credit, sum)","(CREDIT_TYPE_Interbank credit, count)","(CREDIT_TYPE_Interbank credit, min)","(CREDIT_TYPE_Interbank credit, max)","(CREDIT_TYPE_Interbank credit, mean)","(CREDIT_TYPE_Loan for business development, sum)","(CREDIT_TYPE_Loan for business development, count)","(CREDIT_TYPE_Loan for business development, min)","(CREDIT_TYPE_Loan for business development, max)","(CREDIT_TYPE_Loan for business development, mean)","(CREDIT_TYPE_Loan for purchase of shares (margin lending), sum)","(CREDIT_TYPE_Loan for purchase of shares (margin lending), count)","(CREDIT_TYPE_Loan for purchase of shares (margin lending), min)","(CREDIT_TYPE_Loan for purchase of shares (margin lending), max)","(CREDIT_TYPE_Loan for purchase of shares (margin lending), mean)","(CREDIT_TYPE_Loan for the purchase of equipment, sum)","(CREDIT_TYPE_Loan for the purchase of equipment, count)","(CREDIT_TYPE_Loan for the purchase of equipment, min)","(CREDIT_TYPE_Loan for the purchase of equipment, max)","(CREDIT_TYPE_Loan for the purchase of equipment, mean)","(CREDIT_TYPE_Loan for working capital replenishment, sum)","(CREDIT_TYPE_Loan for working capital replenishment, count)","(CREDIT_TYPE_Loan for working capital replenishment, min)","(CREDIT_TYPE_Loan for working capital replenishment, max)","(CREDIT_TYPE_Loan for working capital replenishment, mean)","(CREDIT_TYPE_Microloan, sum)","(CREDIT_TYPE_Microloan, count)","(CREDIT_TYPE_Microloan, min)","(CREDIT_TYPE_Microloan, max)","(CREDIT_TYPE_Microloan, mean)","(CREDIT_TYPE_Mobile operator loan, sum)","(CREDIT_TYPE_Mobile operator loan, count)","(CREDIT_TYPE_Mobile operator loan, min)","(CREDIT_TYPE_Mobile operator loan, max)","(CREDIT_TYPE_Mobile operator loan, mean)","(CREDIT_TYPE_Mortgage, sum)","(CREDIT_TYPE_Mortgage, count)","(CREDIT_TYPE_Mortgage, min)","(CREDIT_TYP

In [26]:
iv={}
for i in bureau.drop('TARGET',axis=1).columns:
    ct=woe(bureau,'TARGET',i)
    iv[i]=sum(ct['IV'])

D:\Software\Anaconda\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Software\Anaconda\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [27]:
impbureau1=[]
for i,j in iv.items():
    if j>=0.03:
        impbureau1.append(i)
impbureau1

[('DAYS_CREDIT', 'sum'),
 ('DAYS_CREDIT', 'min'),
 ('DAYS_CREDIT', 'max'),
 ('DAYS_CREDIT', 'mean'),
 ('DAYS_CREDIT_ENDDATE', 'sum'),
 ('DAYS_CREDIT_ENDDATE', 'min'),
 ('DAYS_CREDIT_ENDDATE', 'max'),
 ('DAYS_CREDIT_ENDDATE', 'mean'),
 ('DAYS_ENDDATE_FACT', 'sum'),
 ('DAYS_ENDDATE_FACT', 'count'),
 ('DAYS_ENDDATE_FACT', 'min'),
 ('DAYS_ENDDATE_FACT', 'max'),
 ('DAYS_ENDDATE_FACT', 'mean'),
 ('AMT_CREDIT_MAX_OVERDUE', 'sum'),
 ('AMT_CREDIT_SUM_DEBT', 'sum'),
 ('AMT_CREDIT_SUM_DEBT', 'min'),
 ('AMT_CREDIT_SUM_DEBT', 'max'),
 ('AMT_CREDIT_SUM_DEBT', 'mean'),
 ('AMT_CREDIT_SUM_LIMIT', 'sum'),
 ('AMT_CREDIT_SUM_LIMIT', 'max'),
 ('AMT_CREDIT_SUM_LIMIT', 'mean'),
 ('DAYS_CREDIT_UPDATE', 'sum'),
 ('DAYS_CREDIT_UPDATE', 'min'),
 ('DAYS_CREDIT_UPDATE', 'max'),
 ('DAYS_CREDIT_UPDATE', 'mean'),
 (('MONTHS_BALANCE', 'sum'), 'mean'),
 (('MONTHS_BALANCE', 'count'), 'mean'),
 (('MONTHS_BALANCE', 'min'), 'mean'),
 (('MONTHS_BALANCE', 'mean'), 'mean'),
 (('STATUS_0', 'count'), 'mean'),
 (('STATUS_1', 'co

In [28]:
impbureau2=[]
for i,j in iv.items():
    if j>=0.05:
        impbureau2.append(i)
impbureau2

[('DAYS_CREDIT', 'sum'),
 ('DAYS_CREDIT', 'min'),
 ('DAYS_CREDIT', 'max'),
 ('DAYS_CREDIT', 'mean'),
 ('DAYS_CREDIT_ENDDATE', 'sum'),
 ('DAYS_CREDIT_ENDDATE', 'min'),
 ('DAYS_CREDIT_ENDDATE', 'max'),
 ('DAYS_CREDIT_ENDDATE', 'mean'),
 ('DAYS_ENDDATE_FACT', 'sum'),
 ('DAYS_ENDDATE_FACT', 'min'),
 ('DAYS_ENDDATE_FACT', 'mean'),
 ('AMT_CREDIT_SUM_DEBT', 'mean'),
 ('AMT_CREDIT_SUM_LIMIT', 'max'),
 ('DAYS_CREDIT_UPDATE', 'sum'),
 ('DAYS_CREDIT_UPDATE', 'min'),
 ('DAYS_CREDIT_UPDATE', 'mean'),
 ('CREDIT_ACTIVE_Active', 'sum'),
 ('CREDIT_ACTIVE_Active', 'mean'),
 ('CREDIT_ACTIVE_Closed', 'mean')]

### Prevapplication

In [29]:
prevapp=pd.read_csv('previous_application.csv')

In [30]:
prevapp.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
prevapp.drop('SK_ID_PREV',axis=1,inplace=True)
prevapp=pd.get_dummies(prevapp)
prevapp.head()

,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,DAYS_DECISION,SELLERPLACE_AREA,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Consumer loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_CONTRACT_TYPE_XNA,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,FLAG_LAST_APPL_PER_CONTRACT_N,FLAG_LAST_APPL_PER_CONTRACT_Y,NAME_CASH_LOAN_PURPOSE_Building a house or an annex,NAME_CASH_LOAN_PURPOSE_Business development,NAME_CASH_LOAN_PURPOSE_Buying a garage,NAME_CASH_LOAN_PURPOSE_Buying a holiday home / land,NAME_CASH_LOAN_PURPOSE_Buying a home,NAME_CASH_LOAN_PURPOSE_Buying a new car,NAME_CASH_LOAN_PURPOSE_Buying a used car,NAME_CASH_LOAN_PURPOSE_Car repairs,NAME_CASH_LOAN_PURPOSE_Education,NAME_CASH_LOAN_PURPOSE_Everyday expenses,NAME_CASH_LOAN_PURPOSE_Furniture,NAME_CASH_LOAN_PURPOSE_Gasification / water supply,NAME_CASH_LOAN_PURPOSE_Hobby,NAME_CASH_LOAN_PURPOSE_Journey,NAME_CASH_LOAN_PURPOSE_Medicine,NAME_CASH_LOAN_PURPOSE_Money for a third person,NAME_CASH_LOAN_PURPOSE_Other,NAME_CASH_LOAN_PURPOSE_Payments on other loans,NAME_CASH_LOAN_PURPOSE_Purchase of electronic equipment,NAME_CASH_LOAN_PURPOSE_Refusal to name the goal,NAME_CASH_LOAN_PURPOSE_Repairs,NAME_CASH_LOAN_PURPOSE_Urgent needs,NAME_CASH_LOAN_PURPOSE_Wedding / gift / holiday,NAME_CASH_LOAN_PURPOSE_XAP,NAME_CASH_LOAN_PURPOSE_XNA,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Unused offer,...,NAME_GOODS_CATEGORY_Insurance,NAME_GOODS_CATEGORY_Jewelry,NAME_GOODS_CATEGORY_Medical Supplies,NAME_GOODS_CATEGORY_Medicine,NAME_GOODS_CATEGORY_Mobile,NAME_GOODS_CATEGORY_Office Appliances,NAME_GOODS_CATEGORY_Other,NAME_GOODS_CATEGORY_Photo / Cinema Equipment,NAME_GOODS_CATEGORY_Sport and Leisure,NAME_GOODS_CATEGORY_Tourism,NAME_GOODS_CATEGORY_Vehicles,NAME_GOODS_CATEGORY_Weapon,NAME_GOODS_CATEGORY_XNA,NAME_PORTFOLIO_Cards,NAME_PORTFOLIO_Cars,NAME_PORTFOLIO_Cash,NAME_PORTFOLIO_POS,NAME_PORTFOLIO_XNA,NAME_PRODUCT_TYPE_XNA,NAME_PRODUCT_TYPE_walk-in,NAME_PRODUCT_TYPE_x-sell,CHANNEL_TYPE_AP+ (Cash loan),CHANNEL_TYPE_Car dealer,CHANNEL_TYPE_Channel of corporate sales,CHANNEL_TYPE_Contact center,CHANNEL_TYPE_Country-wide,CHANNEL_TYPE_Credit and cash offices,CHANNEL_TYPE_Regional / Local,CHANNEL_TYPE_Stone,NAME_SELLER_INDUSTRY_Auto technology,NAME_SELLER_INDUSTRY_Clothing,NAME_SELLER_INDUSTRY_Connectivity,NAME_SELLER_INDUSTRY_Construction,NAME_SELLER_INDUSTRY_Consumer electronics,NAME_SELLER_INDUSTRY_Furniture,NAME_SELLER_INDUSTRY_Industry,NAME_SELLER_INDUSTRY_Jewelry,NAME_SELLER_INDUSTRY_MLM partners,NAME_SELLER_INDUSTRY_Tourism,NAME_SELLER_INDUSTRY_XNA,NAME_YIELD_GROUP_XNA,NAME_YIELD_GROUP_high,NAME_YIELD_GROUP_low_action,NAME_YIELD_GROUP_low_normal,NAME_YIELD_GROUP_middle,PRODUCT_COMBINATION_Card Street,PRODUCT_COMBINATION_Card X-Sell,PRODUCT_COMBINATION_Cash,PRODUCT_COMBINATION_Cash Street: high,PRODUCT_COMBINATION_Cash Street: low,PRODUCT_COMBINATION_Cash Street: middle,PRODUCT_COMBINATION_Cash X-Sell: high,PRODUCT_COMBINATION_Cash X-Sell: low,PRODUCT_COMBINATION_Cash X-Sell: middle,PRODUCT_COMBINATION_POS household with interest,PRODUCT_COMBINATION_POS household without interest,PRODUCT_COMBINATION_POS industry with interest,PRODUCT_COMBINATION_POS industry without interest,PRODUCT_COMBINATION_POS mobile with interest,PRODUCT_COMBINATION_POS mobile without interest,PRODUCT_COMBINATION_POS other with interest,PRODUCT_COMBINATION_POS others without interest
0,271877,1730.430,17145.0,17145.0,0.0,17145.0,15,1,0.0,0.182832,0.867336,-73,35,12.0,365243.0,-42.0,300.0,-42.0,-

In [32]:
prevapp=prevapp.groupby('SK_ID_CURR').agg(['sum','count','min','max','mean'])
prevapp.reset_index(inplace=True)
prevapp.head()

SK_ID_CURR AMT_ANNUITY                                        \
                     sum count       min        max       mean   
0     100001    3951.000     1  3951.000   3951.000   3951.000   
1     100002    9251.775     1  9251.775   9251.775   9251.775   
2     100003  169661.970     3  6737.310  98356.995  56553.990   
3     100004    5357.250     1  5357.250   5357.250   5357.250   
4     100005    4813.200     1  4813.200   4813.200   4813.200   

  AMT_APPLICATION                                      AMT_CREDIT        \
              sum count       min       max       mean        sum count   
0         24835.5     1   24835.5   24835.5   24835.50    23787.0     1   
1        179055.0     1  179055.0  179055.0  179055.00   179055.0     1   
2       1306309.5     3   68809.5  900000.0  435436.50  1452573.0     3   
3         24282.0     1   24282.0   24282.0   24282.00    20106.0     1   
4         44617.5     2       0.0   44617.5   22308.75    40153.5     2   

                                  AMT_DOWN_PAYMENT                        \
        min        max       mean              sum count     min     max   
0   23787.0    23787.0   23787.00           2520.0     1  2520.0  2520.0   
1  179055.0   179055.0  179055.00              0.0     1     0.0     0.0   
2   68053.5  1035882.0  484191.00           6885.0     2     0.0  6885.0   
3   20106.0    20106.0   20106.00           4860.0     1  4860.0  4860.0   
4       0.0    40153.5   20076.75           4464.0     1  4464.0  4464.0   

          AMT_GOODS_PRICE                                      \
     mean             sum count       min       max      mean   
0  2520.0         24835.5     1   24835.5   24835.5   24835.5   
1     0.0        179055.0     1  179055.0  179055.0  179055.0   
2  3442.5       1306309.5     3   68809.5  900000.0  435436.5   
3  4860.0         24282.0     1   24282.0   24282.0   24282.0   
4  4464.0         44617.5     1   44617.5   44617.5   44617.5   

  HOUR_APPR_PROCESS_START                          NFLAG_LAST_APPL_IN_DAY  \
                      sum count min max       mean                    sum   
0                      13     1  13  13  13.000000                      1   
1                       9     1   9   9   9.000000                      1   
2                      44     3  12  17  14.666667                      3   
3                       5     1   5   5   5.000000                      1   
4                      21     2  10  11  10.500000                      2   

                     RATE_DOWN_PAYMENT                                      \
  count min max mean               sum count       min       max      mean   
0     1   1   1  1.0          0.104326     1  0.104326  0.104326  0.104326   
1     1   1   1  1.0          0.000000     1  0.000000  0.000000  0.000000   
2     3   1   1  1.0          0.100061     2  0.000000  0.100061  0.050030   
3     1   1   1  1.0          0.212008     1  0.212008  0.212008  0.212008   
4     2   1   1  1.0          0.108964     1  0.108964  0.108964  0.108964   

  RATE_INTEREST_PRIMARY                    RATE_INTEREST_PRIVILEGED            \
                    sum count min max mean                      sum count min   
0                   0.0     0 NaN NaN  NaN                      0.0     0 NaN   
1                   0.0     0 NaN NaN  NaN                      0.0     0 NaN   
2                   0.0     0 NaN NaN  NaN                      0.0     0 NaN   
3                   0.0     0 NaN NaN  NaN                      0.0     0 NaN   
4                   0.0     0 NaN NaN  NaN                      0.0     0 NaN   

           DAYS_DECISION                           SELLERPLACE_AREA        \
  max mean           sum count   min   max    mean              sum count   
0 NaN  NaN         -1740     1 -1740 -1740 -1740.0               23     1   
1 NaN  NaN          -606     1  -606  -606  -606.0              500     1   
2 NaN  NaN         -3915     3 -2341  -746 -1305.0           

In [33]:
prevapptest=test.merge(prevapp,'left','SK_ID_CURR')
prevapptest.dropna(subset=['TARGET'],inplace=True)
prevapptest.head()

,TARGET,SK_ID_CURR,"(AMT_ANNUITY, sum)","(AMT_ANNUITY, count)","(AMT_ANNUITY, min)","(AMT_ANNUITY, max)","(AMT_ANNUITY, mean)","(AMT_APPLICATION, sum)","(AMT_APPLICATION, count)","(AMT_APPLICATION, min)","(AMT_APPLICATION, max)","(AMT_APPLICATION, mean)","(AMT_CREDIT, sum)","(AMT_CREDIT, count)","(AMT_CREDIT, min)","(AMT_CREDIT, max)","(AMT_CREDIT, mean)","(AMT_DOWN_PAYMENT, sum)","(AMT_DOWN_PAYMENT, count)","(AMT_DOWN_PAYMENT, min)","(AMT_DOWN_PAYMENT, max)","(AMT_DOWN_PAYMENT, mean)","(AMT_GOODS_PRICE, sum)","(AMT_GOODS_PRICE, count)","(AMT_GOODS_PRICE, min)","(AMT_GOODS_PRICE, max)","(AMT_GOODS_PRICE, mean)","(HOUR_APPR_PROCESS_START, sum)","(HOUR_APPR_PROCESS_START, count)","(HOUR_APPR_PROCESS_START, min)","(HOUR_APPR_PROCESS_START, max)","(HOUR_APPR_PROCESS_START, mean)","(NFLAG_LAST_APPL_IN_DAY, sum)","(NFLAG_LAST_APPL_IN_DAY, count)","(NFLAG_LAST_APPL_IN_DAY, min)","(NFLAG_LAST_APPL_IN_DAY, max)","(NFLAG_LAST_APPL_IN_DAY, mean)","(RATE_DOWN_PAYMENT, sum)","(RATE_DOWN_PAYMENT, count)","(RATE_DOWN_PAYMENT, min)","(RATE_DOWN_PAYMENT, max)","(RATE_DOWN_PAYMENT, mean)","(RATE_INTEREST_PRIMARY, sum)","(RATE_INTEREST_PRIMARY, count)","(RATE_INTEREST_PRIMARY, min)","(RATE_INTEREST_PRIMARY, max)","(RATE_INTEREST_PRIMARY, mean)","(RATE_INTEREST_PRIVILEGED, sum)","(RATE_INTEREST_PRIVILEGED, count)","(RATE_INTEREST_PRIVILEGED, min)","(RATE_INTEREST_PRIVILEGED, max)","(RATE_INTEREST_PRIVILEGED, mean)","(DAYS_DECISION, sum)","(DAYS_DECISION, count)","(DAYS_DECISION, min)","(DAYS_DECISION, max)","(DAYS_DECISION, mean)","(SELLERPLACE_AREA, sum)","(SELLERPLACE_AREA, count)","(SELLERPLACE_AREA, min)","(SELLERPLACE_AREA, max)","(SELLERPLACE_AREA, mean)",...,"(PRODUCT_COMBINATION_Cash Street: low, max)","(PRODUCT_COMBINATION_Cash Street: low, mean)","(PRODUCT_COMBINATION_Cash Street: middle, sum)","(PRODUCT_COMBINATION_Cash Street: middle, count)","(PRODUCT_COMBINATION_Cash Street: middle, min)","(PRODUCT_COMBINATION_Cash Street: middle, max)","(PRODUCT_COMBINATION_Cash Street: middle, mean)","(PRODUCT_COMBINATION_Cash X-Sell: high, sum)","(PRODUCT_COMBINATION_Cash X-Sell: high, count)","(PRODUCT_COMBINATION_Cash X-Sell: high, min)","(PRODUCT_COMBINATION_Cash X-Sell: high, max)","(PRODUCT_COMBINATION_Cash X-Sell: high, mean)","(PRODUCT_COMBINATION_Cash X-Sell: low, sum)","(PRODUCT_COMBINATION_Cash X-Sell: low, count)","(PRODUCT_COMBINATION_Cash X-Sell: low, min)","(PRODUCT_COMBINATION_Cash X-Sell: low, max)","(PRODUCT_COMBINATION_Cash X-Sell: low, mean)","(PRODUCT_COMBINATION_Cash X-Sell: middle, sum)","(PRODUCT_COMBINATION_Cash X-Sell: middle, count)","(PRODUCT_COMBINATION_Cash X-Sell: middle, min)","(PRODUCT_COMBINATION_Cash X-Sell: middle, max)","(PRODUCT_COMBINATION_Cash X-Sell: middle, mean)","(PRODUCT_COMBINATION_POS household with interest, sum)","(PRODUCT_COMBINATION_POS household with interest, count)","(PRODUCT_COMBINATION_POS household with interest, min)","(PRODUCT_COMBINATION_POS household with interest, max)","(PRODUCT_COMBINATION_POS household with interest, mean)","(PRODUCT_COMBINATION_POS household without interest, sum)","(PRODUCT_COMBINATION_POS household without interest, count)","(PRODUCT_COMBINATION_POS household without interest, min)","(PRODUCT_COMBINATION_POS household without interest, max)","(PRODUCT_COMBINATION_POS household without interest, mean)","(PRODUCT_COMBINATION_POS industry with interest, sum)","(PRODUCT_COMBINATION_POS industry with interest, count)","(PRODUCT_COMBINATION_POS industry with interest, min)","(PRODUCT_COMBINATION_POS industry with interest, max)","(PRODUCT_COMBINATION_POS industry with interest, mean)","(PRODUCT_COMBINATION_POS industry without interest, sum)","(PRODUCT_COMBINATION_POS industry without interest, count)","(PRODUCT_COMBINATION_POS industry without interest, min)","(PRODUCT_COMBINATION_POS industry without interest, max)","(PRODUCT_COMBINATION_POS industry without interest, mean)","(PRODUCT_COMBINATION_POS mobile with interest, sum)","(PRODUCT_COMBINATION_POS mobile with in

In [34]:
prevapp=targ.merge(prevapp,'left','SK_ID_CURR')
prevapp.dropna(subset=['TARGET'],inplace=True)
prevapp.head()

,TARGET,SK_ID_CURR,"(AMT_ANNUITY, sum)","(AMT_ANNUITY, count)","(AMT_ANNUITY, min)","(AMT_ANNUITY, max)","(AMT_ANNUITY, mean)","(AMT_APPLICATION, sum)","(AMT_APPLICATION, count)","(AMT_APPLICATION, min)","(AMT_APPLICATION, max)","(AMT_APPLICATION, mean)","(AMT_CREDIT, sum)","(AMT_CREDIT, count)","(AMT_CREDIT, min)","(AMT_CREDIT, max)","(AMT_CREDIT, mean)","(AMT_DOWN_PAYMENT, sum)","(AMT_DOWN_PAYMENT, count)","(AMT_DOWN_PAYMENT, min)","(AMT_DOWN_PAYMENT, max)","(AMT_DOWN_PAYMENT, mean)","(AMT_GOODS_PRICE, sum)","(AMT_GOODS_PRICE, count)","(AMT_GOODS_PRICE, min)","(AMT_GOODS_PRICE, max)","(AMT_GOODS_PRICE, mean)","(HOUR_APPR_PROCESS_START, sum)","(HOUR_APPR_PROCESS_START, count)","(HOUR_APPR_PROCESS_START, min)","(HOUR_APPR_PROCESS_START, max)","(HOUR_APPR_PROCESS_START, mean)","(NFLAG_LAST_APPL_IN_DAY, sum)","(NFLAG_LAST_APPL_IN_DAY, count)","(NFLAG_LAST_APPL_IN_DAY, min)","(NFLAG_LAST_APPL_IN_DAY, max)","(NFLAG_LAST_APPL_IN_DAY, mean)","(RATE_DOWN_PAYMENT, sum)","(RATE_DOWN_PAYMENT, count)","(RATE_DOWN_PAYMENT, min)","(RATE_DOWN_PAYMENT, max)","(RATE_DOWN_PAYMENT, mean)","(RATE_INTEREST_PRIMARY, sum)","(RATE_INTEREST_PRIMARY, count)","(RATE_INTEREST_PRIMARY, min)","(RATE_INTEREST_PRIMARY, max)","(RATE_INTEREST_PRIMARY, mean)","(RATE_INTEREST_PRIVILEGED, sum)","(RATE_INTEREST_PRIVILEGED, count)","(RATE_INTEREST_PRIVILEGED, min)","(RATE_INTEREST_PRIVILEGED, max)","(RATE_INTEREST_PRIVILEGED, mean)","(DAYS_DECISION, sum)","(DAYS_DECISION, count)","(DAYS_DECISION, min)","(DAYS_DECISION, max)","(DAYS_DECISION, mean)","(SELLERPLACE_AREA, sum)","(SELLERPLACE_AREA, count)","(SELLERPLACE_AREA, min)","(SELLERPLACE_AREA, max)","(SELLERPLACE_AREA, mean)",...,"(PRODUCT_COMBINATION_Cash Street: low, max)","(PRODUCT_COMBINATION_Cash Street: low, mean)","(PRODUCT_COMBINATION_Cash Street: middle, sum)","(PRODUCT_COMBINATION_Cash Street: middle, count)","(PRODUCT_COMBINATION_Cash Street: middle, min)","(PRODUCT_COMBINATION_Cash Street: middle, max)","(PRODUCT_COMBINATION_Cash Street: middle, mean)","(PRODUCT_COMBINATION_Cash X-Sell: high, sum)","(PRODUCT_COMBINATION_Cash X-Sell: high, count)","(PRODUCT_COMBINATION_Cash X-Sell: high, min)","(PRODUCT_COMBINATION_Cash X-Sell: high, max)","(PRODUCT_COMBINATION_Cash X-Sell: high, mean)","(PRODUCT_COMBINATION_Cash X-Sell: low, sum)","(PRODUCT_COMBINATION_Cash X-Sell: low, count)","(PRODUCT_COMBINATION_Cash X-Sell: low, min)","(PRODUCT_COMBINATION_Cash X-Sell: low, max)","(PRODUCT_COMBINATION_Cash X-Sell: low, mean)","(PRODUCT_COMBINATION_Cash X-Sell: middle, sum)","(PRODUCT_COMBINATION_Cash X-Sell: middle, count)","(PRODUCT_COMBINATION_Cash X-Sell: middle, min)","(PRODUCT_COMBINATION_Cash X-Sell: middle, max)","(PRODUCT_COMBINATION_Cash X-Sell: middle, mean)","(PRODUCT_COMBINATION_POS household with interest, sum)","(PRODUCT_COMBINATION_POS household with interest, count)","(PRODUCT_COMBINATION_POS household with interest, min)","(PRODUCT_COMBINATION_POS household with interest, max)","(PRODUCT_COMBINATION_POS household with interest, mean)","(PRODUCT_COMBINATION_POS household without interest, sum)","(PRODUCT_COMBINATION_POS household without interest, count)","(PRODUCT_COMBINATION_POS household without interest, min)","(PRODUCT_COMBINATION_POS household without interest, max)","(PRODUCT_COMBINATION_POS household without interest, mean)","(PRODUCT_COMBINATION_POS industry with interest, sum)","(PRODUCT_COMBINATION_POS industry with interest, count)","(PRODUCT_COMBINATION_POS industry with interest, min)","(PRODUCT_COMBINATION_POS industry with interest, max)","(PRODUCT_COMBINATION_POS industry with interest, mean)","(PRODUCT_COMBINATION_POS industry without interest, sum)","(PRODUCT_COMBINATION_POS industry without interest, count)","(PRODUCT_COMBINATION_POS industry without interest, min)","(PRODUCT_COMBINATION_POS industry without interest, max)","(PRODUCT_COMBINATION_POS industry without interest, mean)","(PRODUCT_COMBINATION_POS mobile with interest, sum)","(PRODUCT_COMBINATION_POS mobile with in

In [35]:
iv={}
for i in prevapp.drop('TARGET',axis=1).columns:
    ct=woe(prevapp,'TARGET',i)
    iv[i]=sum(ct['IV'])

D:\Software\Anaconda\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
D:\Software\Anaconda\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Software\Anaconda\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [36]:
impprevapp1=[]
for i,j in iv.items():
    if j>=0.03:
        impprevapp1.append(i)
impprevapp1

[('AMT_DOWN_PAYMENT', 'sum'),
 ('AMT_DOWN_PAYMENT', 'max'),
 ('AMT_DOWN_PAYMENT', 'mean'),
 ('RATE_DOWN_PAYMENT', 'sum'),
 ('RATE_DOWN_PAYMENT', 'max'),
 ('DAYS_DECISION', 'min'),
 ('DAYS_DECISION', 'mean'),
 ('DAYS_FIRST_DRAWING', 'sum'),
 ('DAYS_FIRST_DRAWING', 'mean'),
 ('DAYS_FIRST_DUE', 'sum'),
 ('DAYS_FIRST_DUE', 'min'),
 ('DAYS_FIRST_DUE', 'mean'),
 ('DAYS_LAST_DUE_1ST_VERSION', 'sum'),
 ('DAYS_LAST_DUE_1ST_VERSION', 'min'),
 ('DAYS_LAST_DUE_1ST_VERSION', 'mean'),
 ('DAYS_LAST_DUE', 'sum'),
 ('DAYS_LAST_DUE', 'min'),
 ('DAYS_LAST_DUE', 'mean'),
 ('DAYS_TERMINATION', 'sum'),
 ('DAYS_TERMINATION', 'min'),
 ('DAYS_TERMINATION', 'mean'),
 ('NAME_CONTRACT_STATUS_Approved', 'mean'),
 ('NAME_CONTRACT_STATUS_Refused', 'sum'),
 ('NAME_CONTRACT_STATUS_Refused', 'max'),
 ('NAME_CONTRACT_STATUS_Refused', 'mean'),
 ('CODE_REJECT_REASON_HC', 'max'),
 ('CODE_REJECT_REASON_HC', 'mean'),
 ('CODE_REJECT_REASON_SCOFR', 'max'),
 ('CODE_REJECT_REASON_XAP', 'min'),
 ('CODE_REJECT_REASON_XAP', 'mean')

In [37]:
impprevapp2=[]
for i,j in iv.items():
    if j>=0.05:
        impprevapp2.append(i)
impprevapp2

[('AMT_DOWN_PAYMENT', 'sum'),
 ('DAYS_LAST_DUE_1ST_VERSION', 'sum'),
 ('NAME_CONTRACT_STATUS_Approved', 'mean'),
 ('NAME_CONTRACT_STATUS_Refused', 'mean'),
 ('CODE_REJECT_REASON_XAP', 'mean')]

### Pos cash

In [38]:
poscash=pd.read_csv('POS_CASH_balance.csv')

In [39]:
poscash.drop('SK_ID_PREV',axis=1,inplace=True)
poscash=pd.get_dummies(poscash)
poscash.head()

,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Amortized debt,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Returned to the store,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_XNA
0,182943,-31,48.0,45.0,0,0,1,0,0,0,0,0,0,0,0
1,367990,-33,36.0,35.0,0,0,1,0,0,0,0,0,0,0,0
2,397406,-32,12.0,9.0,0,0,1,0,0,0,0,0,0,0,0
3,269225,-35,48.0,42.0,0,0,1,0,0,0,0,0,0,0,0
4,334279,-35,36.0,35.0,0,0,1,0,0,0,0,0,0,0,0


In [40]:
poscash=poscash.groupby('SK_ID_CURR').agg(['sum','count','min','max','mean'])
poscash.reset_index(inplace=True)
poscash.head()

SK_ID_CURR MONTHS_BALANCE                          CNT_INSTALMENT        \
                        sum count min max       mean            sum count   
0     100001           -653     9 -96 -53 -72.555556           36.0     9   
1     100002           -190    19 -19  -1 -10.000000          456.0    19   
2     100003          -1226    28 -77 -18 -43.785714          283.0    28   
3     100004           -102     4 -27 -24 -25.500000           15.0     4   
4     100005           -220    11 -25 -15 -20.000000          117.0    10   

                         CNT_INSTALMENT_FUTURE                              \
    min   max       mean                   sum count  min   max       mean   
0   4.0   4.0   4.000000                  13.0     9  0.0   4.0   1.444444   
1  24.0  24.0  24.000000                 285.0    19  6.0  24.0  15.000000   
2   6.0  12.0  10.107143                 162.0    28  0.0  12.0   5.785714   
3   3.0   4.0   3.750000                   9.0     4  0.0   4.0   2.250000   
4   9.0  12.0  11.700000                  72.0    10  0.0  12.0   7.200000   

  SK_DPD                         SK_DPD_DEF                          \
     sum count min max      mean        sum count min max      mean   
0      7     9   0   7  0.777778          7     9   0   7  0.777778   
1      0    19   0   0  0.000000          0    19   0   0  0.000000   
2      0    28   0   0  0.000000          0    28   0   0  0.000000   
3      0     4   0   0  0.000000          0     4   0   0  0.000000   
4      0    11   0   0  0.000000          0    11   0   0  0.000000   

  NAME_CONTRACT_STATUS_Active                          \
                          sum count min max      mean   
0                         7.0     9   0   1  0.777778   
1                        19.0    19   1   1  1.000000   
2                        26.0    28   0   1  0.928571   
3                         3.0     4   0   1  0.750000   
4                         9.0    11   0   1  0.818182   

  NAME_CONTRACT_STATUS_Amortized debt                     \
                                  sum count min max mean   
0                                   0     9   0   0  0.0   
1                                   0    19   0   0  0.0   
2                                   0    28   0   0  0.0   
3                                   0     4   0   0  0.0   
4                                   0    11   0   0  0.0   

  NAME_CONTRACT_STATUS_Approved                     \
                            sum count min max mean   
0                             0     9   0   0  0.0   
1                             0    19   0   0  0.0   
2                             0    28   0   0  0.0   
3                             0     4   0   0  0.0   
4                             0    11   0   0  0.0   

  NAME_CONTRACT_STATUS_Canceled                     \
                            sum count min max mean   
0                             0     9   0   0  0.0   
1                             0    19   0   0  0.0   
2                             0    28   0   0  0.0   
3                             0     4   0   0  0.0   
4                             0    11   0   0  0.0   

  NAME_CONTRACT_STATUS_Completed                          \
                             sum count min max      mean   
0                              2     9   0   1  0.222222   
1                              0    19   0   0  0.000000   
2                              2    28   0   1  0.071429   
3                              1     4   0   1  0.250000   
4                              1    11   0   1  0.090909   

  NAME_CONTRACT_STATUS_Demand                     \
                          sum count min max mean   
0                           0     9   0   0  0.0   
1                           0    19   0   0  0.0   
2                           0    28   0   0  0.0   
3                           0     4   0   0  0.0   
4                           0    11   0   0  0.0   

  NAME_CONTRACT_STATUS_Returned to the store        

In [41]:
poscashtest=test.merge(poscash,'left','SK_ID_CURR')
poscashtest.dropna(subset=['TARGET'],inplace=True)
poscashtest.head()

,TARGET,SK_ID_CURR,"(MONTHS_BALANCE, sum)","(MONTHS_BALANCE, count)","(MONTHS_BALANCE, min)","(MONTHS_BALANCE, max)","(MONTHS_BALANCE, mean)","(CNT_INSTALMENT, sum)","(CNT_INSTALMENT, count)","(CNT_INSTALMENT, min)","(CNT_INSTALMENT, max)","(CNT_INSTALMENT, mean)","(CNT_INSTALMENT_FUTURE, sum)","(CNT_INSTALMENT_FUTURE, count)","(CNT_INSTALMENT_FUTURE, min)","(CNT_INSTALMENT_FUTURE, max)","(CNT_INSTALMENT_FUTURE, mean)","(SK_DPD, sum)","(SK_DPD, count)","(SK_DPD, min)","(SK_DPD, max)","(SK_DPD, mean)","(SK_DPD_DEF, sum)","(SK_DPD_DEF, count)","(SK_DPD_DEF, min)","(SK_DPD_DEF, max)","(SK_DPD_DEF, mean)","(NAME_CONTRACT_STATUS_Active, sum)","(NAME_CONTRACT_STATUS_Active, count)","(NAME_CONTRACT_STATUS_Active, min)","(NAME_CONTRACT_STATUS_Active, max)","(NAME_CONTRACT_STATUS_Active, mean)","(NAME_CONTRACT_STATUS_Amortized debt, sum)","(NAME_CONTRACT_STATUS_Amortized debt, count)","(NAME_CONTRACT_STATUS_Amortized debt, min)","(NAME_CONTRACT_STATUS_Amortized debt, max)","(NAME_CONTRACT_STATUS_Amortized debt, mean)","(NAME_CONTRACT_STATUS_Approved, sum)","(NAME_CONTRACT_STATUS_Approved, count)","(NAME_CONTRACT_STATUS_Approved, min)","(NAME_CONTRACT_STATUS_Approved, max)","(NAME_CONTRACT_STATUS_Approved, mean)","(NAME_CONTRACT_STATUS_Canceled, sum)","(NAME_CONTRACT_STATUS_Canceled, count)","(NAME_CONTRACT_STATUS_Canceled, min)","(NAME_CONTRACT_STATUS_Canceled, max)","(NAME_CONTRACT_STATUS_Canceled, mean)","(NAME_CONTRACT_STATUS_Completed, sum)","(NAME_CONTRACT_STATUS_Completed, count)","(NAME_CONTRACT_STATUS_Completed, min)","(NAME_CONTRACT_STATUS_Completed, max)","(NAME_CONTRACT_STATUS_Completed, mean)","(NAME_CONTRACT_STATUS_Demand, sum)","(NAME_CONTRACT_STATUS_Demand, count)","(NAME_CONTRACT_STATUS_Demand, min)","(NAME_CONTRACT_STATUS_Demand, max)","(NAME_CONTRACT_STATUS_Demand, mean)","(NAME_CONTRACT_STATUS_Returned to the store, sum)","(NAME_CONTRACT_STATUS_Returned to the store, count)","(NAME_CONTRACT_STATUS_Returned to the store, min)","(NAME_CONTRACT_STATUS_Returned to the store, max)","(NAME_CONTRACT_STATUS_Returned to the store, mean)","(NAME_CONTRACT_STATUS_Signed, sum)","(NAME_CONTRACT_STATUS_Signed, count)","(NAME_CONTRACT_STATUS_Signed, min)","(NAME_CONTRACT_STATUS_Signed, max)","(NAME_CONTRACT_STATUS_Signed, mean)","(NAME_CONTRACT_STATUS_XNA, sum)","(NAME_CONTRACT_STATUS_XNA, count)","(NAME_CONTRACT_STATUS_XNA, min)","(NAME_CONTRACT_STATUS_XNA, max)","(NAME_CONTRACT_STATUS_XNA, mean)"
0,0,384575,-1485.0,23.0,-75.0,-54.0,-64.565217,256.0,23.0,10.0,12.0,11.130435,133.0,23.0,0.0,12.0,5.782609,0.0,23.0,0.0,0.0,0.000000,0.0,23.0,0.0,0.0,0.0,22.0,23.0,0.0,1.0,0.956522,0.0,23.0,0.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0,1.0,23.0,0.0,1.0,0.043478,0.0,23.0,0.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0,23.0,0.0,0.0,0.000000,0.0,23.0,0.0,0.0,0.0
1,0,214010,-2067.0,44.0,-79.0,-24.0,-46.977273,333.0,44.0,1.0,12.0,7.568182,195.0,44.0,0.0,12.0,4.431818,0.0,44.0,0.0,0.0,0.000000,0.0,44.0,0.0,0.0,0.0,36.0,44.0,0.0,1.0,0.818182,0.0,44.0,0.0,0.0,0.0,0.0,44.0,0.0,0.0,0.0,0.0,44.0,0.0,0.0,0.0,7.0,44.0,0.0,1.0,0.159091,0.0,44.0,0.0,0.0,0.0,0.0,44.0,0.0,0.0,0.0,1.0,44.0,0.0,1.0,0.022727,0.0,44.0,0.0,0.0,0.0
2,0,142232,-535.0,21.0,-47.0,-5.0,-25.476190,190.0,21.0,8.0,10.0,9.047619,95.0,21.0,0.0,10.0,4.523810,3.0,21.0,0.0,3.0,0.142857,0.0,21.0,0.0,0.0,0.0,18.0,21.0,0.0,1.0,0.857143,0.0,21.0,0.0,0.0,0.0,0.0,21.0,0.0,0.0,0.0,0.0,21.0,0.0,0.0,0.0,2.0,21.0,0.0,1.0,0.095238,0.0,21.0,0.0,0.0,0.0,0.0,21.0,0.0,0.0,0.0,1.0,21.0,0.0,1.0,0.047619,0.0,21.0,0.0,0.0,0.0
3,0,389171,-1350.0,25.0,-66.0,-42.0,-54.000000,300.0,25.0,12.0,12.0,12.000000,78.0,25.0,0.0,12.0,3.120000,1833.0,25.0,0.0,320.0,73.320000,0.0,25.0,0.0,0.0,0.0,24.0,25.0,0.0,1.0,0.960000,0.0,25.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,1.0,25.0,0.0,1.0,0.040000,0.0,25.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.000000,0.0,25.0,0.0,0.0,0.0
4,0,283617,-1026.0,26.0,-60.0,-4.0,-39.461538,245.0,26.0,2.0,12.0,9.423077,148.0,26.0,0.0,12.0,5.692308,0.0,26.0,0.0,0.0,0.000000,

In [42]:
poscash=targ.merge(poscash,'left','SK_ID_CURR')
poscash.dropna(subset=['TARGET'],inplace=True)
poscash.head()

,TARGET,SK_ID_CURR,"(MONTHS_BALANCE, sum)","(MONTHS_BALANCE, count)","(MONTHS_BALANCE, min)","(MONTHS_BALANCE, max)","(MONTHS_BALANCE, mean)","(CNT_INSTALMENT, sum)","(CNT_INSTALMENT, count)","(CNT_INSTALMENT, min)","(CNT_INSTALMENT, max)","(CNT_INSTALMENT, mean)","(CNT_INSTALMENT_FUTURE, sum)","(CNT_INSTALMENT_FUTURE, count)","(CNT_INSTALMENT_FUTURE, min)","(CNT_INSTALMENT_FUTURE, max)","(CNT_INSTALMENT_FUTURE, mean)","(SK_DPD, sum)","(SK_DPD, count)","(SK_DPD, min)","(SK_DPD, max)","(SK_DPD, mean)","(SK_DPD_DEF, sum)","(SK_DPD_DEF, count)","(SK_DPD_DEF, min)","(SK_DPD_DEF, max)","(SK_DPD_DEF, mean)","(NAME_CONTRACT_STATUS_Active, sum)","(NAME_CONTRACT_STATUS_Active, count)","(NAME_CONTRACT_STATUS_Active, min)","(NAME_CONTRACT_STATUS_Active, max)","(NAME_CONTRACT_STATUS_Active, mean)","(NAME_CONTRACT_STATUS_Amortized debt, sum)","(NAME_CONTRACT_STATUS_Amortized debt, count)","(NAME_CONTRACT_STATUS_Amortized debt, min)","(NAME_CONTRACT_STATUS_Amortized debt, max)","(NAME_CONTRACT_STATUS_Amortized debt, mean)","(NAME_CONTRACT_STATUS_Approved, sum)","(NAME_CONTRACT_STATUS_Approved, count)","(NAME_CONTRACT_STATUS_Approved, min)","(NAME_CONTRACT_STATUS_Approved, max)","(NAME_CONTRACT_STATUS_Approved, mean)","(NAME_CONTRACT_STATUS_Canceled, sum)","(NAME_CONTRACT_STATUS_Canceled, count)","(NAME_CONTRACT_STATUS_Canceled, min)","(NAME_CONTRACT_STATUS_Canceled, max)","(NAME_CONTRACT_STATUS_Canceled, mean)","(NAME_CONTRACT_STATUS_Completed, sum)","(NAME_CONTRACT_STATUS_Completed, count)","(NAME_CONTRACT_STATUS_Completed, min)","(NAME_CONTRACT_STATUS_Completed, max)","(NAME_CONTRACT_STATUS_Completed, mean)","(NAME_CONTRACT_STATUS_Demand, sum)","(NAME_CONTRACT_STATUS_Demand, count)","(NAME_CONTRACT_STATUS_Demand, min)","(NAME_CONTRACT_STATUS_Demand, max)","(NAME_CONTRACT_STATUS_Demand, mean)","(NAME_CONTRACT_STATUS_Returned to the store, sum)","(NAME_CONTRACT_STATUS_Returned to the store, count)","(NAME_CONTRACT_STATUS_Returned to the store, min)","(NAME_CONTRACT_STATUS_Returned to the store, max)","(NAME_CONTRACT_STATUS_Returned to the store, mean)","(NAME_CONTRACT_STATUS_Signed, sum)","(NAME_CONTRACT_STATUS_Signed, count)","(NAME_CONTRACT_STATUS_Signed, min)","(NAME_CONTRACT_STATUS_Signed, max)","(NAME_CONTRACT_STATUS_Signed, mean)","(NAME_CONTRACT_STATUS_XNA, sum)","(NAME_CONTRACT_STATUS_XNA, count)","(NAME_CONTRACT_STATUS_XNA, min)","(NAME_CONTRACT_STATUS_XNA, max)","(NAME_CONTRACT_STATUS_XNA, mean)"
0,0,252724,-672.0,19.0,-47.0,-19.0,-35.368421,196.0,19.0,4.0,12.0,10.315789,96.0,19.0,0.0,12.0,5.052632,0.0,19.0,0.0,0.0,0.000000,0.0,19.0,0.0,0.0,0.0,17.0,19.0,0.0,1.0,0.894737,0.0,19.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,2.0,19.0,0.0,1.0,0.105263,0.0,19.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0
1,0,372834,-130.0,5.0,-28.0,-24.0,-26.000000,20.0,5.0,4.0,4.0,4.000000,10.0,5.0,0.0,4.0,2.000000,0.0,5.0,0.0,0.0,0.000000,0.0,5.0,0.0,0.0,0.0,4.0,5.0,0.0,1.0,0.800000,0.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,1.0,5.0,0.0,1.0,0.200000,0.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
2,1,326336,-480.0,15.0,-44.0,-5.0,-32.000000,181.0,15.0,2.0,18.0,12.066667,112.0,15.0,0.0,18.0,7.466667,8.0,15.0,0.0,8.0,0.533333,0.0,15.0,0.0,0.0,0.0,13.0,15.0,0.0,1.0,0.866667,0.0,15.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,2.0,15.0,0.0,1.0,0.133333,0.0,15.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0
3,0,382390,-559.0,19.0,-74.0,-4.0,-29.421053,192.0,19.0,6.0,12.0,10.105263,99.0,19.0,0.0,12.0,5.210526,0.0,19.0,0.0,0.0,0.000000,0.0,19.0,0.0,0.0,0.0,18.0,19.0,0.0,1.0,0.947368,0.0,19.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,1.0,19.0,0.0,1.0,0.052632,0.0,19.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0
4,0,330511,-320.0,15.0,-29.0,-13.0,-21.333333,139.0,15.0,4.0,12.0,9.266667,84.0,15.0,0.0,12.0,5.600000,0.0,15.0,0.0,0.0,0.000000,0.0,15.0,0.0,0.0,0.0,13.0,15.0,0.0,1.0,0.866667,0.0

In [43]:
iv={}
for i in poscash.drop('TARGET',axis=1).columns:
    ct=woe(poscash,'TARGET',i)
    iv[i]=sum(ct['IV'])

D:\Software\Anaconda\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Software\Anaconda\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [44]:
impposcash1=[]
for i,j in iv.items():
    if j>=0.03:
        impposcash1.append(i)
impposcash1

[('MONTHS_BALANCE', 'sum'),
 ('MONTHS_BALANCE', 'count'),
 ('MONTHS_BALANCE', 'min'),
 ('CNT_INSTALMENT', 'count'),
 ('CNT_INSTALMENT_FUTURE', 'count'),
 ('SK_DPD', 'count'),
 ('SK_DPD_DEF', 'count'),
 ('NAME_CONTRACT_STATUS_Active', 'sum'),
 ('NAME_CONTRACT_STATUS_Active', 'count'),
 ('NAME_CONTRACT_STATUS_Amortized debt', 'count'),
 ('NAME_CONTRACT_STATUS_Approved', 'count'),
 ('NAME_CONTRACT_STATUS_Canceled', 'count'),
 ('NAME_CONTRACT_STATUS_Completed', 'count'),
 ('NAME_CONTRACT_STATUS_Demand', 'count'),
 ('NAME_CONTRACT_STATUS_Returned to the store', 'count'),
 ('NAME_CONTRACT_STATUS_Signed', 'count'),
 ('NAME_CONTRACT_STATUS_XNA', 'count')]

In [45]:
impposcash2=[]
for i,j in iv.items():
    if j>=0.05:
        impposcash2.append(i)
impposcash2

[]

### Inst Payments

In [52]:
instpaym=pd.read_csv('installments_payments.csv')

In [53]:
instpaym.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [54]:
instpaym.drop('SK_ID_PREV',axis=1,inplace=True)
instpaym=pd.get_dummies(instpaym)
instpaym.head()

,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [55]:
instpaym=instpaym.groupby('SK_ID_CURR').agg(['sum','count','min','max','mean'])
instpaym.reset_index(inplace=True)
instpaym.head()

SK_ID_CURR NUM_INSTALMENT_VERSION                            \
                                sum count  min  max      mean   
0     100001                    8.0     7  1.0  2.0  1.142857   
1     100002                   20.0    19  1.0  2.0  1.052632   
2     100003                   26.0    25  1.0  2.0  1.040000   
3     100004                    4.0     3  1.0  2.0  1.333333   
4     100005                   10.0     9  1.0  2.0  1.111111   

  NUM_INSTALMENT_NUMBER                          DAYS_INSTALMENT        \
                    sum count min max       mean             sum count   
0                    19     7   1   4   2.714286        -15314.0     7   
1                   190    19   1  19  10.000000         -5605.0    19   
2                   127    25   1  12   5.080000        -34454.0    25   
3                     6     3   1   3   2.000000         -2262.0     3   
4                    45     9   1   9   5.000000         -5274.0     9   

                               DAYS_ENTRY_PAYMENT                        \
      min     max         mean                sum count     min     max   
0 -2916.0 -1619.0 -2187.714286           -15365.0     7 -2916.0 -1628.0   
1  -565.0   -25.0  -295.000000            -5993.0    19  -587.0   -49.0   
2 -2310.0  -536.0 -1378.160000           -34633.0    25 -2324.0  -544.0   
3  -784.0  -724.0  -754.000000            -2285.0     3  -795.0  -727.0   
4  -706.0  -466.0  -586.000000            -5486.0     9  -736.0  -470.0   

               AMT_INSTALMENT                                            \
          mean            sum count       min         max          mean   
0 -2195.000000      41195.925     7  3951.000   17397.900   5885.132143   
1  -315.421053     219625.695    19  9251.775   53093.745  11559.247105   
2 -1385.320000    1618864.650    25  6662.970  560835.360  64754.586000   
3  -761.666667      21288.465     3  5357.250   10573.965   7096.155000   
4  -609.555556      56161.845     9  4813.200   17656.245   6240.205000   

   AMT_PAYMENT                                            
           sum count       min         max          mean  
0    41195.925     7  3951.000   17397.900   5885.132143  
1   219625.695    19  9251.775   53093.745  11559.247105  
2  1618864.650    25  6662.970  560835.360  64754.586000  
3    21288.465     3  5357.250   10573.965   7096.155000  
4    56161.845     9  4813.200   17656.245   6240.205000

In [56]:
instpaymtest=test.merge(instpaym,'left','SK_ID_CURR')
instpaymtest.dropna(subset=['TARGET'],inplace=True)
instpaymtest.head()

,TARGET,SK_ID_CURR,"(NUM_INSTALMENT_VERSION, sum)","(NUM_INSTALMENT_VERSION, count)","(NUM_INSTALMENT_VERSION, min)","(NUM_INSTALMENT_VERSION, max)","(NUM_INSTALMENT_VERSION, mean)","(NUM_INSTALMENT_NUMBER, sum)","(NUM_INSTALMENT_NUMBER, count)","(NUM_INSTALMENT_NUMBER, min)","(NUM_INSTALMENT_NUMBER, max)","(NUM_INSTALMENT_NUMBER, mean)","(DAYS_INSTALMENT, sum)","(DAYS_INSTALMENT, count)","(DAYS_INSTALMENT, min)","(DAYS_INSTALMENT, max)","(DAYS_INSTALMENT, mean)","(DAYS_ENTRY_PAYMENT, sum)","(DAYS_ENTRY_PAYMENT, count)","(DAYS_ENTRY_PAYMENT, min)","(DAYS_ENTRY_PAYMENT, max)","(DAYS_ENTRY_PAYMENT, mean)","(AMT_INSTALMENT, sum)","(AMT_INSTALMENT, count)","(AMT_INSTALMENT, min)","(AMT_INSTALMENT, max)","(AMT_INSTALMENT, mean)","(AMT_PAYMENT, sum)","(AMT_PAYMENT, count)","(AMT_PAYMENT, min)","(AMT_PAYMENT, max)","(AMT_PAYMENT, mean)"
0,0,384575,29.0,29.0,1.0,1.0,1.000000,177.0,29.0,1.0,12.0,6.103448,-55317.0,29.0,-2246.0,-1633.0,-1907.482759,-55548.0,29.0,-2254.0,-1627.0,-1915.448276,120875.805,29.0,3019.455,4776.840,4168.131207,87492.060,29.0,0.045,4776.840,3016.967586
1,0,214010,43.0,70.0,0.0,2.0,0.614286,739.0,70.0,1.0,34.0,10.557143,-61453.0,70.0,-2375.0,-34.0,-877.900000,-62114.0,70.0,-2397.0,-34.0,-887.342857,3372231.780,70.0,83.655,962456.985,48174.739714,3354329.520,70.0,83.655,962456.985,47918.993143
2,0,142232,19.0,19.0,1.0,1.0,1.000000,93.0,19.0,1.0,10.0,4.894737,-13102.0,19.0,-1368.0,-128.0,-689.578947,-13452.0,19.0,-1387.0,-144.0,-708.000000,238677.750,19.0,9610.875,14707.035,12561.986842,223970.715,19.0,68.535,14707.035,11787.932368
3,0,389171,13.0,13.0,1.0,1.0,1.000000,90.0,13.0,1.0,12.0,6.923077,-23287.0,13.0,-1969.0,-1639.0,-1791.307692,-22902.0,13.0,-1964.0,-1295.0,-1761.692308,145935.810,13.0,11223.090,11240.910,11225.831538,134694.900,13.0,17.820,11223.090,10361.146154
4,0,283617,32.0,29.0,1.0,2.0,1.103448,165.0,29.0,1.0,12.0,5.689655,-35285.0,29.0,-1770.0,-113.0,-1216.724138,-35669.0,29.0,-1787.0,-116.0,-1229.965517,661535.730,29.0,4458.375,242226.585,22811.576897,554478.570,29.0,4.995,242226.585,19119.950690


In [57]:
instpaym=targ.merge(instpaym,'left','SK_ID_CURR')
instpaym.dropna(subset=['TARGET'],inplace=True)
instpaym.head()

,TARGET,SK_ID_CURR,"(NUM_INSTALMENT_VERSION, sum)","(NUM_INSTALMENT_VERSION, count)","(NUM_INSTALMENT_VERSION, min)","(NUM_INSTALMENT_VERSION, max)","(NUM_INSTALMENT_VERSION, mean)","(NUM_INSTALMENT_NUMBER, sum)","(NUM_INSTALMENT_NUMBER, count)","(NUM_INSTALMENT_NUMBER, min)","(NUM_INSTALMENT_NUMBER, max)","(NUM_INSTALMENT_NUMBER, mean)","(DAYS_INSTALMENT, sum)","(DAYS_INSTALMENT, count)","(DAYS_INSTALMENT, min)","(DAYS_INSTALMENT, max)","(DAYS_INSTALMENT, mean)","(DAYS_ENTRY_PAYMENT, sum)","(DAYS_ENTRY_PAYMENT, count)","(DAYS_ENTRY_PAYMENT, min)","(DAYS_ENTRY_PAYMENT, max)","(DAYS_ENTRY_PAYMENT, mean)","(AMT_INSTALMENT, sum)","(AMT_INSTALMENT, count)","(AMT_INSTALMENT, min)","(AMT_INSTALMENT, max)","(AMT_INSTALMENT, mean)","(AMT_PAYMENT, sum)","(AMT_PAYMENT, count)","(AMT_PAYMENT, min)","(AMT_PAYMENT, max)","(AMT_PAYMENT, mean)"
0,0,252724,17.0,16.0,1.0,2.0,1.062500,88.0,16.0,1.0,12.0,5.500000,-16876.0,16.0,-1371.0,-556.0,-1054.750000,-16954.0,16.0,-1375.0,-556.0,-1059.625000,229164.345,16.0,5852.160,16970.715,14322.771563,229164.345,16.0,5852.160,16970.715,14322.771563
1,0,372834,5.0,5.0,1.0,1.0,1.000000,14.0,5.0,1.0,4.0,2.800000,-3835.0,5.0,-821.0,-731.0,-767.000000,-3838.0,5.0,-830.0,-720.0,-767.600000,105379.605,5.0,21066.120,21082.455,21075.921000,84313.485,5.0,95.985,21082.455,16862.697000
2,1,326336,16.0,14.0,1.0,2.0,1.142857,77.0,14.0,1.0,11.0,5.500000,-13884.0,14.0,-1286.0,-141.0,-991.714286,-14102.0,14.0,-1305.0,-159.0,-1007.285714,183790.665,14.0,6118.740,81026.370,13127.904643,176320.035,14.0,212.040,81026.370,12594.288214
3,0,382390,23.0,23.0,1.0,1.0,1.000000,142.0,23.0,1.0,12.0,6.173913,-17383.0,23.0,-2220.0,-119.0,-755.782609,-17551.0,23.0,-2237.0,-113.0,-763.086957,533772.360,23.0,10001.790,27860.445,23207.493913,394492.095,23.0,529.110,27860.445,17151.830217
4,0,330511,21.0,13.0,1.0,3.0,1.615385,55.0,13.0,1.0,9.0,4.230769,-8289.0,13.0,-841.0,-405.0,-637.615385,-8477.0,13.0,-859.0,-410.0,-652.076923,259577.190,13.0,9171.045,55801.125,19967.476154,259577.190,13.0,9171.045,55801.125,19967.476154


In [58]:
iv={}
for i in instpaym.drop('TARGET',axis=1).columns:
    ct=woe(instpaym,'TARGET',i)
    iv[i]=sum(ct['IV'])

D:\Software\Anaconda\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Software\Anaconda\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [59]:
impinstpaym1=[]
for i,j in iv.items():
    if j>=0.03:
        impinstpaym1.append(i)
impinstpaym1

[('DAYS_INSTALMENT', 'sum'),
 ('DAYS_INSTALMENT', 'min'),
 ('DAYS_INSTALMENT', 'mean'),
 ('DAYS_ENTRY_PAYMENT', 'sum'),
 ('DAYS_ENTRY_PAYMENT', 'min'),
 ('DAYS_ENTRY_PAYMENT', 'mean'),
 ('AMT_INSTALMENT', 'min'),
 ('AMT_INSTALMENT', 'max'),
 ('AMT_PAYMENT', 'sum'),
 ('AMT_PAYMENT', 'min'),
 ('AMT_PAYMENT', 'max'),
 ('AMT_PAYMENT', 'mean')]

In [60]:
impinstpaym2=[]
for i,j in iv.items():
    if j>=0.05:
        impinstpaym2.append(i)
impinstpaym2

[('DAYS_INSTALMENT', 'min'), ('DAYS_ENTRY_PAYMENT', 'min')]

### CCbalance

In [61]:
ccbalance=pd.read_csv('credit_card_balance.csv')

In [62]:
ccbalance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.000,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.080,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,2250.0,26926.425,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,11925.0,224949.285,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,27000.0,443044.395,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [63]:
ccbalance.drop('SK_ID_PREV',axis=1,inplace=True)
ccbalance=pd.get_dummies(ccbalance)
ccbalance.head()

,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Signed
0,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.000,0.000,0.000,0.0,1,0.0,1.0,35.0,0,0,1,0,0,0,0,0,0
1,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.080,64875.555,64875.555,1.0,1,0.0,0.0,69.0,0,0,1,0,0,0,0,0,0
2,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,2250.0,26926.425,31460.085,31460.085,0.0,0,0.0,0.0,30.0,0,0,1,0,0,0,0,0,0
3,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,11925.0,224949.285,233048.970,233048.970,1.0,1,0.0,0.0,10.0,0,0,1,0,0,0,0,0,0
4,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,27000.0,443044.395,453919.455,453919.455,0.0,1,0.0,1.0,101.0,0,0,1,0,0,0,0,0,0


In [64]:
ccbalance=ccbalance.groupby('SK_ID_CURR').agg(['sum','count','min','max','mean'])
ccbalance.reset_index(inplace=True)
ccbalance.head()

SK_ID_CURR MONTHS_BALANCE                      AMT_BALANCE             \
                        sum count min max  mean          sum count  min   
0     100006            -21     6  -6  -1  -3.5        0.000     6  0.0   
1     100011          -2849    74 -75  -2 -38.5  4031676.225    74  0.0   
2     100013          -4656    96 -96  -1 -48.5  1743352.245    96  0.0   
3     100021           -170    17 -18  -2 -10.0        0.000    17  0.0   
4     100023            -60     8 -11  -4  -7.5        0.000     8  0.0   

                           AMT_CREDIT_LIMIT_ACTUAL                        \
         max          mean                     sum count     min     max   
0       0.00      0.000000                 1620000     6  270000  270000   
1  189000.00  54482.111149                12150000    74   90000  180000   
2  161420.22  18159.919219                12645000    96   45000  157500   
3       0.00      0.000000                11475000    17  675000  675000   
4       0.00      0.000000                 1080000     8   45000  225000   

                 AMT_DRAWINGS_ATM_CURRENT                                    \
            mean                      sum count  min       max         mean   
0  270000.000000                      0.0     0  NaN       NaN          NaN   
1  164189.189189                 180000.0    74  0.0  180000.0  2432.432432   
2  131718.750000                 571500.0    90  0.0  157500.0  6350.000000   
3  675000.000000                      0.0     0  NaN       NaN          NaN   
4  135000.000000                      0.0     0  NaN       NaN          NaN   

  AMT_DRAWINGS_CURRENT                                    \
                   sum count  min       max         mean   
0                  0.0     6  0.0       0.0     0.000000   
1             180000.0    74  0.0  180000.0  2432.432432   
2             571500.0    96  0.0  157500.0  5953.125000   
3                  0.0    17  0.0       0.0     0.000000   
4                  0.0     8  0.0       0.0     0.000000   

  AMT_DRAWINGS_OTHER_CURRENT                      AMT_DRAWINGS_POS_CURRENT  \
                         sum count  min  max mean                      sum   
0                        0.0     0  NaN  NaN  NaN                      0.0   
1                        0.0    74  0.0  0.0  0.0                      0.0   
2                        0.0    90  0.0  0.0  0.0                      0.0   
3                        0.0     0  NaN  NaN  NaN                      0.0   
4                        0.0     0  NaN  NaN  NaN                      0.0   

                       AMT_INST_MIN_REGULARITY                     \
  count  min  max mean                     sum count  min     max   
0     0  NaN  NaN  NaN                   0.000     6  0.0     0.0   
1    74  0.0  0.0  0.0              288804.195    73  0.0  9000.0   
2    90  0.0  0.0  0.0              129454.020    89  0.0  7875.0   
3     0  NaN  NaN  NaN                   0.000    17  0.0     0.0   
4     0  NaN  NaN  NaN                   0.000     8  0.0     0.0   

               AMT_PAYMENT_CURRENT                                    \
          mean                 sum count  min       max         mean   
0     0.000000                0.00     0  NaN       NaN          NaN   
1  3956.221849           358386.75    74  0.0   55485.0  4843.064189   
2  1454.539551           688161.24    96  0.0  153675.0  7168.346250   
3     0.000000                0.00     0  NaN       NaN          NaN   
4     0.000000                0.00     0  NaN       NaN          NaN   

  AMT_PAYMENT_TOTAL_CURRENT                                    \
                        sum count  min       max         mean   
0                     0.000     6  0.0       0.0     0.000000   
1                334485.000    74  0.0   55485.0  4520.067568   
2                654448.545    96  0.0  153675.0  6817.172344   
3                     0.000    17  0.0       0.0     0.000000   
4                     0.000     8  0.0    

In [65]:
ccbalancetest=test.merge(ccbalance,'left','SK_ID_CURR')
ccbalancetest.dropna(subset=['TARGET'],inplace=True)
ccbalancetest.head()

,TARGET,SK_ID_CURR,"(MONTHS_BALANCE, sum)","(MONTHS_BALANCE, count)","(MONTHS_BALANCE, min)","(MONTHS_BALANCE, max)","(MONTHS_BALANCE, mean)","(AMT_BALANCE, sum)","(AMT_BALANCE, count)","(AMT_BALANCE, min)","(AMT_BALANCE, max)","(AMT_BALANCE, mean)","(AMT_CREDIT_LIMIT_ACTUAL, sum)","(AMT_CREDIT_LIMIT_ACTUAL, count)","(AMT_CREDIT_LIMIT_ACTUAL, min)","(AMT_CREDIT_LIMIT_ACTUAL, max)","(AMT_CREDIT_LIMIT_ACTUAL, mean)","(AMT_DRAWINGS_ATM_CURRENT, sum)","(AMT_DRAWINGS_ATM_CURRENT, count)","(AMT_DRAWINGS_ATM_CURRENT, min)","(AMT_DRAWINGS_ATM_CURRENT, max)","(AMT_DRAWINGS_ATM_CURRENT, mean)","(AMT_DRAWINGS_CURRENT, sum)","(AMT_DRAWINGS_CURRENT, count)","(AMT_DRAWINGS_CURRENT, min)","(AMT_DRAWINGS_CURRENT, max)","(AMT_DRAWINGS_CURRENT, mean)","(AMT_DRAWINGS_OTHER_CURRENT, sum)","(AMT_DRAWINGS_OTHER_CURRENT, count)","(AMT_DRAWINGS_OTHER_CURRENT, min)","(AMT_DRAWINGS_OTHER_CURRENT, max)","(AMT_DRAWINGS_OTHER_CURRENT, mean)","(AMT_DRAWINGS_POS_CURRENT, sum)","(AMT_DRAWINGS_POS_CURRENT, count)","(AMT_DRAWINGS_POS_CURRENT, min)","(AMT_DRAWINGS_POS_CURRENT, max)","(AMT_DRAWINGS_POS_CURRENT, mean)","(AMT_INST_MIN_REGULARITY, sum)","(AMT_INST_MIN_REGULARITY, count)","(AMT_INST_MIN_REGULARITY, min)","(AMT_INST_MIN_REGULARITY, max)","(AMT_INST_MIN_REGULARITY, mean)","(AMT_PAYMENT_CURRENT, sum)","(AMT_PAYMENT_CURRENT, count)","(AMT_PAYMENT_CURRENT, min)","(AMT_PAYMENT_CURRENT, max)","(AMT_PAYMENT_CURRENT, mean)","(AMT_PAYMENT_TOTAL_CURRENT, sum)","(AMT_PAYMENT_TOTAL_CURRENT, count)","(AMT_PAYMENT_TOTAL_CURRENT, min)","(AMT_PAYMENT_TOTAL_CURRENT, max)","(AMT_PAYMENT_TOTAL_CURRENT, mean)","(AMT_RECEIVABLE_PRINCIPAL, sum)","(AMT_RECEIVABLE_PRINCIPAL, count)","(AMT_RECEIVABLE_PRINCIPAL, min)","(AMT_RECEIVABLE_PRINCIPAL, max)","(AMT_RECEIVABLE_PRINCIPAL, mean)","(AMT_RECIVABLE, sum)","(AMT_RECIVABLE, count)","(AMT_RECIVABLE, min)","(AMT_RECIVABLE, max)","(AMT_RECIVABLE, mean)",...,"(CNT_DRAWINGS_CURRENT, max)","(CNT_DRAWINGS_CURRENT, mean)","(CNT_DRAWINGS_OTHER_CURRENT, sum)","(CNT_DRAWINGS_OTHER_CURRENT, count)","(CNT_DRAWINGS_OTHER_CURRENT, min)","(CNT_DRAWINGS_OTHER_CURRENT, max)","(CNT_DRAWINGS_OTHER_CURRENT, mean)","(CNT_DRAWINGS_POS_CURRENT, sum)","(CNT_DRAWINGS_POS_CURRENT, count)","(CNT_DRAWINGS_POS_CURRENT, min)","(CNT_DRAWINGS_POS_CURRENT, max)","(CNT_DRAWINGS_POS_CURRENT, mean)","(CNT_INSTALMENT_MATURE_CUM, sum)","(CNT_INSTALMENT_MATURE_CUM, count)","(CNT_INSTALMENT_MATURE_CUM, min)","(CNT_INSTALMENT_MATURE_CUM, max)","(CNT_INSTALMENT_MATURE_CUM, mean)","(SK_DPD, sum)","(SK_DPD, count)","(SK_DPD, min)","(SK_DPD, max)","(SK_DPD, mean)","(SK_DPD_DEF, sum)","(SK_DPD_DEF, count)","(SK_DPD_DEF, min)","(SK_DPD_DEF, max)","(SK_DPD_DEF, mean)","(NAME_CONTRACT_STATUS_Active, sum)","(NAME_CONTRACT_STATUS_Active, count)","(NAME_CONTRACT_STATUS_Active, min)","(NAME_CONTRACT_STATUS_Active, max)","(NAME_CONTRACT_STATUS_Active, mean)","(NAME_CONTRACT_STATUS_Approved, sum)","(NAME_CONTRACT_STATUS_Approved, count)","(NAME_CONTRACT_STATUS_Approved, min)","(NAME_CONTRACT_STATUS_Approved, max)","(NAME_CONTRACT_STATUS_Approved, mean)","(NAME_CONTRACT_STATUS_Completed, sum)","(NAME_CONTRACT_STATUS_Completed, count)","(NAME_CONTRACT_STATUS_Completed, min)","(NAME_CONTRACT_STATUS_Completed, max)","(NAME_CONTRACT_STATUS_Completed, mean)","(NAME_CONTRACT_STATUS_Demand, sum)","(NAME_CONTRACT_STATUS_Demand, count)","(NAME_CONTRACT_STATUS_Demand, min)","(NAME_CONTRACT_STATUS_Demand, max)","(NAME_CONTRACT_STATUS_Demand, mean)","(NAME_CONTRACT_STATUS_Refused, sum)","(NAME_CONTRACT_STATUS_Refused, count)","(NAME_CONTRACT_STATUS_Refused, min)","(NAME_CONTRACT_STATUS_Refused, max)","(NAME_CONTRACT_STATUS_Refused, mean)","(NAME_CONTRACT_STATUS_Sent proposal, sum)","(NAME_CONTRACT_STATUS_Sent proposal, count)","(NAME_CONTRACT_STATUS_Sent proposal, min)","(NAME_CONTRACT_STATUS_Sent proposal, max)","(NAME_CONTRACT_STATUS_Sent proposal, mean)","(NAME_CONTRACT_STATUS_Signed, sum)","(NAME_CONTRACT_STATUS_Signed, count)","(NAME_CONTRACT_STATUS_Signed, min)","(NAME_CONTRACT_STATUS_S

In [66]:
ccbalance=targ.merge(ccbalance,'left','SK_ID_CURR')
ccbalance.dropna(subset=['TARGET'],inplace=True)
ccbalance.head()

,TARGET,SK_ID_CURR,"(MONTHS_BALANCE, sum)","(MONTHS_BALANCE, count)","(MONTHS_BALANCE, min)","(MONTHS_BALANCE, max)","(MONTHS_BALANCE, mean)","(AMT_BALANCE, sum)","(AMT_BALANCE, count)","(AMT_BALANCE, min)","(AMT_BALANCE, max)","(AMT_BALANCE, mean)","(AMT_CREDIT_LIMIT_ACTUAL, sum)","(AMT_CREDIT_LIMIT_ACTUAL, count)","(AMT_CREDIT_LIMIT_ACTUAL, min)","(AMT_CREDIT_LIMIT_ACTUAL, max)","(AMT_CREDIT_LIMIT_ACTUAL, mean)","(AMT_DRAWINGS_ATM_CURRENT, sum)","(AMT_DRAWINGS_ATM_CURRENT, count)","(AMT_DRAWINGS_ATM_CURRENT, min)","(AMT_DRAWINGS_ATM_CURRENT, max)","(AMT_DRAWINGS_ATM_CURRENT, mean)","(AMT_DRAWINGS_CURRENT, sum)","(AMT_DRAWINGS_CURRENT, count)","(AMT_DRAWINGS_CURRENT, min)","(AMT_DRAWINGS_CURRENT, max)","(AMT_DRAWINGS_CURRENT, mean)","(AMT_DRAWINGS_OTHER_CURRENT, sum)","(AMT_DRAWINGS_OTHER_CURRENT, count)","(AMT_DRAWINGS_OTHER_CURRENT, min)","(AMT_DRAWINGS_OTHER_CURRENT, max)","(AMT_DRAWINGS_OTHER_CURRENT, mean)","(AMT_DRAWINGS_POS_CURRENT, sum)","(AMT_DRAWINGS_POS_CURRENT, count)","(AMT_DRAWINGS_POS_CURRENT, min)","(AMT_DRAWINGS_POS_CURRENT, max)","(AMT_DRAWINGS_POS_CURRENT, mean)","(AMT_INST_MIN_REGULARITY, sum)","(AMT_INST_MIN_REGULARITY, count)","(AMT_INST_MIN_REGULARITY, min)","(AMT_INST_MIN_REGULARITY, max)","(AMT_INST_MIN_REGULARITY, mean)","(AMT_PAYMENT_CURRENT, sum)","(AMT_PAYMENT_CURRENT, count)","(AMT_PAYMENT_CURRENT, min)","(AMT_PAYMENT_CURRENT, max)","(AMT_PAYMENT_CURRENT, mean)","(AMT_PAYMENT_TOTAL_CURRENT, sum)","(AMT_PAYMENT_TOTAL_CURRENT, count)","(AMT_PAYMENT_TOTAL_CURRENT, min)","(AMT_PAYMENT_TOTAL_CURRENT, max)","(AMT_PAYMENT_TOTAL_CURRENT, mean)","(AMT_RECEIVABLE_PRINCIPAL, sum)","(AMT_RECEIVABLE_PRINCIPAL, count)","(AMT_RECEIVABLE_PRINCIPAL, min)","(AMT_RECEIVABLE_PRINCIPAL, max)","(AMT_RECEIVABLE_PRINCIPAL, mean)","(AMT_RECIVABLE, sum)","(AMT_RECIVABLE, count)","(AMT_RECIVABLE, min)","(AMT_RECIVABLE, max)","(AMT_RECIVABLE, mean)",...,"(CNT_DRAWINGS_CURRENT, max)","(CNT_DRAWINGS_CURRENT, mean)","(CNT_DRAWINGS_OTHER_CURRENT, sum)","(CNT_DRAWINGS_OTHER_CURRENT, count)","(CNT_DRAWINGS_OTHER_CURRENT, min)","(CNT_DRAWINGS_OTHER_CURRENT, max)","(CNT_DRAWINGS_OTHER_CURRENT, mean)","(CNT_DRAWINGS_POS_CURRENT, sum)","(CNT_DRAWINGS_POS_CURRENT, count)","(CNT_DRAWINGS_POS_CURRENT, min)","(CNT_DRAWINGS_POS_CURRENT, max)","(CNT_DRAWINGS_POS_CURRENT, mean)","(CNT_INSTALMENT_MATURE_CUM, sum)","(CNT_INSTALMENT_MATURE_CUM, count)","(CNT_INSTALMENT_MATURE_CUM, min)","(CNT_INSTALMENT_MATURE_CUM, max)","(CNT_INSTALMENT_MATURE_CUM, mean)","(SK_DPD, sum)","(SK_DPD, count)","(SK_DPD, min)","(SK_DPD, max)","(SK_DPD, mean)","(SK_DPD_DEF, sum)","(SK_DPD_DEF, count)","(SK_DPD_DEF, min)","(SK_DPD_DEF, max)","(SK_DPD_DEF, mean)","(NAME_CONTRACT_STATUS_Active, sum)","(NAME_CONTRACT_STATUS_Active, count)","(NAME_CONTRACT_STATUS_Active, min)","(NAME_CONTRACT_STATUS_Active, max)","(NAME_CONTRACT_STATUS_Active, mean)","(NAME_CONTRACT_STATUS_Approved, sum)","(NAME_CONTRACT_STATUS_Approved, count)","(NAME_CONTRACT_STATUS_Approved, min)","(NAME_CONTRACT_STATUS_Approved, max)","(NAME_CONTRACT_STATUS_Approved, mean)","(NAME_CONTRACT_STATUS_Completed, sum)","(NAME_CONTRACT_STATUS_Completed, count)","(NAME_CONTRACT_STATUS_Completed, min)","(NAME_CONTRACT_STATUS_Completed, max)","(NAME_CONTRACT_STATUS_Completed, mean)","(NAME_CONTRACT_STATUS_Demand, sum)","(NAME_CONTRACT_STATUS_Demand, count)","(NAME_CONTRACT_STATUS_Demand, min)","(NAME_CONTRACT_STATUS_Demand, max)","(NAME_CONTRACT_STATUS_Demand, mean)","(NAME_CONTRACT_STATUS_Refused, sum)","(NAME_CONTRACT_STATUS_Refused, count)","(NAME_CONTRACT_STATUS_Refused, min)","(NAME_CONTRACT_STATUS_Refused, max)","(NAME_CONTRACT_STATUS_Refused, mean)","(NAME_CONTRACT_STATUS_Sent proposal, sum)","(NAME_CONTRACT_STATUS_Sent proposal, count)","(NAME_CONTRACT_STATUS_Sent proposal, min)","(NAME_CONTRACT_STATUS_Sent proposal, max)","(NAME_CONTRACT_STATUS_Sent proposal, mean)","(NAME_CONTRACT_STATUS_Signed, sum)","(NAME_CONTRACT_STATUS_Signed, count)","(NAME_CONTRACT_STATUS_Signed, min)","(NAME_CONTRACT_STATUS_S

In [67]:
iv={}
for i in ccbalance.drop('TARGET',axis=1).columns:
    ct=woe(ccbalance,'TARGET',i)
    iv[i]=sum(ct['IV'])

D:\Software\Anaconda\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Software\Anaconda\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [68]:
impccbalance1=[]
for i,j in iv.items():
    if j>=0.03:
        impccbalance1.append(i)
impccbalance1

[('AMT_BALANCE', 'sum'),
 ('AMT_BALANCE', 'min'),
 ('AMT_BALANCE', 'max'),
 ('AMT_BALANCE', 'mean'),
 ('AMT_DRAWINGS_ATM_CURRENT', 'count'),
 ('AMT_DRAWINGS_ATM_CURRENT', 'mean'),
 ('AMT_DRAWINGS_CURRENT', 'sum'),
 ('AMT_DRAWINGS_CURRENT', 'max'),
 ('AMT_DRAWINGS_CURRENT', 'mean'),
 ('AMT_DRAWINGS_OTHER_CURRENT', 'count'),
 ('AMT_DRAWINGS_POS_CURRENT', 'count'),
 ('AMT_INST_MIN_REGULARITY', 'sum'),
 ('AMT_INST_MIN_REGULARITY', 'max'),
 ('AMT_INST_MIN_REGULARITY', 'mean'),
 ('AMT_PAYMENT_CURRENT', 'sum'),
 ('AMT_PAYMENT_CURRENT', 'count'),
 ('AMT_RECEIVABLE_PRINCIPAL', 'sum'),
 ('AMT_RECEIVABLE_PRINCIPAL', 'min'),
 ('AMT_RECEIVABLE_PRINCIPAL', 'max'),
 ('AMT_RECEIVABLE_PRINCIPAL', 'mean'),
 ('AMT_RECIVABLE', 'sum'),
 ('AMT_RECIVABLE', 'min'),
 ('AMT_RECIVABLE', 'max'),
 ('AMT_RECIVABLE', 'mean'),
 ('AMT_TOTAL_RECEIVABLE', 'sum'),
 ('AMT_TOTAL_RECEIVABLE', 'min'),
 ('AMT_TOTAL_RECEIVABLE', 'max'),
 ('AMT_TOTAL_RECEIVABLE', 'mean'),
 ('CNT_DRAWINGS_ATM_CURRENT', 'count'),
 ('CNT_DRAWINGS_

In [69]:
impccbalance2=[]
for i,j in iv.items():
    if j>=0.05:
        impccbalance2.append(i)
impccbalance2

[('AMT_DRAWINGS_ATM_CURRENT', 'count'),
 ('AMT_DRAWINGS_CURRENT', 'mean'),
 ('AMT_DRAWINGS_OTHER_CURRENT', 'count'),
 ('AMT_DRAWINGS_POS_CURRENT', 'count'),
 ('AMT_PAYMENT_CURRENT', 'count'),
 ('CNT_DRAWINGS_ATM_CURRENT', 'count'),
 ('CNT_DRAWINGS_ATM_CURRENT', 'mean'),
 ('CNT_DRAWINGS_CURRENT', 'max'),
 ('CNT_DRAWINGS_CURRENT', 'mean'),
 ('CNT_DRAWINGS_OTHER_CURRENT', 'count'),
 ('CNT_DRAWINGS_POS_CURRENT', 'count')]

### Getting important columns

In [101]:
imptrain1.append('SK_ID_CURR')
imptrain1.append('TARGET')
imptrain2.append('SK_ID_CURR')
imptrain2.append('TARGET')

In [102]:
trainmore=credit.loc[:,imptrain1]
trainless=credit.loc[:,imptrain2]

In [103]:
testless=xtest.loc[:,imptrain2]

In [73]:
impbureau1.append('SK_ID_CURR')
impbureau2.append('SK_ID_CURR')

In [74]:
train2more=bureau.loc[:,impbureau1]
train2less=bureau.loc[:,impbureau2]

In [75]:
test2less=bureautest.loc[:,impbureau2]

In [76]:
impccbalance1.append('SK_ID_CURR')
impccbalance2.append('SK_ID_CURR')

In [77]:
train3more=ccbalance.loc[:,impccbalance1]
train3less=ccbalance.loc[:,impccbalance2]

In [79]:
test3less=ccbalancetest.loc[:,impccbalance2]

In [80]:
impposcash1.append('SK_ID_CURR')
impposcash2.append('SK_ID_CURR')

In [81]:
train4more=poscash.loc[:,impposcash1]
train4less=poscash.loc[:,impposcash2]

In [83]:
test4less=poscashtest.loc[:,impposcash2]

In [84]:
impprevapp1.append('SK_ID_CURR')
impprevapp2.append('SK_ID_CURR')

In [85]:
train5more=prevapp.loc[:,impprevapp1]
train5less=prevapp.loc[:,impprevapp2]

In [86]:
test5less=prevapptest.loc[:,impprevapp2]

In [87]:
impinstpaym1.append('SK_ID_CURR')
impinstpaym2.append('SK_ID_CURR')

In [88]:
train6more=instpaym.loc[:,impinstpaym1]
train6less=instpaym.loc[:,impinstpaym2]

In [89]:
test6less=instpaymtest.loc[:,impinstpaym2]

In [168]:
print(trainmore.shape)
print(train2more.shape)
print(train3more.shape)
print(train4more.shape)
print(train5more.shape)
print(train6more.shape)

(276759, 35)
(307511, 46)
(307511, 38)
(307511, 19)
(307511, 37)
(307511, 13)


In [110]:
print(trainless.shape)
print(train2less.shape)
print(train3less.shape)
print(train4less.shape)
print(train5less.shape)
print(train6less.shape)

(276759, 14)
(276759, 20)
(276759, 12)
(276759, 1)
(276759, 6)
(276759, 3)


### Merging the train files

In [172]:
#trainmore=trainmore.merge(train2more,'left','SK_ID_CURR')
#trainmore=trainmore.merge(train3more,'left','SK_ID_CURR')
#trainmore=trainmore.merge(train4more,'left','SK_ID_CURR')
#trainmore=trainmore.merge(train5more,'left','SK_ID_CURR')
#trainmore=trainmore.merge(train6more,'left','SK_ID_CURR')

In [111]:
trainless=trainless.merge(train2less,'left','SK_ID_CURR')
trainless=trainless.merge(train3less,'left','SK_ID_CURR')
trainless=trainless.merge(train4less,'left','SK_ID_CURR')
trainless=trainless.merge(train5less,'left','SK_ID_CURR')
trainless=trainless.merge(train6less,'left','SK_ID_CURR')

In [112]:
trainless.shape

(276759, 51)

In [105]:
testless=testless.merge(test2less,'left','SK_ID_CURR')
testless=testless.merge(test3less,'left','SK_ID_CURR')
testless=testless.merge(test4less,'left','SK_ID_CURR')
testless=testless.merge(test5less,'left','SK_ID_CURR')
testless=testless.merge(test6less,'left','SK_ID_CURR')

In [106]:
testless.head()

,AMT_CREDIT,AMT_GOODS_PRICE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,OCCUPATION_TYPE,REGION_RATING_CLIENT_W_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,SK_ID_CURR,TARGET,"(DAYS_CREDIT, sum)","(DAYS_CREDIT, min)","(DAYS_CREDIT, max)","(DAYS_CREDIT, mean)","(DAYS_CREDIT_ENDDATE, sum)","(DAYS_CREDIT_ENDDATE, min)","(DAYS_CREDIT_ENDDATE, max)","(DAYS_CREDIT_ENDDATE, mean)","(DAYS_ENDDATE_FACT, sum)","(DAYS_ENDDATE_FACT, min)","(DAYS_ENDDATE_FACT, mean)","(AMT_CREDIT_SUM_DEBT, mean)","(AMT_CREDIT_SUM_LIMIT, max)","(DAYS_CREDIT_UPDATE, sum)","(DAYS_CREDIT_UPDATE, min)","(DAYS_CREDIT_UPDATE, mean)","(CREDIT_ACTIVE_Active, sum)","(CREDIT_ACTIVE_Active, mean)","(CREDIT_ACTIVE_Closed, mean)","(AMT_DRAWINGS_ATM_CURRENT, count)","(AMT_DRAWINGS_CURRENT, mean)","(AMT_DRAWINGS_OTHER_CURRENT, count)","(AMT_DRAWINGS_POS_CURRENT, count)","(AMT_PAYMENT_CURRENT, count)","(CNT_DRAWINGS_ATM_CURRENT, count)","(CNT_DRAWINGS_ATM_CURRENT, mean)","(CNT_DRAWINGS_CURRENT, max)","(CNT_DRAWINGS_CURRENT, mean)","(CNT_DRAWINGS_OTHER_CURRENT, count)","(CNT_DRAWINGS_POS_CURRENT, count)","(AMT_DOWN_PAYMENT, sum)","(DAYS_LAST_DUE_1ST_VERSION, sum)","(NAME_CONTRACT_STATUS_Approved, mean)","(NAME_CONTRACT_STATUS_Refused, mean)","(CODE_REJECT_REASON_XAP, mean)","(DAYS_INSTALMENT, min)","(DAYS_ENTRY_PAYMENT, min)"
0,465457.5,418500.0,Commercial associate,Secondary / secondary special,-13297,-762,Sales staff,2,Business Entity Type 3,0.675878,0.604894,0.000527,384575,0,-8000.0,-2857.0,-129.0,-1142.857143,-5346.0,-2710.0,231.0,-891.000000,-2867.0,-1036.0,-716.750000,39569.785714,27000.0,-3387.0,-1033.0,-483.857143,3.0,0.428571,0.571429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6642.0,-3609.0,0.400000,0.600000,0.400000,-2246.0,-2254.0
1,1281712.5,1179000.0,Commercial associate,Higher education,-14778,-1141,Managers,3,Business Entity Type 3,0.430827,0.425351,0.712155,214010,0,-4208.0,-1487.0,-295.0,-841.600000,11864.0,-391.0,9479.0,2372.800000,-1236.0,-765.0,-412.000000,0.000000,0.0,-1327.0,-765.0,-265.400000,2.0,0.400000,0.600000,16.0,30495.481875,16.0,16.0,16.0,16.0,0.875,9.0,1.291667,16.0,16.0,36603.0,356106.0,0.833333,0.166667,0.833333,-2375.0,-2397.0
2,495000.0,495000.0,Working,Secondary / secondary special,-17907,-639,Sales staff,2,Self-employed,0.527239,0.531760,0.207964,142232,0,-5155.0,-2305.0,-399.0,-859.166667,-335.0,-2123.0,1189.0,-55.833333,-2914.0,-2123.0,-971.333333,238527.750000,0.0,-2449.0,-1706.0,-408.166667,3.0,0.500000,0.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12330.9,-1286.0,0.428571,0.142857,0.714286,-1368.0,-1387.0
3,254700.0,225000.0,State servant,Secondary / secondary special,-19626,-6982,High skill tech staff,1,Business Entity Type 3,NaN,0.693521,0.614414,389171,0,-4842.0,-2657.0,-2185.0,-2421.000000,-4326.0,-2322.0,-2004.0,-2163.000000,-3555.0,-2321.0,-1777.500000,0.000000,0.0,-3552.0,-2321.0,-1776.000000,0.0,0.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10431.0,-1639.0,1.000000,0.000000,1.000000,-1969.0,-1964.0
4,308133.0,234000.0,Working,Secondary / secondary special,-20327,-1105,Laborers,2,Business Entity Type 3,0.654882,0.560690,0.636376,283617,0,-3647.0,-2618.0,-217.0,-1215.666667,-903.0,-2430.0,1609.0,-301.000000,-3123.0,-2430.0,-1561.500000,162492.000000,0.0,-3132.0,-2415.0,-1044.000000,1.0,0.333333,0.666667,0.0,0.000000,0.0,0.0,0.0,0.0,NaN,0.0,0.000000,0.0,0.0,2475.0,362247.0,0.555556,0.333333,0.666667,-1770.0,-1787.0


In [107]:
testless.shape

(30752, 51)

### Substituting WOE

In [184]:
#woetrain1=pd.DataFrame()
#woetrain1['TARGET']=trainmore['TARGET']
#woetrain1

,TARGET
0,0
1,0
2,1
3,0
4,0
...,...
276754,0
276755,0
276756,0
276757,0


In [113]:
woetrain2=pd.DataFrame()
woetrain2['TARGET']=trainless['TARGET']
woetrain2

,TARGET
0,0
1,0
2,1
3,0
4,0
...,...
276754,0
276755,0
276756,0
276757,0


In [114]:
f=1
for i in trainless.drop(['TARGET','SK_ID_CURR'],axis=1).columns:
    print(str(f)+'/'+str(len(trainless.drop(['TARGET','SK_ID_CURR'],axis=1).columns)))
    f=f+1
    ct=woe(trainless,'TARGET',i)
    if 'bin' in ct.columns:
        val=[]
        if trainless[i].isna().sum()>1:
            for j in trainless[i]:
                flag=0
                if np.isnan(j):
                    val.append(ct['WOE'][len(ct)-1])
                    flag=1
                else:
                    for m in range(0,len(ct)-1,1):
                        if j in ct['bin'][m]:
                            val.append(ct['WOE'][m])
                            flag=1
                            continue
                if flag==0:
                    val.append(ct['WOE'][0])
                    
        else:
            for j in trainless[i]:
                flag=0
                for m in range(0,len(ct),1):
                    if j in ct['bin'][m]:
                        val.append(ct['WOE'][m])
                        flag=1
                        continue
                if flag==0:
                    val.append(ct['WOE'][0])
    else:
        val=[]
        if trainless[i].isna().sum()>1:
            for j in trainless[i]:
                flag=0
                if str(j)=='nan':
                    val.append(ct['WOE'][len(ct)-1])
                    flag=1
                else:
                    for m in range(0,len(ct)-1,1):
                        if j==ct[i][m]:
                            val.append(ct['WOE'][m])
                            flag=1
                            continue
                if flag==0:
                    val.append(ct['WOE'][0])
        else:
            for j in trainless[i]:
                flag=0
                for m in range(0,len(ct),1):
                    if j==ct[i][m]:
                        val.append(ct['WOE'][m])
                        flag=1
                        continue
                if flag==0:
                    val.append(ct['WOE'][0])
    val=np.asarray(val,'float64')
    woetrain2[str('WOE_'+str(np.array(i).flatten()))]=val                
woetrain2

1/49
2/49
3/49


D:\Software\Anaconda\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


4/49
5/49
6/49
7/49
8/49
9/49
10/49
11/49
12/49
13/49
14/49
15/49
16/49
17/49
18/49
19/49
20/49
21/49
22/49
23/49
24/49
25/49
26/49
27/49
28/49
29/49
30/49
31/49
32/49
33/49
34/49
35/49
36/49
37/49
38/49
39/49
40/49
41/49
42/49
43/49
44/49
45/49
46/49
47/49
48/49
49/49


,TARGET,WOE_['AMT_CREDIT'],WOE_['AMT_GOODS_PRICE'],WOE_['NAME_INCOME_TYPE'],WOE_['NAME_EDUCATION_TYPE'],WOE_['DAYS_BIRTH'],WOE_['DAYS_EMPLOYED'],WOE_['OCCUPATION_TYPE'],WOE_['REGION_RATING_CLIENT_W_CITY'],WOE_['ORGANIZATION_TYPE'],WOE_['EXT_SOURCE_1'],WOE_['EXT_SOURCE_2'],WOE_['EXT_SOURCE_3'],WOE_['DAYS_CREDIT' 'sum'],WOE_['DAYS_CREDIT' 'min'],WOE_['DAYS_CREDIT' 'max'],WOE_['DAYS_CREDIT' 'mean'],WOE_['DAYS_CREDIT_ENDDATE' 'sum'],WOE_['DAYS_CREDIT_ENDDATE' 'min'],WOE_['DAYS_CREDIT_ENDDATE' 'max'],WOE_['DAYS_CREDIT_ENDDATE' 'mean'],WOE_['DAYS_ENDDATE_FACT' 'sum'],WOE_['DAYS_ENDDATE_FACT' 'min'],WOE_['DAYS_ENDDATE_FACT' 'mean'],WOE_['AMT_CREDIT_SUM_DEBT' 'mean'],WOE_['AMT_CREDIT_SUM_LIMIT' 'max'],WOE_['DAYS_CREDIT_UPDATE' 'sum'],WOE_['DAYS_CREDIT_UPDATE' 'min'],WOE_['DAYS_CREDIT_UPDATE' 'mean'],WOE_['CREDIT_ACTIVE_Active' 'sum'],WOE_['CREDIT_ACTIVE_Active' 'mean'],WOE_['CREDIT_ACTIVE_Closed' 'mean'],WOE_['AMT_DRAWINGS_ATM_CURRENT' 'count'],WOE_['AMT_DRAWINGS_CURRENT' 'mean'],WOE_['AMT_DRAWINGS_OTHER_CURRENT' 'count'],WOE_['AMT_DRAWINGS_POS_CURRENT' 'count'],WOE_['AMT_PAYMENT_CURRENT' 'count'],WOE_['CNT_DRAWINGS_ATM_CURRENT' 'count'],WOE_['CNT_DRAWINGS_ATM_CURRENT' 'mean'],WOE_['CNT_DRAWINGS_CURRENT' 'max'],WOE_['CNT_DRAWINGS_CURRENT' 'mean'],WOE_['CNT_DRAWINGS_OTHER_CURRENT' 'count'],WOE_['CNT_DRAWINGS_POS_CURRENT' 'count'],WOE_['AMT_DOWN_PAYMENT' 'sum'],WOE_['DAYS_LAST_DUE_1ST_VERSION' 'sum'],WOE_['NAME_CONTRACT_STATUS_Approved' 'mean'],WOE_['NAME_CONTRACT_STATUS_Refused' 'mean'],WOE_['CODE_REJECT_REASON_XAP' 'mean'],WOE_['DAYS_INSTALMENT' 'min'],WOE_['DAYS_ENTRY_PAYMENT' 'min']
0,0,0.249766,0.163212,-0.187342,0.436057,-0.304800,-0.391887,0.251763,0.021248,0.156295,-0.466422,0.599615,0.523564,0.248168,0.333144,0.118735,0.364417,0.441435,0.332041,0.020782,0.511075,0.298887,0.393971,0.344199,0.246110,0.133783,0.281010,0.365687,0.549580,0.085685,0.360033,0.395371,0.034488,0.034488,0.034488,0.034488,0.034488,0.034488,0.069919,0.034488,0.034488,0.034488,0.034488,0.468999,0.087712,0.136890,0.139476,0.146435,0.032716,-0.003404
1,0,0.296124,0.380928,0.078430,-0.110769,0.002260,-0.083280,0.266238,0.021248,-0.161475,-0.054113,0.761515,0.202892,0.092652,0.034755,-0.123846,-0.106635,-0.034063,0.107708,-0.177724,-0.239588,0.182058,0.170246,0.171377,-0.134059,0.133783,0.186259,0.232031,0.023426,-0.304004,-0.411651,-0.258740,0.034488,0.034488,0.034488,0.034488,0.034488,0.034488,0.069919,0.034488,0.034488,0.034488,0.034488,0.166176,-0.042089,-0.164042,0.139476,0.146435,-0.164961,-0.163835
2,1,-0.358265,-0.554948,-0.187342,-0.110769,0.042607,-0.368778,-0.197619,0.021248,0.085583,0.473007,0.012740,0.652092,-0.003175,-0.167312,-0.081418,-0.269498,-0.233657,0.001154,0.020782,-0.271042,0.014292,-0.035837,-0.172625,-0.244246,0.133783,0.031720,0.026772,-0.095437,0.226656,0.477236,0.395371,0.034488,0.034488,0.034488,0.034488,0.034488,0.034488,0.069919,0.034488,0.034488,0.034488,0.034488,-0.079710,-0.042089,0.053093,-0.262131,-0.254040,-0.002635,-0.003404
3,0,0.047504,-0.124632,0.078430,0.436057,-0.367292,0.120151,0.560757,0.021248,-0.158223,0.808814,0.486644,-1.250475,0.204977,-0.079371,0.106260,-0.051801,-0.447632,-0.010142,-0.132811,-0.368336,0.005931,0.063001,0.106109,-0.201340,0.133783,0.113255,0.077346,-0.237972,-0.574296,-0.379437,-0.380719,0.034488,0.034488,0.034488,0.034488,0.034488,0.034488,0.069919,0.034488,0.034488,0.034488,0.034488,0.312118,0.124375,0.136890,0.139476,0.146435,0.106336,0.109711
4,0,0.144611,-0.124632,-0.187342,0.436057,-0.304800,0.336621,-0.295216,0.021248,-0.161475,0.073998,0.110608,-0.215943,-0.509828,-0.564015,-0.078997,-0.653885,-0.318931,-0.288809,-0.269810,-0.129616,-0.327070,-0.284409,-0.284409,-0.201340,0.133783,-0.348276,-0.391633,-0.432981,0.085685,-0.379437,-0.380719,0.034488,0.034488,0.034488,0.034488,0.034488,0.034488,0.069919,0.034488,0.034488,0.034488,0.034488,-0.176258,0.025519,0.197280,-0.137508,-0.110980,-0.162799,-0.170266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [115]:
woetest2=pd.DataFrame()
woetest2['TARGET']=testless['TARGET']
woetest2

,TARGET
0,0
1,0
2,0
3,0
4,0
...,...
30747,0
30748,0
30749,0
30750,0


In [117]:
f=1
for i in trainless.drop(['TARGET','SK_ID_CURR'],axis=1).columns:
    print(str(f)+'/'+str(len(trainless.drop(['TARGET','SK_ID_CURR'],axis=1).columns)))
    f=f+1
    ct=woe(trainless,'TARGET',i)
    if 'bin' in ct.columns:
        val=[]
        if testless[i].isna().sum()>1:
            for j in testless[i]:
                flag=0
                if np.isnan(j):
                    val.append(ct['WOE'][len(ct)-1])
                    flag=1
                else:
                    for m in range(0,len(ct)-1,1):
                        if j in ct['bin'][m]:
                            val.append(ct['WOE'][m])
                            flag=1
                            continue
                if flag==0:
                    val.append(ct['WOE'][0])
                    
        else:
            for j in testless[i]:
                flag=0
                for m in range(0,len(ct),1):
                    if j in ct['bin'][m]:
                        val.append(ct['WOE'][m])
                        flag=1
                        continue
                if flag==0:
                    val.append(ct['WOE'][0])
    else:
        val=[]
        if testless[i].isna().sum()>1:
            for j in testless[i]:
                flag=0
                if str(j)=='nan':
                    val.append(ct['WOE'][len(ct)-1])
                    flag=1
                else:
                    for m in range(0,len(ct)-1,1):
                        if j==ct[i][m]:
                            val.append(ct['WOE'][m])
                            flag=1
                            continue
                if flag==0:
                    val.append(ct['WOE'][0])
        else:
            for j in testless[i]:
                flag=0
                for m in range(0,len(ct),1):
                    if j==ct[i][m]:
                        val.append(ct['WOE'][m])
                        flag=1
                        continue
                if flag==0:
                    val.append(ct['WOE'][0])
    val=np.asarray(val,'float64')
    woetest2[str('WOE_'+str(np.array(i).flatten()))]=val                
woetest2

1/49
2/49
3/49


D:\Software\Anaconda\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


4/49
5/49
6/49
7/49
8/49
9/49
10/49
11/49
12/49
13/49
14/49
15/49
16/49
17/49
18/49
19/49
20/49
21/49
22/49
23/49
24/49
25/49
26/49
27/49
28/49
29/49
30/49
31/49
32/49
33/49
34/49
35/49
36/49
37/49
38/49
39/49
40/49
41/49
42/49
43/49
44/49
45/49
46/49
47/49
48/49
49/49


,TARGET,WOE_['AMT_CREDIT'],WOE_['AMT_GOODS_PRICE'],WOE_['NAME_INCOME_TYPE'],WOE_['NAME_EDUCATION_TYPE'],WOE_['DAYS_BIRTH'],WOE_['DAYS_EMPLOYED'],WOE_['OCCUPATION_TYPE'],WOE_['REGION_RATING_CLIENT_W_CITY'],WOE_['ORGANIZATION_TYPE'],WOE_['EXT_SOURCE_1'],WOE_['EXT_SOURCE_2'],WOE_['EXT_SOURCE_3'],WOE_['DAYS_CREDIT' 'sum'],WOE_['DAYS_CREDIT' 'min'],WOE_['DAYS_CREDIT' 'max'],WOE_['DAYS_CREDIT' 'mean'],WOE_['DAYS_CREDIT_ENDDATE' 'sum'],WOE_['DAYS_CREDIT_ENDDATE' 'min'],WOE_['DAYS_CREDIT_ENDDATE' 'max'],WOE_['DAYS_CREDIT_ENDDATE' 'mean'],WOE_['DAYS_ENDDATE_FACT' 'sum'],WOE_['DAYS_ENDDATE_FACT' 'min'],WOE_['DAYS_ENDDATE_FACT' 'mean'],WOE_['AMT_CREDIT_SUM_DEBT' 'mean'],WOE_['AMT_CREDIT_SUM_LIMIT' 'max'],WOE_['DAYS_CREDIT_UPDATE' 'sum'],WOE_['DAYS_CREDIT_UPDATE' 'min'],WOE_['DAYS_CREDIT_UPDATE' 'mean'],WOE_['CREDIT_ACTIVE_Active' 'sum'],WOE_['CREDIT_ACTIVE_Active' 'mean'],WOE_['CREDIT_ACTIVE_Closed' 'mean'],WOE_['AMT_DRAWINGS_ATM_CURRENT' 'count'],WOE_['AMT_DRAWINGS_CURRENT' 'mean'],WOE_['AMT_DRAWINGS_OTHER_CURRENT' 'count'],WOE_['AMT_DRAWINGS_POS_CURRENT' 'count'],WOE_['AMT_PAYMENT_CURRENT' 'count'],WOE_['CNT_DRAWINGS_ATM_CURRENT' 'count'],WOE_['CNT_DRAWINGS_ATM_CURRENT' 'mean'],WOE_['CNT_DRAWINGS_CURRENT' 'max'],WOE_['CNT_DRAWINGS_CURRENT' 'mean'],WOE_['CNT_DRAWINGS_OTHER_CURRENT' 'count'],WOE_['CNT_DRAWINGS_POS_CURRENT' 'count'],WOE_['AMT_DOWN_PAYMENT' 'sum'],WOE_['DAYS_LAST_DUE_1ST_VERSION' 'sum'],WOE_['NAME_CONTRACT_STATUS_Approved' 'mean'],WOE_['NAME_CONTRACT_STATUS_Refused' 'mean'],WOE_['CODE_REJECT_REASON_XAP' 'mean'],WOE_['DAYS_INSTALMENT' 'min'],WOE_['DAYS_ENTRY_PAYMENT' 'min']
0,0,0.249766,0.163212,-0.187342,0.436057,-0.304800,-0.391887,0.251763,0.021248,0.156295,-0.466422,0.599615,0.523564,0.248168,0.333144,0.118735,0.364417,0.441435,0.332041,0.020782,0.511075,0.298887,0.393971,0.344199,0.246110,0.133783,0.281010,0.365687,0.549580,0.085685,0.360033,0.395371,0.034488,0.034488,0.034488,0.034488,0.034488,0.034488,0.069919,0.034488,0.034488,0.034488,0.034488,0.468999,0.087712,0.136890,0.139476,0.146435,0.032716,-0.003404
1,0,0.296124,0.380928,0.078430,-0.110769,0.002260,-0.083280,0.266238,0.021248,-0.161475,-0.054113,0.761515,0.202892,0.092652,0.034755,-0.123846,-0.106635,-0.034063,0.107708,-0.177724,-0.239588,0.182058,0.170246,0.171377,-0.134059,0.133783,0.186259,0.232031,0.023426,-0.304004,-0.411651,-0.258740,0.034488,0.034488,0.034488,0.034488,0.034488,0.034488,0.069919,0.034488,0.034488,0.034488,0.034488,0.166176,-0.042089,-0.164042,0.139476,0.146435,-0.164961,-0.163835
2,1,-0.358265,-0.554948,-0.187342,-0.110769,0.042607,-0.368778,-0.197619,0.021248,0.085583,0.473007,0.012740,0.652092,-0.003175,-0.167312,-0.081418,-0.269498,-0.233657,0.001154,0.020782,-0.271042,0.014292,-0.035837,-0.172625,-0.244246,0.133783,0.031720,0.026772,-0.095437,0.226656,0.477236,0.395371,0.034488,0.034488,0.034488,0.034488,0.034488,0.034488,0.069919,0.034488,0.034488,0.034488,0.034488,-0.079710,-0.042089,0.053093,-0.262131,-0.254040,-0.002635,-0.003404
3,0,0.047504,-0.124632,0.078430,0.436057,-0.367292,0.120151,0.560757,0.021248,-0.158223,0.808814,0.486644,-1.250475,0.204977,-0.079371,0.106260,-0.051801,-0.447632,-0.010142,-0.132811,-0.368336,0.005931,0.063001,0.106109,-0.201340,0.133783,0.113255,0.077346,-0.237972,-0.574296,-0.379437,-0.380719,0.034488,0.034488,0.034488,0.034488,0.034488,0.034488,0.069919,0.034488,0.034488,0.034488,0.034488,0.312118,0.124375,0.136890,0.139476,0.146435,0.106336,0.109711
4,0,0.144611,-0.124632,-0.187342,0.436057,-0.304800,0.336621,-0.295216,0.021248,-0.161475,0.073998,0.110608,-0.215943,-0.509828,-0.564015,-0.078997,-0.653885,-0.318931,-0.288809,-0.269810,-0.129616,-0.327070,-0.284409,-0.284409,-0.201340,0.133783,-0.348276,-0.391633,-0.432981,0.085685,-0.379437,-0.380719,0.034488,0.034488,0.034488,0.034488,0.034488,0.034488,0.069919,0.034488,0.034488,0.034488,0.034488,-0.176258,0.025519,0.197280,-0.137508,-0.110980,-0.162799,-0.170266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [119]:
woetest2

,TARGET,WOE_['AMT_CREDIT'],WOE_['AMT_GOODS_PRICE'],WOE_['NAME_INCOME_TYPE'],WOE_['NAME_EDUCATION_TYPE'],WOE_['DAYS_BIRTH'],WOE_['DAYS_EMPLOYED'],WOE_['OCCUPATION_TYPE'],WOE_['REGION_RATING_CLIENT_W_CITY'],WOE_['ORGANIZATION_TYPE'],WOE_['EXT_SOURCE_1'],WOE_['EXT_SOURCE_2'],WOE_['EXT_SOURCE_3'],WOE_['DAYS_CREDIT' 'sum'],WOE_['DAYS_CREDIT' 'min'],WOE_['DAYS_CREDIT' 'max'],WOE_['DAYS_CREDIT' 'mean'],WOE_['DAYS_CREDIT_ENDDATE' 'sum'],WOE_['DAYS_CREDIT_ENDDATE' 'min'],WOE_['DAYS_CREDIT_ENDDATE' 'max'],WOE_['DAYS_CREDIT_ENDDATE' 'mean'],WOE_['DAYS_ENDDATE_FACT' 'sum'],WOE_['DAYS_ENDDATE_FACT' 'min'],WOE_['DAYS_ENDDATE_FACT' 'mean'],WOE_['AMT_CREDIT_SUM_DEBT' 'mean'],WOE_['AMT_CREDIT_SUM_LIMIT' 'max'],WOE_['DAYS_CREDIT_UPDATE' 'sum'],WOE_['DAYS_CREDIT_UPDATE' 'min'],WOE_['DAYS_CREDIT_UPDATE' 'mean'],WOE_['CREDIT_ACTIVE_Active' 'sum'],WOE_['CREDIT_ACTIVE_Active' 'mean'],WOE_['CREDIT_ACTIVE_Closed' 'mean'],WOE_['AMT_DRAWINGS_ATM_CURRENT' 'count'],WOE_['AMT_DRAWINGS_CURRENT' 'mean'],WOE_['AMT_DRAWINGS_OTHER_CURRENT' 'count'],WOE_['AMT_DRAWINGS_POS_CURRENT' 'count'],WOE_['AMT_PAYMENT_CURRENT' 'count'],WOE_['CNT_DRAWINGS_ATM_CURRENT' 'count'],WOE_['CNT_DRAWINGS_ATM_CURRENT' 'mean'],WOE_['CNT_DRAWINGS_CURRENT' 'max'],WOE_['CNT_DRAWINGS_CURRENT' 'mean'],WOE_['CNT_DRAWINGS_OTHER_CURRENT' 'count'],WOE_['CNT_DRAWINGS_POS_CURRENT' 'count'],WOE_['AMT_DOWN_PAYMENT' 'sum'],WOE_['DAYS_LAST_DUE_1ST_VERSION' 'sum'],WOE_['NAME_CONTRACT_STATUS_Approved' 'mean'],WOE_['NAME_CONTRACT_STATUS_Refused' 'mean'],WOE_['CODE_REJECT_REASON_XAP' 'mean'],WOE_['DAYS_INSTALMENT' 'min'],WOE_['DAYS_ENTRY_PAYMENT' 'min']
0,0,-0.102815,-0.554948,0.078430,-0.110769,-0.128890,-0.362443,-0.197619,0.021248,-0.158223,0.700889,0.199532,-1.250475,0.198488,0.317066,-0.078997,0.235388,0.431288,0.392136,0.202249,0.511075,0.195975,0.054441,0.022161,-0.041560,-0.364783,0.186145,0.077346,0.087297,-0.022053,-0.018065,-0.158755,0.034488,0.034488,0.034488,0.034488,0.034488,0.034488,0.069919,0.034488,0.034488,0.034488,0.034488,-0.079710,0.295009,-0.323232,-0.772721,-0.570855,0.106336,0.109711
1,0,0.296124,0.380928,0.078430,0.436057,0.002260,-0.253243,0.266238,-0.380451,-0.158223,0.073998,-0.149785,0.838096,0.147155,0.034755,0.118735,0.018429,0.130961,-0.181321,0.407180,-0.042475,0.005931,-0.035837,-0.141427,0.399831,0.133783,0.025686,0.026772,-0.168156,0.085685,0.064820,0.081011,-0.473890,-0.523765,-0.473890,-0.473890,-0.375778,-0.473890,-0.554087,-0.389532,-0.427506,-0.473890,-0.473890,0.325977,-0.134279,0.312672,-0.006185,-0.016294,0.127082,0.129291
2,0,-0.102815,0.084100,-0.187342,-0.110769,0.108332,-0.391887,-0.197619,0.021248,-0.261872,0.247787,0.079063,-0.821951,0.092652,0.261464,0.280952,0.018429,0.066447,0.251862,0.020782,0.002649,0.195975,0.279287,0.171377,-0.201340,0.133783,0.186259,0.267768,0.003976,-0.022053,-0.018065,-0.039739,0.034488,0.034488,0.034488,0.034488,0.034488,0.034488,0.069919,0.034488,0.034488,0.034488,0.034488,0.130278,0.049324,-0.323232,0.139476,-0.193130,0.032716,0.020988
3,0,0.121238,-0.128924,0.349761,-0.110769,0.205804,0.675450,0.304565,0.539581,-0.158223,-0.054113,0.618028,0.523564,0.092652,0.342204,0.323993,0.396990,0.441791,0.332041,0.248324,0.348495,0.212231,0.393971,0.388182,0.399831,0.133783,0.239392,0.365687,0.459678,0.226656,0.401778,0.423709,0.034488,0.034488,0.034488,0.034488,0.034488,0.034488,0.069919,0.034488,0.034488,0.034488,0.034488,0.046093,0.087712,0.136890,0.139476,0.146435,0.175352,0.074836
4,0,-0.233570,0.031668,-0.187342,-0.110769,0.315534,-0.253243,-0.295216,0.021248,-0.158223,0.597679,0.110608,0.625795,0.065734,0.342204,-0.081418,0.331439,0.151640,0.344478,-0.269810,0.170434,0.195975,0.333012,0.417997,-0.244246,0.133783,0.186145,0.381877,0.549580,0.226656,0.251842,0.216927,0.404213,0.413021,0.404213,0.404213,0.387199,0.404213,0.069919,0.407189,0.418024,0.404213,0.404213,-0.307704,0.109442,-0.035878,-0.262131,-0.254040,0.074530,0.074836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [187]:
f=0
for i in trainmore.drop(['TARGET','SK_ID_CURR'],axis=1).columns:
    print(str(f)+'/'+str(len(trainmore.drop(['TARGET','SK_ID_CURR'],axis=1).columns)))
    f=f+1
    ct=woe(trainmore,'TARGET',i)
    if 'bin' in ct.columns:
        val=[]
        if trainmore[i].isna().sum()>1:
            for j in trainmore[i]:
                flag=0
                if np.isnan(j):
                    val.append(ct['WOE'][len(ct)-1])
                    flag=1
                else:
                    for m in range(0,len(ct)-1,1):
                        if j in ct['bin'][m]:
                            val.append(ct['WOE'][m])
                            flag=1
                            continue
                if flag==0:
                    val.append(ct['WOE'][0])
                    
        else:
            for j in trainmore[i]:
                flag=0
                for m in range(0,len(ct),1):
                    if j in ct['bin'][m]:
                        val.append(ct['WOE'][m])
                        flag=1
                        continue
                if flag==0:
                    val.append(ct['WOE'][0])
    else:
        val=[]
        if trainmore[i].isna().sum()>1:
            for j in trainmore[i]:
                flag=0
                if str(j)=='nan':
                    val.append(ct['WOE'][len(ct)-1])
                    flag=1
                else:
                    for m in range(0,len(ct)-1,1):
                        if j==ct[i][m]:
                            val.append(ct['WOE'][m])
                            flag=1
                            continue
                if flag==0:
                    val.append(ct['WOE'][0])
        else:
            for j in trainmore[i]:
                flag=0
                for m in range(0,len(ct),1):
                    if j==ct[i][m]:
                        val.append(ct['WOE'][m])
                        flag=1
                        continue
                if flag==0:
                    val.append(ct['WOE'][0])
    val=np.asarray(val,'float64')
    woetrain1[str('WOE_'+str(np.array(i).flatten()))]=val                
woetrain1

0/181


D:\Software\Anaconda\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1/181
2/181
3/181
4/181
5/181
6/181
7/181
8/181
9/181
10/181
11/181
12/181
13/181
14/181
15/181
16/181
17/181
18/181
19/181
20/181
21/181
22/181
23/181
24/181
25/181
26/181
27/181
28/181
29/181
30/181
31/181
32/181
33/181
34/181
35/181
36/181
37/181
38/181
39/181
40/181
41/181
42/181
43/181
44/181
45/181
46/181
47/181
48/181
49/181
50/181
51/181
52/181
53/181
54/181
55/181
56/181
57/181
58/181
59/181
60/181
61/181
62/181
63/181
64/181
65/181
66/181
67/181
68/181
69/181
70/181
71/181
72/181
73/181
74/181
75/181
76/181
77/181
78/181
79/181
80/181
81/181
82/181
83/181
84/181
85/181
86/181
87/181
88/181
89/181
90/181
91/181
92/181
93/181
94/181
95/181
96/181
97/181
98/181
99/181
100/181
101/181
102/181
103/181
104/181
105/181
106/181
107/181
108/181
109/181
110/181
111/181
112/181
113/181
114/181
115/181
116/181
117/181
118/181
119/181
120/181
121/181
122/181
123/181
124/181
125/181
126/181
127/181
128/181
129/181
130/181
131/181
132/181
133/181
134/181
135/181
136/181
137/181
138/181
139/

,TARGET,WOE_['CODE_GENDER'],WOE_['AMT_CREDIT'],WOE_['AMT_ANNUITY'],WOE_['AMT_GOODS_PRICE'],WOE_['NAME_INCOME_TYPE'],WOE_['NAME_EDUCATION_TYPE'],WOE_['REGION_POPULATION_RELATIVE'],WOE_['DAYS_BIRTH'],WOE_['DAYS_EMPLOYED'],WOE_['DAYS_ID_PUBLISH'],WOE_['FLAG_EMP_PHONE'],WOE_['OCCUPATION_TYPE'],WOE_['REGION_RATING_CLIENT'],WOE_['REGION_RATING_CLIENT_W_CITY'],WOE_['REG_CITY_NOT_WORK_CITY'],WOE_['ORGANIZATION_TYPE'],WOE_['EXT_SOURCE_1'],WOE_['EXT_SOURCE_2'],WOE_['EXT_SOURCE_3'],WOE_['APARTMENTS_AVG'],WOE_['ELEVATORS_AVG'],WOE_['FLOORSMAX_AVG'],WOE_['LIVINGAREA_AVG'],WOE_['APARTMENTS_MODE'],WOE_['ELEVATORS_MODE'],WOE_['FLOORSMAX_MODE'],WOE_['LIVINGAREA_MODE'],WOE_['APARTMENTS_MEDI'],WOE_['ELEVATORS_MEDI'],WOE_['FLOORSMAX_MEDI'],WOE_['LIVINGAREA_MEDI'],WOE_['TOTALAREA_MODE'],WOE_['DAYS_LAST_PHONE_CHANGE'],WOE_['DAYS_CREDIT' 'sum'],WOE_['DAYS_CREDIT' 'min'],WOE_['DAYS_CREDIT' 'max'],WOE_['DAYS_CREDIT' 'mean'],WOE_['DAYS_CREDIT_ENDDATE' 'sum'],WOE_['DAYS_CREDIT_ENDDATE' 'min'],WOE_['DAYS_CREDIT_ENDDATE' 'max'],WOE_['DAYS_CREDIT_ENDDATE' 'mean'],WOE_['DAYS_ENDDATE_FACT' 'sum'],WOE_['DAYS_ENDDATE_FACT' 'count'],WOE_['DAYS_ENDDATE_FACT' 'min'],WOE_['DAYS_ENDDATE_FACT' 'max'],WOE_['DAYS_ENDDATE_FACT' 'mean'],WOE_['AMT_CREDIT_MAX_OVERDUE' 'sum'],WOE_['AMT_CREDIT_SUM_DEBT' 'sum'],WOE_['AMT_CREDIT_SUM_DEBT' 'min'],WOE_['AMT_CREDIT_SUM_DEBT' 'max'],WOE_['AMT_CREDIT_SUM_DEBT' 'mean'],WOE_['AMT_CREDIT_SUM_LIMIT' 'sum'],WOE_['AMT_CREDIT_SUM_LIMIT' 'max'],WOE_['AMT_CREDIT_SUM_LIMIT' 'mean'],WOE_['DAYS_CREDIT_UPDATE' 'sum'],WOE_['DAYS_CREDIT_UPDATE' 'min'],WOE_['DAYS_CREDIT_UPDATE' 'max'],WOE_['DAYS_CREDIT_UPDATE' 'mean'],"WOE_[('MONTHS_BALANCE', 'sum') 'mean']","WOE_[('MONTHS_BALANCE', 'count') 'mean']","WOE_[('MONTHS_BALANCE', 'min') 'mean']",...,WOE_['CNT_INSTALMENT' 'count'],WOE_['CNT_INSTALMENT_FUTURE' 'count'],WOE_['SK_DPD' 'count'],WOE_['SK_DPD_DEF' 'count'],WOE_['NAME_CONTRACT_STATUS_Active' 'sum'],WOE_['NAME_CONTRACT_STATUS_Active' 'count'],WOE_['NAME_CONTRACT_STATUS_Amortized debt' 'count'],WOE_['NAME_CONTRACT_STATUS_Approved' 'count'],WOE_['NAME_CONTRACT_STATUS_Canceled' 'count'],WOE_['NAME_CONTRACT_STATUS_Completed' 'count'],WOE_['NAME_CONTRACT_STATUS_Demand' 'count'],WOE_['NAME_CONTRACT_STATUS_Returned to the store' 'count'],WOE_['NAME_CONTRACT_STATUS_Signed' 'count'],WOE_['NAME_CONTRACT_STATUS_XNA' 'count'],WOE_['AMT_DOWN_PAYMENT' 'sum'],WOE_['AMT_DOWN_PAYMENT' 'max'],WOE_['AMT_DOWN_PAYMENT' 'mean'],WOE_['RATE_DOWN_PAYMENT' 'sum'],WOE_['RATE_DOWN_PAYMENT' 'max'],WOE_['DAYS_DECISION' 'min'],WOE_['DAYS_DECISION' 'mean'],WOE_['DAYS_FIRST_DRAWING' 'sum'],WOE_['DAYS_FIRST_DRAWING' 'mean'],WOE_['DAYS_FIRST_DUE' 'sum'],WOE_['DAYS_FIRST_DUE' 'min'],WOE_['DAYS_FIRST_DUE' 'mean'],WOE_['DAYS_LAST_DUE_1ST_VERSION' 'sum'],WOE_['DAYS_LAST_DUE_1ST_VERSION' 'min'],WOE_['DAYS_LAST_DUE_1ST_VERSION' 'mean'],WOE_['DAYS_LAST_DUE' 'sum'],WOE_['DAYS_LAST_DUE' 'min'],WOE_['DAYS_LAST_DUE' 'mean'],WOE_['DAYS_TERMINATION' 'sum'],WOE_['DAYS_TERMINATION' 'min'],WOE_['DAYS_TERMINATION' 'mean'],WOE_['NAME_CONTRACT_STATUS_Approved' 'mean'],WOE_['NAME_CONTRACT_STATUS_Refused' 'sum'],WOE_['NAME_CONTRACT_STATUS_Refused' 'max'],WOE_['NAME_CONTRACT_STATUS_Refused' 'mean'],WOE_['CODE_REJECT_REASON_HC' 'max'],WOE_['CODE_REJECT_REASON_HC' 'mean'],WOE_['CODE_REJECT_REASON_SCOFR' 'max'],WOE_['CODE_REJECT_REASON_XAP' 'min'],WOE_['CODE_REJECT_REASON_XAP' 'mean'],WOE_['NAME_CLIENT_TYPE_New' 'max'],WOE_['NAME_PRODUCT_TYPE_walk-in' 'sum'],WOE_['NAME_PRODUCT_TYPE_walk-in' 'max'],WOE_['NAME_PRODUCT_TYPE_walk-in' 'mean'],WOE_['NAME_YIELD_GROUP_XNA' 'mean'],WOE_['NAME_YIELD_GROUP_low_normal' 'mean'],WOE_['DAYS_INSTALMENT' 'sum'],WOE_['DAYS_INSTALMENT' 'min'],WOE_['DAYS_INSTALMENT' 'mean'],WOE_['DAYS_ENTRY_PAYMENT' 'sum'],WOE_['DAYS_ENTRY_PAYMENT' 'min'],WOE_['DAYS_ENTRY_PAYMENT' 'mean'],WOE_['AMT_INSTALMENT' 'min'],WOE_['AMT_INSTALMENT' 'max'],WOE_['AMT_PAYMENT' 'sum'],WOE_['AMT_PAYMENT' 'min'],WOE_['AMT_PAYMENT' 'max'],WOE_['AMT_PAYMENT' 'mean']
0,0,0.152919,0.249766,-0.300628,0.163212

In [120]:
sum(woetrain2.isna().sum()>0)

0

In [121]:
sum(woetest2.isna().sum()>0)

0

### Model

In [73]:
#from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

In [74]:
#vif(train.drop('TARGET',axis=1),train['TARGET'])

In [122]:
woetrain2.describe()

,TARGET,WOE_['AMT_CREDIT'],WOE_['AMT_GOODS_PRICE'],WOE_['NAME_INCOME_TYPE'],WOE_['NAME_EDUCATION_TYPE'],WOE_['DAYS_BIRTH'],WOE_['DAYS_EMPLOYED'],WOE_['OCCUPATION_TYPE'],WOE_['REGION_RATING_CLIENT_W_CITY'],WOE_['ORGANIZATION_TYPE'],WOE_['EXT_SOURCE_1'],WOE_['EXT_SOURCE_2'],WOE_['EXT_SOURCE_3'],WOE_['DAYS_CREDIT' 'sum'],WOE_['DAYS_CREDIT' 'min'],WOE_['DAYS_CREDIT' 'max'],WOE_['DAYS_CREDIT' 'mean'],WOE_['DAYS_CREDIT_ENDDATE' 'sum'],WOE_['DAYS_CREDIT_ENDDATE' 'min'],WOE_['DAYS_CREDIT_ENDDATE' 'max'],WOE_['DAYS_CREDIT_ENDDATE' 'mean'],WOE_['DAYS_ENDDATE_FACT' 'sum'],WOE_['DAYS_ENDDATE_FACT' 'min'],WOE_['DAYS_ENDDATE_FACT' 'mean'],WOE_['AMT_CREDIT_SUM_DEBT' 'mean'],WOE_['AMT_CREDIT_SUM_LIMIT' 'max'],WOE_['DAYS_CREDIT_UPDATE' 'sum'],WOE_['DAYS_CREDIT_UPDATE' 'min'],WOE_['DAYS_CREDIT_UPDATE' 'mean'],WOE_['CREDIT_ACTIVE_Active' 'sum'],WOE_['CREDIT_ACTIVE_Active' 'mean'],WOE_['CREDIT_ACTIVE_Closed' 'mean'],WOE_['AMT_DRAWINGS_ATM_CURRENT' 'count'],WOE_['AMT_DRAWINGS_CURRENT' 'mean'],WOE_['AMT_DRAWINGS_OTHER_CURRENT' 'count'],WOE_['AMT_DRAWINGS_POS_CURRENT' 'count'],WOE_['AMT_PAYMENT_CURRENT' 'count'],WOE_['CNT_DRAWINGS_ATM_CURRENT' 'count'],WOE_['CNT_DRAWINGS_ATM_CURRENT' 'mean'],WOE_['CNT_DRAWINGS_CURRENT' 'max'],WOE_['CNT_DRAWINGS_CURRENT' 'mean'],WOE_['CNT_DRAWINGS_OTHER_CURRENT' 'count'],WOE_['CNT_DRAWINGS_POS_CURRENT' 'count'],WOE_['AMT_DOWN_PAYMENT' 'sum'],WOE_['DAYS_LAST_DUE_1ST_VERSION' 'sum'],WOE_['NAME_CONTRACT_STATUS_Approved' 'mean'],WOE_['NAME_CONTRACT_STATUS_Refused' 'mean'],WOE_['CODE_REJECT_REASON_XAP' 'mean'],WOE_['DAYS_INSTALMENT' 'min'],WOE_['DAYS_ENTRY_PAYMENT' 'min']
count,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000,276759.000000
mean,0.080597,0.024385,0.043595,0.025134,0.022356,0.037015,0.043580,0.034180,0.021526,0.032587,0.066577,0.128371,0.139453,0.021049,0.032364,0.035006,0.052163,0.033959,0.024655,0.021769,0.031109,0.025541,0.028748,0.029979,0.023593,0.019688,0.022444,0.026059,0.040934,0.021088,0.038389,0.038785,0.022937,0.020400,0.022937,0.022937,0.022614,0.022937,0.018878,0.020133,0.024336,0.022937,0.022937,0.022722,0.023447,0.025889,0.027873,0.025966,0.021463,0.021383
std,0.272216,0.238471,0.323255,0.250190,0.238981,0.300766,0.325213,0.283176,0.229856,0.282580,0.414485,0.549737,0.571521,0.221729,0.275911,0.288644,0.351717,0.287830,0.243170,0.226734,0.276380,0.246862,0.262748,0.267988,0.241354,0.209163,0.230722,0.249408,0.315639,0.219955,0.303963,0.306221,0.227429,0.216535,0.227429,0.227429,0.225481,0.227429,0.198361,0.212982,0.233816,0.227429,0.227429,0.236567,0.236148,0.241426,0.246986,0.240284,0.226545,0.226060
min,0.000000,-0.358265,-0.554948,-3.127408,-0.336842,-0.427731,-0.391887,-0.857654,-0.380451,-0.759910,-1.077892,-1.164800,-1.250475,-0.509828,-0.564015,-0.607509,-0.653885,-0.447632,-0.315583,-0.411374,-0.368336,-0.327070,-0.301615,-0.341884,-0.245107,-0.672442,-0.348276,-0.391633,-0.432981,-0.574296,-0.411651,-0.380719,-0.690678,-0.531448,-0.690678,-0.690678,-0.727409,-0.690678,-1.002693,-0.775708,-0.743923,-0.690678,-0.690678,-0.307704,-0.524248,-0.625698,-0.772721,-0.570855,-0.349549,-0.355076
25%,0.000000,-0.188568,-0.128924,-0.187342,-0.110769,-0.205246,-0.323482,-0.295216,0.021248,-0.161475,-0.054113,-0.149785,-0.157430,-0.202815,-0.244364,-0.224649,-0.244364,-0.244364,-0.247774,-0.177724,-0.239588,-0.244364,-0.284409,-0.284409,-0.190260,-0.172810,-0.244364,-0.226

In [123]:
xtrain=woetrain2.drop('TARGET',axis=1)
ytrain=woetrain2['TARGET']
xtest=woetest2.drop('TARGET',axis=1)
ytest=woetest2['TARGET']

In [129]:
logit=LogisticRegression(class_weight='balanced',penalty='none')

In [130]:
logit.fit(xtrain,ytrain)

D:\Software\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None,
                   penalty='none', random_state=None, solver='lbfgs',
                   tol=0.0001, verbose=0, warm_start=False)

In [131]:
pred=logit.predict(xtest)

In [132]:
print("Accuracy:",metrics.accuracy_score(ytest,pred))
print('Confusion matrix:\n',metrics.confusion_matrix(ytest,pred))
print('Precision score:',metrics.precision_score(ytest,pred))
print('Recall score:',metrics.recall_score(ytest,pred))
print('ROC:',metrics.roc_auc_score(ytest,pred))

Accuracy: 0.6859066077003122
Confusion matrix:
 [[19375  8858]
 [  801  1718]]
Precision score: 0.16244326777609683
Recall score: 0.6820166732830488
ROC: 0.6841351740303956


In [133]:
woetrain2.to_csv('D:\DS\BSFA\home-credit-default-risk\woetrain2.csv')
woetest2.to_csv('D:\DS\BSFA\home-credit-default-risk\woetest2.csv')

### Kaggle test prediction

In [134]:
appltest=pd.read_csv('D:/DS/BSFA/home-credit-default-risk/application_test.csv')

In [135]:
appltest.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,Unaccompanied,Working,Higher education,Married,House / apartment,0.018850,-19241,-2329,-5170.0,-812,NaN,1,1,0,1,0,1,NaN,2.0,2,2,TUESDAY,18,0,0,0,0,0,0,Kindergarten,0.752614,0.789654,0.159520,0.0660,0.0590,0.9732,NaN,NaN,NaN,0.1379,0.125,NaN,NaN,NaN,0.0505,NaN,NaN,0.0672,0.0612,0.9732,NaN,NaN,NaN,0.1379,0.125,NaN,NaN,NaN,0.0526,NaN,NaN,0.0666,0.0590,0.9732,NaN,NaN,NaN,0.1379,0.125,NaN,NaN,NaN,0.0514,NaN,NaN,NaN,block of flats,0.0392,"Stone, brick",No,0.0,0.0,0.0,0.0,-1740.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.035792,-18064,-4469,-9118.0,-1623,NaN,1,1,0,1,0,0,Low-skill Laborers,2.0,2,2,FRIDAY,9,0,0,0,0,0,0,Self-employed,0.564990,0.291656,0.432962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,NaN,Working,Higher education,Married,House / apartment,0.019101,-20038,-4458,-2175.0,-3503,5.0,1,1,0,1,0,0,Drivers,2.0,2,2,MONDAY,14,0,0,0,0,0,0,Transport: type 3,NaN,0.699787,0.610991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-856.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,Unaccompanied,Working,Secondary / secondary special,Married

In [139]:
kagtest=appltest.loc[:,['AMT_CREDIT',
 'AMT_GOODS_PRICE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'OCCUPATION_TYPE',
 'REGION_RATING_CLIENT_W_CITY',
 'ORGANIZATION_TYPE',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3','SK_ID_CURR']]
kagtest.head()

,AMT_CREDIT,AMT_GOODS_PRICE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,OCCUPATION_TYPE,REGION_RATING_CLIENT_W_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,SK_ID_CURR
0,568800.0,450000.0,Working,Higher education,-19241,-2329,NaN,2,Kindergarten,0.752614,0.789654,0.159520,100001
1,222768.0,180000.0,Working,Secondary / secondary special,-18064,-4469,Low-skill Laborers,2,Self-employed,0.564990,0.291656,0.432962,100005
2,663264.0,630000.0,Working,Higher education,-20038,-4458,Drivers,2,Transport: type 3,NaN,0.699787,0.610991,100013
3,1575000.0,1575000.0,Working,Secondary / secondary special,-13976,-1866,Sales staff,2,Business Entity Type 3,0.525734,0.509677,0.612704,100028
4,625500.0,625500.0,Working,Secondary / secondary special,-13040,-2191,NaN,2,Business Entity Type 3,0.202145,0.425687,NaN,100038


In [141]:
kag=kagtest[['SK_ID_CURR']]
kag.head()

,SK_ID_CURR
0,100001
1,100005
2,100013
3,100028
4,100038


In [142]:
bureau=pd.read_csv('bureau.csv')
bureaubal=pd.read_csv('bureau_balance.csv')

In [143]:
bureaubal=pd.get_dummies(bureaubal)
bureaubal.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X
0,5715448,0,0,0,0,0,0,0,1,0
1,5715448,-1,0,0,0,0,0,0,1,0
2,5715448,-2,0,0,0,0,0,0,1,0
3,5715448,-3,0,0,0,0,0,0,1,0
4,5715448,-4,0,0,0,0,0,0,1,0


In [144]:
bureaubalagg=bureaubal.groupby('SK_ID_BUREAU')
bureaubalagg=bureaubalagg.agg(['sum','count','min','max','mean'])
bureaubalagg.reset_index(inplace=True)
bureaubalagg.head()

SK_ID_BUREAU MONTHS_BALANCE                     STATUS_0                \
                          sum count min max  mean      sum count min max   
0      5001709          -4656    97 -96   0 -48.0        0    97   0   0   
1      5001710          -3403    83 -82   0 -41.0        5    83   0   1   
2      5001711             -6     4  -3   0  -1.5        3     4   0   1   
3      5001712           -171    19 -18   0  -9.0       10    19   0   1   
4      5001713           -231    22 -21   0 -10.5        0    22   0   0   

            STATUS_1                    STATUS_2                    STATUS_3  \
       mean      sum count min max mean      sum count min max mean      sum   
0  0.000000        0    97   0   0  0.0        0    97   0   0  0.0        0   
1  0.060241        0    83   0   0  0.0        0    83   0   0  0.0        0   
2  0.750000        0     4   0   0  0.0        0     4   0   0  0.0        0   
3  0.526316        0    19   0   0  0.0        0    19   0   0  0.0        0   
4  0.000000        0    22   0   0  0.0        0    22   0   0  0.0        0   

                     STATUS_4                    STATUS_5                     \
  count min max mean      sum count min max mean      sum count min max mean   
0    97   0   0  0.0        0    97   0   0  0.0        0    97   0   0  0.0   
1    83   0   0  0.0        0    83   0   0  0.0        0    83   0   0  0.0   
2     4   0   0  0.0        0     4   0   0  0.0        0     4   0   0  0.0   
3    19   0   0  0.0        0    19   0   0  0.0        0    19   0   0  0.0   
4    22   0   0  0.0        0    22   0   0  0.0        0    22   0   0  0.0   

  STATUS_C                         STATUS_X                          
       sum count min max      mean      sum count min max      mean  
0       86    97   0   1  0.886598       11    97   0   1  0.113402  
1       48    83   0   1  0.578313       30    83   0   1  0.361446  
2        0     4   0   0  0.000000        1     4   0   1  0.250000  
3        9    19   0   1  0.473684        0    19   0   0  0.000000  
4        0    22   0   0  0.000000       22    22   1   1  1.000000

In [145]:
bureau=bureau.merge(bureaubalagg,'left','SK_ID_BUREAU')
bureau=pd.get_dummies(bureau.drop('SK_ID_BUREAU',axis=1))
bureau=bureau.groupby('SK_ID_CURR').agg(['sum','count','min','max','mean'])
bureau.reset_index(inplace=True)
bureau.head()

D:\Software\Anaconda\lib\site-packages\pandas\core\reshape\merge.py:617: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
D:\Software\Anaconda\lib\site-packages\pandas\core\generic.py:3946: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


SK_ID_CURR DAYS_CREDIT                               CREDIT_DAY_OVERDUE  \
                     sum count   min  max         mean                sum   
0     100001       -5145     7 -1572  -49  -735.000000                  0   
1     100002       -6992     8 -1437 -103  -874.000000                  0   
2     100003       -5603     4 -2586 -606 -1400.750000                  0   
3     100004       -1734     2 -1326 -408  -867.000000                  0   
4     100005        -572     3  -373  -62  -190.666667                  0   

                     DAYS_CREDIT_ENDDATE                                    \
  count min max mean                 sum count     min     max        mean   
0     7   0   0  0.0               577.0     7 -1329.0  1778.0   82.428571   
1     8   0   0  0.0             -2094.0     6 -1072.0   780.0 -349.000000   
2     4   0   0  0.0             -2178.0     4 -2434.0  1216.0 -544.500000   
3     2   0   0  0.0              -977.0     2  -595.0  -382.0 -488.500000   
4     3   0   0  0.0              1318.0     3  -128.0  1324.0  439.333333   

  DAYS_ENDDATE_FACT                                   AMT_CREDIT_MAX_OVERDUE  \
                sum count     min    max         mean                    sum   
0           -3302.0     4 -1328.0 -544.0  -825.500000                  0.000   
1           -4185.0     6 -1185.0  -36.0  -697.500000               8405.145   
2           -3292.0     3 -2131.0 -540.0 -1097.333333                  0.000   
3           -1065.0     2  -683.0 -382.0  -532.500000                  0.000   
4            -123.0     1  -123.0 -123.0  -123.000000                  0.000   

                                 CNT_CREDIT_PROLONG                     \
  count  min       max      mean                sum count min max mean   
0     0  NaN       NaN       NaN                  0     7   0   0  0.0   
1     5  0.0  5043.645  1681.029                  0     8   0   0  0.0   
2     4  0.0     0.000     0.000                  0     4   0   0  0.0   
3     1  0.0     0.000     0.000                  0     2   0   0  0.0   
4     1  0.0     0.000     0.000                  0     3   0   0  0.0   

  AMT_CREDIT_SUM                                         AMT_CREDIT_SUM_DEBT  \
             sum count      min       max           mean                 sum   
0    1453365.000     7  85500.0  378000.0  207623.571429            596686.5   
1     865055.565     8      0.0  450000.0  108131.945625            245781.0   
2    1017400.500     4  22248.0  810000.0  254350.125000                 0.0   
3     189037.800     2  94500.0   94537.8   94518.900000                 0.0   
4     657126.000     3  29826.0  568800.0  219042.000000            568408.5   

                                      AMT_CREDIT_SUM_LIMIT             \
  count  min       max           mean                  sum count  min   
0     7  0.0  373239.0   85240.928571                0.000     6  0.0   
1     5  0.0  245781.0   49156.200000            31988.565     4  0.0   
2     4  0.0       0.0       0.000000           810000.000     4  0.0   
3     2  0.0       0.0       0.000000                0.000     2  0.0   
4     3  0.0  543087.0  189469.500000                0.000     3  0.0   

                            AMT_CREDIT_SUM_OVERDUE                       \
          max          mean                    sum count  min  max mean   
0       0.000       0.00000                    0.0     7  0.0  0.0  0.0   
1   31988.565    7997.14125                    0.0     8  0.0  0.0  0.0   
2  810000.000  202500.00000                    0.0     4  0.0  0.0  0.0   
3       0.000       0.00000                    0.0     2  0.0  0.0  0.0   
4       0.000       0.00000                    0.0     3  0.0  0.0  0.0   

  DAYS_CREDIT_UPDATE                              AMT_ANNUITY             \
                 sum count   min  max        mean         sum count  min   
0               -652     7  -155   -6  -93.142857     24817.5     7  0.0   
1       

In [150]:
bureaut=kag.merge(bureau,'left','SK_ID_CURR')
bureaut.head()

,SK_ID_CURR,"(DAYS_CREDIT, sum)","(DAYS_CREDIT, count)","(DAYS_CREDIT, min)","(DAYS_CREDIT, max)","(DAYS_CREDIT, mean)","(CREDIT_DAY_OVERDUE, sum)","(CREDIT_DAY_OVERDUE, count)","(CREDIT_DAY_OVERDUE, min)","(CREDIT_DAY_OVERDUE, max)","(CREDIT_DAY_OVERDUE, mean)","(DAYS_CREDIT_ENDDATE, sum)","(DAYS_CREDIT_ENDDATE, count)","(DAYS_CREDIT_ENDDATE, min)","(DAYS_CREDIT_ENDDATE, max)","(DAYS_CREDIT_ENDDATE, mean)","(DAYS_ENDDATE_FACT, sum)","(DAYS_ENDDATE_FACT, count)","(DAYS_ENDDATE_FACT, min)","(DAYS_ENDDATE_FACT, max)","(DAYS_ENDDATE_FACT, mean)","(AMT_CREDIT_MAX_OVERDUE, sum)","(AMT_CREDIT_MAX_OVERDUE, count)","(AMT_CREDIT_MAX_OVERDUE, min)","(AMT_CREDIT_MAX_OVERDUE, max)","(AMT_CREDIT_MAX_OVERDUE, mean)","(CNT_CREDIT_PROLONG, sum)","(CNT_CREDIT_PROLONG, count)","(CNT_CREDIT_PROLONG, min)","(CNT_CREDIT_PROLONG, max)","(CNT_CREDIT_PROLONG, mean)","(AMT_CREDIT_SUM, sum)","(AMT_CREDIT_SUM, count)","(AMT_CREDIT_SUM, min)","(AMT_CREDIT_SUM, max)","(AMT_CREDIT_SUM, mean)","(AMT_CREDIT_SUM_DEBT, sum)","(AMT_CREDIT_SUM_DEBT, count)","(AMT_CREDIT_SUM_DEBT, min)","(AMT_CREDIT_SUM_DEBT, max)","(AMT_CREDIT_SUM_DEBT, mean)","(AMT_CREDIT_SUM_LIMIT, sum)","(AMT_CREDIT_SUM_LIMIT, count)","(AMT_CREDIT_SUM_LIMIT, min)","(AMT_CREDIT_SUM_LIMIT, max)","(AMT_CREDIT_SUM_LIMIT, mean)","(AMT_CREDIT_SUM_OVERDUE, sum)","(AMT_CREDIT_SUM_OVERDUE, count)","(AMT_CREDIT_SUM_OVERDUE, min)","(AMT_CREDIT_SUM_OVERDUE, max)","(AMT_CREDIT_SUM_OVERDUE, mean)","(DAYS_CREDIT_UPDATE, sum)","(DAYS_CREDIT_UPDATE, count)","(DAYS_CREDIT_UPDATE, min)","(DAYS_CREDIT_UPDATE, max)","(DAYS_CREDIT_UPDATE, mean)","(AMT_ANNUITY, sum)","(AMT_ANNUITY, count)","(AMT_ANNUITY, min)","(AMT_ANNUITY, max)","(AMT_ANNUITY, mean)","((MONTHS_BALANCE, sum), sum)",...,"(CREDIT_TYPE_Cash loan (non-earmarked), max)","(CREDIT_TYPE_Cash loan (non-earmarked), mean)","(CREDIT_TYPE_Consumer credit, sum)","(CREDIT_TYPE_Consumer credit, count)","(CREDIT_TYPE_Consumer credit, min)","(CREDIT_TYPE_Consumer credit, max)","(CREDIT_TYPE_Consumer credit, mean)","(CREDIT_TYPE_Credit card, sum)","(CREDIT_TYPE_Credit card, count)","(CREDIT_TYPE_Credit card, min)","(CREDIT_TYPE_Credit card, max)","(CREDIT_TYPE_Credit card, mean)","(CREDIT_TYPE_Interbank credit, sum)","(CREDIT_TYPE_Interbank credit, count)","(CREDIT_TYPE_Interbank credit, min)","(CREDIT_TYPE_Interbank credit, max)","(CREDIT_TYPE_Interbank credit, mean)","(CREDIT_TYPE_Loan for business development, sum)","(CREDIT_TYPE_Loan for business development, count)","(CREDIT_TYPE_Loan for business development, min)","(CREDIT_TYPE_Loan for business development, max)","(CREDIT_TYPE_Loan for business development, mean)","(CREDIT_TYPE_Loan for purchase of shares (margin lending), sum)","(CREDIT_TYPE_Loan for purchase of shares (margin lending), count)","(CREDIT_TYPE_Loan for purchase of shares (margin lending), min)","(CREDIT_TYPE_Loan for purchase of shares (margin lending), max)","(CREDIT_TYPE_Loan for purchase of shares (margin lending), mean)","(CREDIT_TYPE_Loan for the purchase of equipment, sum)","(CREDIT_TYPE_Loan for the purchase of equipment, count)","(CREDIT_TYPE_Loan for the purchase of equipment, min)","(CREDIT_TYPE_Loan for the purchase of equipment, max)","(CREDIT_TYPE_Loan for the purchase of equipment, mean)","(CREDIT_TYPE_Loan for working capital replenishment, sum)","(CREDIT_TYPE_Loan for working capital replenishment, count)","(CREDIT_TYPE_Loan for working capital replenishment, min)","(CREDIT_TYPE_Loan for working capital replenishment, max)","(CREDIT_TYPE_Loan for working capital replenishment, mean)","(CREDIT_TYPE_Microloan, sum)","(CREDIT_TYPE_Microloan, count)","(CREDIT_TYPE_Microloan, min)","(CREDIT_TYPE_Microloan, max)","(CREDIT_TYPE_Microloan, mean)","(CREDIT_TYPE_Mobile operator loan, sum)","(CREDIT_TYPE_Mobile operator loan, count)","(CREDIT_TYPE_Mobile operator loan, min)","(CREDIT_TYPE_Mobile operator loan, max)","(CREDIT_TYPE_Mobile operator loan, mean)","(CREDIT_TYPE_Mortgage, sum)","(CREDIT_TYPE_Mortgage, count)","(CREDIT_TYPE_Mort

#### Prevapp

In [152]:
prevapp=pd.read_csv('previous_application.csv')
prevapp.drop('SK_ID_PREV',axis=1,inplace=True)
prevapp=pd.get_dummies(prevapp)
prevapp=prevapp.groupby('SK_ID_CURR').agg(['sum','count','min','max','mean'])
prevapp.reset_index(inplace=True)
prevappkag=kag.merge(prevapp,'left','SK_ID_CURR')
prevappkag.head()

,SK_ID_CURR,"(AMT_ANNUITY, sum)","(AMT_ANNUITY, count)","(AMT_ANNUITY, min)","(AMT_ANNUITY, max)","(AMT_ANNUITY, mean)","(AMT_APPLICATION, sum)","(AMT_APPLICATION, count)","(AMT_APPLICATION, min)","(AMT_APPLICATION, max)","(AMT_APPLICATION, mean)","(AMT_CREDIT, sum)","(AMT_CREDIT, count)","(AMT_CREDIT, min)","(AMT_CREDIT, max)","(AMT_CREDIT, mean)","(AMT_DOWN_PAYMENT, sum)","(AMT_DOWN_PAYMENT, count)","(AMT_DOWN_PAYMENT, min)","(AMT_DOWN_PAYMENT, max)","(AMT_DOWN_PAYMENT, mean)","(AMT_GOODS_PRICE, sum)","(AMT_GOODS_PRICE, count)","(AMT_GOODS_PRICE, min)","(AMT_GOODS_PRICE, max)","(AMT_GOODS_PRICE, mean)","(HOUR_APPR_PROCESS_START, sum)","(HOUR_APPR_PROCESS_START, count)","(HOUR_APPR_PROCESS_START, min)","(HOUR_APPR_PROCESS_START, max)","(HOUR_APPR_PROCESS_START, mean)","(NFLAG_LAST_APPL_IN_DAY, sum)","(NFLAG_LAST_APPL_IN_DAY, count)","(NFLAG_LAST_APPL_IN_DAY, min)","(NFLAG_LAST_APPL_IN_DAY, max)","(NFLAG_LAST_APPL_IN_DAY, mean)","(RATE_DOWN_PAYMENT, sum)","(RATE_DOWN_PAYMENT, count)","(RATE_DOWN_PAYMENT, min)","(RATE_DOWN_PAYMENT, max)","(RATE_DOWN_PAYMENT, mean)","(RATE_INTEREST_PRIMARY, sum)","(RATE_INTEREST_PRIMARY, count)","(RATE_INTEREST_PRIMARY, min)","(RATE_INTEREST_PRIMARY, max)","(RATE_INTEREST_PRIMARY, mean)","(RATE_INTEREST_PRIVILEGED, sum)","(RATE_INTEREST_PRIVILEGED, count)","(RATE_INTEREST_PRIVILEGED, min)","(RATE_INTEREST_PRIVILEGED, max)","(RATE_INTEREST_PRIVILEGED, mean)","(DAYS_DECISION, sum)","(DAYS_DECISION, count)","(DAYS_DECISION, min)","(DAYS_DECISION, max)","(DAYS_DECISION, mean)","(SELLERPLACE_AREA, sum)","(SELLERPLACE_AREA, count)","(SELLERPLACE_AREA, min)","(SELLERPLACE_AREA, max)","(SELLERPLACE_AREA, mean)","(CNT_PAYMENT, sum)",...,"(PRODUCT_COMBINATION_Cash Street: low, max)","(PRODUCT_COMBINATION_Cash Street: low, mean)","(PRODUCT_COMBINATION_Cash Street: middle, sum)","(PRODUCT_COMBINATION_Cash Street: middle, count)","(PRODUCT_COMBINATION_Cash Street: middle, min)","(PRODUCT_COMBINATION_Cash Street: middle, max)","(PRODUCT_COMBINATION_Cash Street: middle, mean)","(PRODUCT_COMBINATION_Cash X-Sell: high, sum)","(PRODUCT_COMBINATION_Cash X-Sell: high, count)","(PRODUCT_COMBINATION_Cash X-Sell: high, min)","(PRODUCT_COMBINATION_Cash X-Sell: high, max)","(PRODUCT_COMBINATION_Cash X-Sell: high, mean)","(PRODUCT_COMBINATION_Cash X-Sell: low, sum)","(PRODUCT_COMBINATION_Cash X-Sell: low, count)","(PRODUCT_COMBINATION_Cash X-Sell: low, min)","(PRODUCT_COMBINATION_Cash X-Sell: low, max)","(PRODUCT_COMBINATION_Cash X-Sell: low, mean)","(PRODUCT_COMBINATION_Cash X-Sell: middle, sum)","(PRODUCT_COMBINATION_Cash X-Sell: middle, count)","(PRODUCT_COMBINATION_Cash X-Sell: middle, min)","(PRODUCT_COMBINATION_Cash X-Sell: middle, max)","(PRODUCT_COMBINATION_Cash X-Sell: middle, mean)","(PRODUCT_COMBINATION_POS household with interest, sum)","(PRODUCT_COMBINATION_POS household with interest, count)","(PRODUCT_COMBINATION_POS household with interest, min)","(PRODUCT_COMBINATION_POS household with interest, max)","(PRODUCT_COMBINATION_POS household with interest, mean)","(PRODUCT_COMBINATION_POS household without interest, sum)","(PRODUCT_COMBINATION_POS household without interest, count)","(PRODUCT_COMBINATION_POS household without interest, min)","(PRODUCT_COMBINATION_POS household without interest, max)","(PRODUCT_COMBINATION_POS household without interest, mean)","(PRODUCT_COMBINATION_POS industry with interest, sum)","(PRODUCT_COMBINATION_POS industry with interest, count)","(PRODUCT_COMBINATION_POS industry with interest, min)","(PRODUCT_COMBINATION_POS industry with interest, max)","(PRODUCT_COMBINATION_POS industry with interest, mean)","(PRODUCT_COMBINATION_POS industry without interest, sum)","(PRODUCT_COMBINATION_POS industry without interest, count)","(PRODUCT_COMBINATION_POS industry without interest, min)","(PRODUCT_COMBINATION_POS industry without interest, max)","(PRODUCT_COMBINATION_POS industry without interest, mean)","(PRODUCT_COMBINATION_POS mobile with interest, sum)","(PRODUCT_COMBINATION_POS 

#### Poscash

In [153]:
poscash=pd.read_csv('POS_CASH_balance.csv')
poscash.drop('SK_ID_PREV',axis=1,inplace=True)
poscash=pd.get_dummies(poscash)
poscash=poscash.groupby('SK_ID_CURR').agg(['sum','count','min','max','mean'])
poscash.reset_index(inplace=True)
poscashkag=kag.merge(poscash,'left','SK_ID_CURR')
poscashkag.head()

,SK_ID_CURR,"(MONTHS_BALANCE, sum)","(MONTHS_BALANCE, count)","(MONTHS_BALANCE, min)","(MONTHS_BALANCE, max)","(MONTHS_BALANCE, mean)","(CNT_INSTALMENT, sum)","(CNT_INSTALMENT, count)","(CNT_INSTALMENT, min)","(CNT_INSTALMENT, max)","(CNT_INSTALMENT, mean)","(CNT_INSTALMENT_FUTURE, sum)","(CNT_INSTALMENT_FUTURE, count)","(CNT_INSTALMENT_FUTURE, min)","(CNT_INSTALMENT_FUTURE, max)","(CNT_INSTALMENT_FUTURE, mean)","(SK_DPD, sum)","(SK_DPD, count)","(SK_DPD, min)","(SK_DPD, max)","(SK_DPD, mean)","(SK_DPD_DEF, sum)","(SK_DPD_DEF, count)","(SK_DPD_DEF, min)","(SK_DPD_DEF, max)","(SK_DPD_DEF, mean)","(NAME_CONTRACT_STATUS_Active, sum)","(NAME_CONTRACT_STATUS_Active, count)","(NAME_CONTRACT_STATUS_Active, min)","(NAME_CONTRACT_STATUS_Active, max)","(NAME_CONTRACT_STATUS_Active, mean)","(NAME_CONTRACT_STATUS_Amortized debt, sum)","(NAME_CONTRACT_STATUS_Amortized debt, count)","(NAME_CONTRACT_STATUS_Amortized debt, min)","(NAME_CONTRACT_STATUS_Amortized debt, max)","(NAME_CONTRACT_STATUS_Amortized debt, mean)","(NAME_CONTRACT_STATUS_Approved, sum)","(NAME_CONTRACT_STATUS_Approved, count)","(NAME_CONTRACT_STATUS_Approved, min)","(NAME_CONTRACT_STATUS_Approved, max)","(NAME_CONTRACT_STATUS_Approved, mean)","(NAME_CONTRACT_STATUS_Canceled, sum)","(NAME_CONTRACT_STATUS_Canceled, count)","(NAME_CONTRACT_STATUS_Canceled, min)","(NAME_CONTRACT_STATUS_Canceled, max)","(NAME_CONTRACT_STATUS_Canceled, mean)","(NAME_CONTRACT_STATUS_Completed, sum)","(NAME_CONTRACT_STATUS_Completed, count)","(NAME_CONTRACT_STATUS_Completed, min)","(NAME_CONTRACT_STATUS_Completed, max)","(NAME_CONTRACT_STATUS_Completed, mean)","(NAME_CONTRACT_STATUS_Demand, sum)","(NAME_CONTRACT_STATUS_Demand, count)","(NAME_CONTRACT_STATUS_Demand, min)","(NAME_CONTRACT_STATUS_Demand, max)","(NAME_CONTRACT_STATUS_Demand, mean)","(NAME_CONTRACT_STATUS_Returned to the store, sum)","(NAME_CONTRACT_STATUS_Returned to the store, count)","(NAME_CONTRACT_STATUS_Returned to the store, min)","(NAME_CONTRACT_STATUS_Returned to the store, max)","(NAME_CONTRACT_STATUS_Returned to the store, mean)","(NAME_CONTRACT_STATUS_Signed, sum)","(NAME_CONTRACT_STATUS_Signed, count)","(NAME_CONTRACT_STATUS_Signed, min)","(NAME_CONTRACT_STATUS_Signed, max)","(NAME_CONTRACT_STATUS_Signed, mean)","(NAME_CONTRACT_STATUS_XNA, sum)","(NAME_CONTRACT_STATUS_XNA, count)","(NAME_CONTRACT_STATUS_XNA, min)","(NAME_CONTRACT_STATUS_XNA, max)","(NAME_CONTRACT_STATUS_XNA, mean)"
0,100001,-653.0,9.0,-96.0,-53.0,-72.555556,36.0,9.0,4.0,4.0,4.000000,13.0,9.0,0.0,4.0,1.444444,7.0,9.0,0.0,7.0,0.777778,7.0,9.0,0.0,7.0,0.777778,7.0,9.0,0.0,1.0,0.777778,0.0,9.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,2.0,9.0,0.0,1.0,0.222222,0.0,9.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.000000,0.0,9.0,0.0,0.0,0.0
1,100005,-220.0,11.0,-25.0,-15.0,-20.000000,117.0,10.0,9.0,12.0,11.700000,72.0,10.0,0.0,12.0,7.200000,0.0,11.0,0.0,0.0,0.000000,0.0,11.0,0.0,0.0,0.000000,9.0,11.0,0.0,1.0,0.818182,0.0,11.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,1.0,11.0,0.0,1.0,0.090909,0.0,11.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,1.0,11.0,0.0,1.0,0.090909,0.0,11.0,0.0,0.0,0.0
2,100013,-1064.0,36.0,-66.0,-3.0,-29.555556,770.0,36.0,6.0,36.0,21.388889,551.0,36.0,0.0,36.0,15.305556,34.0,36.0,0.0,18.0,0.944444,0.0,36.0,0.0,0.0,0.000000,32.0,36.0,0.0,1.0,0.888889,0.0,36.0,0.0,0.0,0.0,0.0,36.0,0.0,0.0,0.0,0.0,36.0,0.0,0.0,0.0,3.0,36.0,0.0,1.0,0.083333,0.0,36.0,0.0,0.0,0.0,0.0,36.0,0.0,0.0,0.0,1.0,36.0,0.0,1.0,0.027778,0.0,36.0,0.0,0.0,0.0
3,100028,-1301.0,31.0,-59.0,-20.0,-41.967742,631.0,31.0,5.0,24.0,20.354839,314.0,31.0,0.0,24.0,10.129032,0.0,31.0,0.0,0.0,0.000000,0.0,31.0,0.0,0.0,0.000000,29.0,31.0,0.0,1.0,0.935484,0.0,31.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,2.0,31.0,0.0,1.0,0.064516,0.0,31.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.000000,0.0,31.0,0.0,0.0,0.0
4,100038,-273.0,13.0,-27.0,-15.0,-21.000000,156.0,13.0,12.0,12.0,12.000000,76.0,13.0,0.0,12.0,5.846154,0.0,13.0,0.0,0.0,0.000000,0.0,13.0,0.0,0.0,0.

#### CCbalance

In [154]:
ccbalance=pd.read_csv('credit_card_balance.csv')
ccbalance.drop('SK_ID_PREV',axis=1,inplace=True)
ccbalance=pd.get_dummies(ccbalance)
ccbalance=ccbalance.groupby('SK_ID_CURR').agg(['sum','count','min','max','mean'])
ccbalance.reset_index(inplace=True)
ccbalancekag=test.merge(ccbalance,'left','SK_ID_CURR')
#ccbalancetest.dropna(subset=['TARGET'],inplace=True)
ccbalancekag.head()

,TARGET,SK_ID_CURR,"(MONTHS_BALANCE, sum)","(MONTHS_BALANCE, count)","(MONTHS_BALANCE, min)","(MONTHS_BALANCE, max)","(MONTHS_BALANCE, mean)","(AMT_BALANCE, sum)","(AMT_BALANCE, count)","(AMT_BALANCE, min)","(AMT_BALANCE, max)","(AMT_BALANCE, mean)","(AMT_CREDIT_LIMIT_ACTUAL, sum)","(AMT_CREDIT_LIMIT_ACTUAL, count)","(AMT_CREDIT_LIMIT_ACTUAL, min)","(AMT_CREDIT_LIMIT_ACTUAL, max)","(AMT_CREDIT_LIMIT_ACTUAL, mean)","(AMT_DRAWINGS_ATM_CURRENT, sum)","(AMT_DRAWINGS_ATM_CURRENT, count)","(AMT_DRAWINGS_ATM_CURRENT, min)","(AMT_DRAWINGS_ATM_CURRENT, max)","(AMT_DRAWINGS_ATM_CURRENT, mean)","(AMT_DRAWINGS_CURRENT, sum)","(AMT_DRAWINGS_CURRENT, count)","(AMT_DRAWINGS_CURRENT, min)","(AMT_DRAWINGS_CURRENT, max)","(AMT_DRAWINGS_CURRENT, mean)","(AMT_DRAWINGS_OTHER_CURRENT, sum)","(AMT_DRAWINGS_OTHER_CURRENT, count)","(AMT_DRAWINGS_OTHER_CURRENT, min)","(AMT_DRAWINGS_OTHER_CURRENT, max)","(AMT_DRAWINGS_OTHER_CURRENT, mean)","(AMT_DRAWINGS_POS_CURRENT, sum)","(AMT_DRAWINGS_POS_CURRENT, count)","(AMT_DRAWINGS_POS_CURRENT, min)","(AMT_DRAWINGS_POS_CURRENT, max)","(AMT_DRAWINGS_POS_CURRENT, mean)","(AMT_INST_MIN_REGULARITY, sum)","(AMT_INST_MIN_REGULARITY, count)","(AMT_INST_MIN_REGULARITY, min)","(AMT_INST_MIN_REGULARITY, max)","(AMT_INST_MIN_REGULARITY, mean)","(AMT_PAYMENT_CURRENT, sum)","(AMT_PAYMENT_CURRENT, count)","(AMT_PAYMENT_CURRENT, min)","(AMT_PAYMENT_CURRENT, max)","(AMT_PAYMENT_CURRENT, mean)","(AMT_PAYMENT_TOTAL_CURRENT, sum)","(AMT_PAYMENT_TOTAL_CURRENT, count)","(AMT_PAYMENT_TOTAL_CURRENT, min)","(AMT_PAYMENT_TOTAL_CURRENT, max)","(AMT_PAYMENT_TOTAL_CURRENT, mean)","(AMT_RECEIVABLE_PRINCIPAL, sum)","(AMT_RECEIVABLE_PRINCIPAL, count)","(AMT_RECEIVABLE_PRINCIPAL, min)","(AMT_RECEIVABLE_PRINCIPAL, max)","(AMT_RECEIVABLE_PRINCIPAL, mean)","(AMT_RECIVABLE, sum)","(AMT_RECIVABLE, count)","(AMT_RECIVABLE, min)","(AMT_RECIVABLE, max)","(AMT_RECIVABLE, mean)",...,"(CNT_DRAWINGS_CURRENT, max)","(CNT_DRAWINGS_CURRENT, mean)","(CNT_DRAWINGS_OTHER_CURRENT, sum)","(CNT_DRAWINGS_OTHER_CURRENT, count)","(CNT_DRAWINGS_OTHER_CURRENT, min)","(CNT_DRAWINGS_OTHER_CURRENT, max)","(CNT_DRAWINGS_OTHER_CURRENT, mean)","(CNT_DRAWINGS_POS_CURRENT, sum)","(CNT_DRAWINGS_POS_CURRENT, count)","(CNT_DRAWINGS_POS_CURRENT, min)","(CNT_DRAWINGS_POS_CURRENT, max)","(CNT_DRAWINGS_POS_CURRENT, mean)","(CNT_INSTALMENT_MATURE_CUM, sum)","(CNT_INSTALMENT_MATURE_CUM, count)","(CNT_INSTALMENT_MATURE_CUM, min)","(CNT_INSTALMENT_MATURE_CUM, max)","(CNT_INSTALMENT_MATURE_CUM, mean)","(SK_DPD, sum)","(SK_DPD, count)","(SK_DPD, min)","(SK_DPD, max)","(SK_DPD, mean)","(SK_DPD_DEF, sum)","(SK_DPD_DEF, count)","(SK_DPD_DEF, min)","(SK_DPD_DEF, max)","(SK_DPD_DEF, mean)","(NAME_CONTRACT_STATUS_Active, sum)","(NAME_CONTRACT_STATUS_Active, count)","(NAME_CONTRACT_STATUS_Active, min)","(NAME_CONTRACT_STATUS_Active, max)","(NAME_CONTRACT_STATUS_Active, mean)","(NAME_CONTRACT_STATUS_Approved, sum)","(NAME_CONTRACT_STATUS_Approved, count)","(NAME_CONTRACT_STATUS_Approved, min)","(NAME_CONTRACT_STATUS_Approved, max)","(NAME_CONTRACT_STATUS_Approved, mean)","(NAME_CONTRACT_STATUS_Completed, sum)","(NAME_CONTRACT_STATUS_Completed, count)","(NAME_CONTRACT_STATUS_Completed, min)","(NAME_CONTRACT_STATUS_Completed, max)","(NAME_CONTRACT_STATUS_Completed, mean)","(NAME_CONTRACT_STATUS_Demand, sum)","(NAME_CONTRACT_STATUS_Demand, count)","(NAME_CONTRACT_STATUS_Demand, min)","(NAME_CONTRACT_STATUS_Demand, max)","(NAME_CONTRACT_STATUS_Demand, mean)","(NAME_CONTRACT_STATUS_Refused, sum)","(NAME_CONTRACT_STATUS_Refused, count)","(NAME_CONTRACT_STATUS_Refused, min)","(NAME_CONTRACT_STATUS_Refused, max)","(NAME_CONTRACT_STATUS_Refused, mean)","(NAME_CONTRACT_STATUS_Sent proposal, sum)","(NAME_CONTRACT_STATUS_Sent proposal, count)","(NAME_CONTRACT_STATUS_Sent proposal, min)","(NAME_CONTRACT_STATUS_Sent proposal, max)","(NAME_CONTRACT_STATUS_Sent proposal, mean)","(NAME_CONTRACT_STATUS_Signed, sum)","(NAME_CONTRACT_STATUS_Signed, count)","(NAME_CONTRACT_STATUS_Signed, min)","(NAME_CONTRACT_STATUS_S

#### Inst payments

In [155]:
instpaym=pd.read_csv('installments_payments.csv')
instpaym.drop('SK_ID_PREV',axis=1,inplace=True)
instpaym=pd.get_dummies(instpaym)
instpaym=instpaym.groupby('SK_ID_CURR').agg(['sum','count','min','max','mean'])
instpaym.reset_index(inplace=True)
instpaymtkag=kag.merge(instpaym,'left','SK_ID_CURR')
instpaymtkag.head()

,SK_ID_CURR,"(NUM_INSTALMENT_VERSION, sum)","(NUM_INSTALMENT_VERSION, count)","(NUM_INSTALMENT_VERSION, min)","(NUM_INSTALMENT_VERSION, max)","(NUM_INSTALMENT_VERSION, mean)","(NUM_INSTALMENT_NUMBER, sum)","(NUM_INSTALMENT_NUMBER, count)","(NUM_INSTALMENT_NUMBER, min)","(NUM_INSTALMENT_NUMBER, max)","(NUM_INSTALMENT_NUMBER, mean)","(DAYS_INSTALMENT, sum)","(DAYS_INSTALMENT, count)","(DAYS_INSTALMENT, min)","(DAYS_INSTALMENT, max)","(DAYS_INSTALMENT, mean)","(DAYS_ENTRY_PAYMENT, sum)","(DAYS_ENTRY_PAYMENT, count)","(DAYS_ENTRY_PAYMENT, min)","(DAYS_ENTRY_PAYMENT, max)","(DAYS_ENTRY_PAYMENT, mean)","(AMT_INSTALMENT, sum)","(AMT_INSTALMENT, count)","(AMT_INSTALMENT, min)","(AMT_INSTALMENT, max)","(AMT_INSTALMENT, mean)","(AMT_PAYMENT, sum)","(AMT_PAYMENT, count)","(AMT_PAYMENT, min)","(AMT_PAYMENT, max)","(AMT_PAYMENT, mean)"
0,100001,8.0,7.0,1.0,2.0,1.142857,19.0,7.0,1.0,4.0,2.714286,-15314.0,7.0,-2916.0,-1619.0,-2187.714286,-15365.0,7.0,-2916.0,-1628.0,-2195.000000,41195.925,7.0,3951.00,17397.900,5885.132143,41195.925,7.0,3951.000,17397.900,5885.132143
1,100005,10.0,9.0,1.0,2.0,1.111111,45.0,9.0,1.0,9.0,5.000000,-5274.0,9.0,-706.0,-466.0,-586.000000,-5486.0,9.0,-736.0,-470.0,-609.555556,56161.845,9.0,4813.20,17656.245,6240.205000,56161.845,9.0,4813.200,17656.245,6240.205000
2,100013,43.0,155.0,0.0,2.0,0.277419,6778.0,155.0,1.0,113.0,43.729032,-209704.0,155.0,-2705.0,-14.0,-1352.929032,-210507.0,155.0,-2705.0,-14.0,-1358.109677,1689174.270,155.0,67.50,357347.745,10897.898516,1509736.545,155.0,6.165,357347.745,9740.235774
3,100028,52.0,113.0,0.0,2.0,0.460177,3447.0,113.0,1.0,74.0,30.504425,-96677.0,113.0,-1773.0,-27.0,-855.548673,-97016.0,113.0,-1785.0,-29.0,-858.548673,562658.895,113.0,1.17,38988.540,4979.282257,492310.665,113.0,1.170,38988.540,4356.731549
4,100038,12.0,12.0,1.0,1.0,1.000000,78.0,12.0,1.0,12.0,6.500000,-7464.0,12.0,-787.0,-457.0,-622.000000,-7611.0,12.0,-802.0,-466.0,-634.250000,133204.050,12.0,11097.45,11100.600,11100.337500,133204.050,12.0,11097.450,11100.600,11100.337500


#### Creating required columns dataset

In [156]:
kagtest.head()

,AMT_CREDIT,AMT_GOODS_PRICE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,OCCUPATION_TYPE,REGION_RATING_CLIENT_W_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,SK_ID_CURR
0,568800.0,450000.0,Working,Higher education,-19241,-2329,NaN,2,Kindergarten,0.752614,0.789654,0.159520,100001
1,222768.0,180000.0,Working,Secondary / secondary special,-18064,-4469,Low-skill Laborers,2,Self-employed,0.564990,0.291656,0.432962,100005
2,663264.0,630000.0,Working,Higher education,-20038,-4458,Drivers,2,Transport: type 3,NaN,0.699787,0.610991,100013
3,1575000.0,1575000.0,Working,Secondary / secondary special,-13976,-1866,Sales staff,2,Business Entity Type 3,0.525734,0.509677,0.612704,100028
4,625500.0,625500.0,Working,Secondary / secondary special,-13040,-2191,NaN,2,Business Entity Type 3,0.202145,0.425687,NaN,100038


In [157]:
impbureau2

[('DAYS_CREDIT', 'sum'),
 ('DAYS_CREDIT', 'min'),
 ('DAYS_CREDIT', 'max'),
 ('DAYS_CREDIT', 'mean'),
 ('DAYS_CREDIT_ENDDATE', 'sum'),
 ('DAYS_CREDIT_ENDDATE', 'min'),
 ('DAYS_CREDIT_ENDDATE', 'max'),
 ('DAYS_CREDIT_ENDDATE', 'mean'),
 ('DAYS_ENDDATE_FACT', 'sum'),
 ('DAYS_ENDDATE_FACT', 'min'),
 ('DAYS_ENDDATE_FACT', 'mean'),
 ('AMT_CREDIT_SUM_DEBT', 'mean'),
 ('AMT_CREDIT_SUM_LIMIT', 'max'),
 ('DAYS_CREDIT_UPDATE', 'sum'),
 ('DAYS_CREDIT_UPDATE', 'min'),
 ('DAYS_CREDIT_UPDATE', 'mean'),
 ('CREDIT_ACTIVE_Active', 'sum'),
 ('CREDIT_ACTIVE_Active', 'mean'),
 ('CREDIT_ACTIVE_Closed', 'mean'),
 'SK_ID_CURR']

In [159]:
kagbur=bureaut.loc[:,impbureau2]

In [160]:
kagprev=prevappkag.loc[:,impprevapp2]
kagpos=poscashkag.loc[:,impposcash2]
kagccbal=ccbalancekag.loc[:,impccbalance2]
kaginstpaym=instpaymtkag.loc[:,impinstpaym2]

In [161]:
kagtest=kagtest.merge(kagbur,'left','SK_ID_CURR')
kagtest=kagtest.merge(kagpos,'left','SK_ID_CURR')
kagtest=kagtest.merge(kagccbal,'left','SK_ID_CURR')
kagtest=kagtest.merge(kaginstpaym,'left','SK_ID_CURR')
kagtest=kagtest.merge(kagprev,'left','SK_ID_CURR')


In [163]:
kagtest.head()

,AMT_CREDIT,AMT_GOODS_PRICE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,OCCUPATION_TYPE,REGION_RATING_CLIENT_W_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,SK_ID_CURR,"(DAYS_CREDIT, sum)","(DAYS_CREDIT, min)","(DAYS_CREDIT, max)","(DAYS_CREDIT, mean)","(DAYS_CREDIT_ENDDATE, sum)","(DAYS_CREDIT_ENDDATE, min)","(DAYS_CREDIT_ENDDATE, max)","(DAYS_CREDIT_ENDDATE, mean)","(DAYS_ENDDATE_FACT, sum)","(DAYS_ENDDATE_FACT, min)","(DAYS_ENDDATE_FACT, mean)","(AMT_CREDIT_SUM_DEBT, mean)","(AMT_CREDIT_SUM_LIMIT, max)","(DAYS_CREDIT_UPDATE, sum)","(DAYS_CREDIT_UPDATE, min)","(DAYS_CREDIT_UPDATE, mean)","(CREDIT_ACTIVE_Active, sum)","(CREDIT_ACTIVE_Active, mean)","(CREDIT_ACTIVE_Closed, mean)","(AMT_DRAWINGS_ATM_CURRENT, count)","(AMT_DRAWINGS_CURRENT, mean)","(AMT_DRAWINGS_OTHER_CURRENT, count)","(AMT_DRAWINGS_POS_CURRENT, count)","(AMT_PAYMENT_CURRENT, count)","(CNT_DRAWINGS_ATM_CURRENT, count)","(CNT_DRAWINGS_ATM_CURRENT, mean)","(CNT_DRAWINGS_CURRENT, max)","(CNT_DRAWINGS_CURRENT, mean)","(CNT_DRAWINGS_OTHER_CURRENT, count)","(CNT_DRAWINGS_POS_CURRENT, count)","(DAYS_INSTALMENT, min)","(DAYS_ENTRY_PAYMENT, min)","(AMT_DOWN_PAYMENT, sum)","(DAYS_LAST_DUE_1ST_VERSION, sum)","(NAME_CONTRACT_STATUS_Approved, mean)","(NAME_CONTRACT_STATUS_Refused, mean)","(CODE_REJECT_REASON_XAP, mean)"
0,568800.0,450000.0,Working,Higher education,-19241,-2329,NaN,2,Kindergarten,0.752614,0.789654,0.159520,100001,-5145.0,-1572.0,-49.0,-735.000000,577.0,-1329.0,1778.0,82.428571,-3302.0,-1328.0,-825.500000,85240.928571,0.00,-652.0,-155.0,-93.142857,3.0,0.428571,0.571429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2916.0,-2916.0,2520.0,-1499.0,1.00,0.0,1.0
1,222768.0,180000.0,Working,Secondary / secondary special,-18064,-4469,Low-skill Laborers,2,Self-employed,0.564990,0.291656,0.432962,100005,-572.0,-373.0,-62.0,-190.666667,1318.0,-128.0,1324.0,439.333333,-123.0,-123.0,-123.000000,189469.500000,0.00,-163.0,-121.0,-54.333333,2.0,0.666667,0.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-706.0,-736.0,4464.0,-376.0,0.50,0.0,1.0
2,663264.0,630000.0,Working,Higher education,-20038,-4458,Drivers,2,Transport: type 3,NaN,0.699787,0.610991,100013,-6950.0,-2070.0,-1210.0,-1737.500000,-4272.0,-1707.0,-567.0,-1068.000000,-4219.0,-1334.0,-1054.750000,0.000000,NaN,-3102.0,-1334.0,-775.500000,0.0,0.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2705.0,-2705.0,6750.0,-1547.0,0.75,0.0,1.0
3,1575000.0,1575000.0,Working,Secondary / secondary special,-13976,-1866,Sales staff,2,Business Entity Type 3,0.525734,0.509677,0.612704,100028,-16821.0,-2105.0,-269.0,-1401.750000,23877.0,-1862.0,30885.0,2387.700000,-8668.0,-1862.0,-1238.285714,18630.450000,101390.76,-7818.0,-1564.0,-651.500000,5.0,0.416667,0.583333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1773.0,-1785.0,11250.0,363664.0,0.60,0.0,0.8
4,625500.0,625500.0,Working,Secondary / secondary special,-13040,-2191,NaN,2,Business Entity Type 3,0.202145,0.425687,NaN,100038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-787.0,-802.0,8095.5,-457.0,0.50,0.0,1.0


In [165]:
woekag2=pd.DataFrame()
woekag2['SK_ID_CURR']=kagtest['SK_ID_CURR']
woekag2

,SK_ID_CURR
0,100001
1,100005
2,100013
3,100028
4,100038
...,...
48739,456221
48740,456222
48741,456223
48742,456224


In [184]:
f=1
for i in trainless.drop(['TARGET','SK_ID_CURR'],axis=1).columns:
    print(str(f)+'/'+str(len(trainless.drop(['TARGET','SK_ID_CURR'],axis=1).columns)))
    f=f+1
    ct=woe(trainless,'TARGET',i)
    if 'bin' in ct.columns:
        val=[]
        if kagtest[i].isna().sum()>1:
            for j in kagtest[i]:
                flag=0
                if np.isnan(j):
                    val.append(ct['WOE'][len(ct)-1])
                    flag=1
                else:
                    for m in range(0,len(ct)-1,1):
                        if j in ct['bin'][m]:
                            val.append(ct['WOE'][m])
                            flag=1
                            continue
                if flag==0:
                    val.append(ct['WOE'][0])
                    
        else:
            for j in kagtest[i]:
                flag=0
                if trainless[i].isna().sum()==0:
                    for m in range(0,len(ct),1):
                        if j in ct['bin'][m]:
                            val.append(ct['WOE'][m])
                            flag=1
                            continue
                else:
                    for m in range(0,len(ct)-1,1):
                        if j in ct['bin'][m]:
                            val.append(ct['WOE'][m])
                            flag=1
                            continue
                if flag==0:
                    val.append(ct['WOE'][0])
    else:
        val=[]
        if kagtest[i].isna().sum()>1:
            for j in kagtest[i]:
                flag=0
                if str(j)=='nan':
                    val.append(ct['WOE'][len(ct)-1])
                    flag=1
                else:
                    for m in range(0,len(ct)-1,1):
                        if j==ct[i][m]:
                            val.append(ct['WOE'][m])
                            flag=1
                            continue
                if flag==0:
                    val.append(ct['WOE'][0])
        else:
            for j in kagtest[i]:
                flag=0
                for m in range(0,len(ct),1):
                    if j==ct[i][m]:
                        val.append(ct['WOE'][m])
                        flag=1
                        continue
                if flag==0:
                    val.append(ct['WOE'][0])
    val=np.asarray(val,'float64')
    woekag2[str('WOE_'+str(np.array(i).flatten()))]=val                
woekag2

1/49
2/49
3/49


D:\Software\Anaconda\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


4/49
5/49
6/49
7/49
8/49
9/49
10/49
11/49
12/49
13/49
14/49
15/49
16/49
17/49
18/49
19/49
20/49
21/49
22/49
23/49
24/49
25/49
26/49
27/49
28/49
29/49
30/49
31/49
32/49
33/49
34/49
35/49
36/49
37/49
38/49
39/49
40/49
41/49
42/49
43/49
44/49
45/49
46/49
47/49
48/49
49/49


,SK_ID_CURR,WOE_['AMT_CREDIT'],WOE_['AMT_GOODS_PRICE'],WOE_['NAME_INCOME_TYPE'],WOE_['NAME_EDUCATION_TYPE'],WOE_['DAYS_BIRTH'],WOE_['DAYS_EMPLOYED'],WOE_['OCCUPATION_TYPE'],WOE_['REGION_RATING_CLIENT_W_CITY'],WOE_['ORGANIZATION_TYPE'],WOE_['EXT_SOURCE_1'],WOE_['EXT_SOURCE_2'],WOE_['EXT_SOURCE_3'],WOE_['DAYS_CREDIT' 'sum'],WOE_['DAYS_CREDIT' 'min'],WOE_['DAYS_CREDIT' 'max'],WOE_['DAYS_CREDIT' 'mean'],WOE_['DAYS_CREDIT_ENDDATE' 'sum'],WOE_['DAYS_CREDIT_ENDDATE' 'min'],WOE_['DAYS_CREDIT_ENDDATE' 'max'],WOE_['DAYS_CREDIT_ENDDATE' 'mean'],WOE_['DAYS_ENDDATE_FACT' 'sum'],WOE_['DAYS_ENDDATE_FACT' 'min'],WOE_['DAYS_ENDDATE_FACT' 'mean'],WOE_['AMT_CREDIT_SUM_DEBT' 'mean'],WOE_['AMT_CREDIT_SUM_LIMIT' 'max'],WOE_['DAYS_CREDIT_UPDATE' 'sum'],WOE_['DAYS_CREDIT_UPDATE' 'min'],WOE_['DAYS_CREDIT_UPDATE' 'mean'],WOE_['CREDIT_ACTIVE_Active' 'sum'],WOE_['CREDIT_ACTIVE_Active' 'mean'],WOE_['CREDIT_ACTIVE_Closed' 'mean'],WOE_['AMT_DRAWINGS_ATM_CURRENT' 'count'],WOE_['AMT_DRAWINGS_CURRENT' 'mean'],WOE_['AMT_DRAWINGS_OTHER_CURRENT' 'count'],WOE_['AMT_DRAWINGS_POS_CURRENT' 'count'],WOE_['AMT_PAYMENT_CURRENT' 'count'],WOE_['CNT_DRAWINGS_ATM_CURRENT' 'count'],WOE_['CNT_DRAWINGS_ATM_CURRENT' 'mean'],WOE_['CNT_DRAWINGS_CURRENT' 'max'],WOE_['CNT_DRAWINGS_CURRENT' 'mean'],WOE_['CNT_DRAWINGS_OTHER_CURRENT' 'count'],WOE_['CNT_DRAWINGS_POS_CURRENT' 'count'],WOE_['AMT_DOWN_PAYMENT' 'sum'],WOE_['DAYS_LAST_DUE_1ST_VERSION' 'sum'],WOE_['NAME_CONTRACT_STATUS_Approved' 'mean'],WOE_['NAME_CONTRACT_STATUS_Refused' 'mean'],WOE_['CODE_REJECT_REASON_XAP' 'mean'],WOE_['DAYS_INSTALMENT' 'min'],WOE_['DAYS_ENTRY_PAYMENT' 'min']
0,100001,-0.188568,-0.554948,-0.187342,0.436057,0.205804,0.014862,0.236744,0.021248,0.156295,0.930129,1.114627,-0.821951,0.092652,0.100404,-0.375131,-0.106635,-0.094376,0.135004,-0.177724,-0.098265,0.212231,0.170246,0.116614,-0.082695,0.133783,-0.061446,-0.209660,-0.316194,-0.022053,-0.018065,-0.158755,0.034488,0.034488,0.034488,0.034488,0.034488,0.034488,0.069919,0.034488,0.034488,0.034488,0.034488,-0.307704,0.049324,0.136890,0.139476,0.146435,0.405727,0.365542
1,100005,0.043054,-0.024098,-0.187342,-0.110769,0.129366,0.384673,-0.857654,0.021248,-0.261872,0.325046,-0.398062,-0.022072,-0.208073,-0.379048,-0.375131,-0.653885,-0.233657,-0.248020,-0.007285,-0.199444,-0.327070,-0.301615,-0.341884,-0.201340,0.133783,-0.201845,-0.209660,-0.333061,0.085685,-0.234539,-0.258740,0.034488,0.034488,0.034488,0.034488,0.034488,0.034488,0.069919,0.034488,0.034488,0.034488,0.034488,-0.184946,-0.150284,-0.164042,0.139476,0.146435,-0.288992,-0.163835
2,100013,-0.138857,0.068776,-0.187342,0.436057,0.315534,0.384673,-0.367161,0.021248,-0.759910,-0.054113,0.618028,0.523564,0.204977,0.234357,0.309581,0.539836,0.441791,0.242944,0.275151,0.481539,0.230644,0.170246,0.276876,0.399831,-0.172810,0.186145,0.232031,0.400025,0.226656,0.401778,0.423709,0.034488,0.034488,0.034488,0.034488,0.034488,0.034488,0.069919,0.034488,0.034488,0.034488,0.034488,-0.079710,0.087712,0.197280,0.139476,0.146435,0.196530,0.202106
3,100028,0.633336,0.704362,-0.187342,-0.110769,-0.054081,-0.083280,-0.197619,0.021248,-0.158223,0.247787,0.012740,0.523564,0.294302,0.234357,0.118735,0.439678,-0.245814,0.275973,-0.411374,-0.042475,0.295948,0.251728,0.317719,0.129662,0.058509,0.302102,0.218873,0.307747,-0.304004,0.064820,0.081011,0.034488,0.034488,0.034488,0.034488,0.034488,0.034488,0.069919,0.034488,0.034488,0.034488,0.034488,0.046093,-0.071750,-0.035878,0.139476,-0.016294,0.074530,0.074836
4,100038,-0.138857,0.068776,-0.187342,-0.110769,-0.205246,0.014862,0.236744,0.021248,-0.158223,-0.670219,-0.149785,-0.157430,-0.244364,-0.244364,-0.244364,-0.244364,-0.244364,-0.247774,-0.247774,-0.247774,-0.244364,-0.284409,-0.284409,-0.190260,-0.172810,-0.244364,-0.244364,-0.244364,-0.244364,-0.244364,-0.244364,0.034488,0.034488,0.034488,0.034488,0.034488,0.034488,0.069919,0.034488,0.034488,0.034488,0.034488,-0.011542,-0.150284,-0.164042,0.139476,0.146435,-0.164961,-0.163835
...,...,...,...,...,...,...,...,...,...,...,

In [189]:
kagpred=logit.predict_proba(woekag2.drop('SK_ID_CURR',axis=1))[:,1]

In [190]:
kagpred

array([0.34037785, 0.64747319, 0.28130083, ..., 0.38685587, 0.52521373,
       0.61615891])

In [191]:
submission = woekag2[['SK_ID_CURR']]
submission['TARGET'] = kagpred

D:\Software\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [193]:
submission.to_csv(r'D:\DS\BSFA\home-credit-default-risk\submission.csv')

### Metrics

In [197]:
ytestpred=logit.predict_proba(xtest)[:,1]

In [198]:
tab=pd.DataFrame()
tab['Actual']=ytest
tab['Probability']=ytestpred

In [199]:
tab

,Actual,Probability
0,0,0.788661
1,0,0.327854
2,0,0.609398
3,0,0.198628
4,0,0.385977
...,...,...
30747,0,0.750579
30748,0,0.489044
30749,0,0.109720
30750,0,0.347680


In [202]:
train1=tab.loc[tab['Actual']==1,'Probability']
train0=tab.loc[tab['Actual']==0,'Probability']

In [208]:
conc=0
disc=0
t=0
for i in train1:
    for j in train0:
        if i>j:
            conc+=1
        elif j>i:
            disc+=1
        else:
            t+=1
print(conc,disc)

53368844 17750083


In [209]:
print('Total pairs:',len(train1)*len(train0))
print('Concordant pairs:',conc)
print('Discordant pairs:',disc)
print('Tied pairs:',t)

Total pairs: 71118927
Concordant pairs: 53368844
Discordant pairs: 17750083
Tied pairs: 0


In [210]:
print('Somers D:',(conc-disc)/(len(train1)*len(train0)))

Somers D: 0.5008337794522687


### Dropping multicollinear columns using VIF

In [214]:
from statsmodels.stats.outliers_influence import variance_inflation_factor    

def calculate_vif_(X, thresh=5.0):
    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        dropped = False
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]

        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +
                  '\' at index: ' + str(maxloc))
            del variables[maxloc]
            dropped = True

    print('Remaining variables:')
    print(X.columns[variables])
    return X.iloc[:, variables]

In [216]:
calculate_vif_(xtrain)

D:\Software\Anaconda\lib\site-packages\statsmodels\stats\outliers_influence.py:185: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


dropping 'WOE_['AMT_DRAWINGS_ATM_CURRENT' 'count']' at index: 31
dropping 'WOE_['AMT_DRAWINGS_OTHER_CURRENT' 'count']' at index: 32
dropping 'WOE_['AMT_DRAWINGS_POS_CURRENT' 'count']' at index: 32
dropping 'WOE_['CNT_DRAWINGS_ATM_CURRENT' 'count']' at index: 33
dropping 'WOE_['CNT_DRAWINGS_OTHER_CURRENT' 'count']' at index: 36
dropping 'WOE_['DAYS_ENTRY_PAYMENT' 'min']' at index: 43
dropping 'WOE_['DAYS_ENDDATE_FACT' 'sum']' at index: 20
dropping 'WOE_['CREDIT_ACTIVE_Active' 'mean']' at index: 28
dropping 'WOE_['CNT_DRAWINGS_POS_CURRENT' 'count']' at index: 34
dropping 'WOE_['DAYS_ENDDATE_FACT' 'min']' at index: 20
dropping 'WOE_['DAYS_CREDIT_UPDATE' 'sum']' at index: 23
dropping 'WOE_['DAYS_CREDIT_UPDATE' 'min']' at index: 23
dropping 'WOE_['DAYS_CREDIT' 'min']' at index: 13
dropping 'WOE_['CNT_DRAWINGS_CURRENT' 'mean']' at index: 29
dropping 'WOE_['DAYS_CREDIT_ENDDATE' 'min']' at index: 16
dropping 'WOE_['NAME_CONTRACT_STATUS_Refused' 'mean']' at index: 31
dropping 'WOE_['DAYS_CREDIT

,WOE_['AMT_CREDIT'],WOE_['AMT_GOODS_PRICE'],WOE_['NAME_INCOME_TYPE'],WOE_['NAME_EDUCATION_TYPE'],WOE_['DAYS_BIRTH'],WOE_['DAYS_EMPLOYED'],WOE_['OCCUPATION_TYPE'],WOE_['REGION_RATING_CLIENT_W_CITY'],WOE_['ORGANIZATION_TYPE'],WOE_['EXT_SOURCE_1'],WOE_['EXT_SOURCE_2'],WOE_['EXT_SOURCE_3'],WOE_['DAYS_CREDIT' 'sum'],WOE_['DAYS_CREDIT' 'max'],WOE_['DAYS_CREDIT_ENDDATE' 'sum'],WOE_['DAYS_CREDIT_ENDDATE' 'max'],WOE_['DAYS_ENDDATE_FACT' 'mean'],WOE_['AMT_CREDIT_SUM_DEBT' 'mean'],WOE_['AMT_CREDIT_SUM_LIMIT' 'max'],WOE_['CREDIT_ACTIVE_Active' 'sum'],WOE_['CREDIT_ACTIVE_Closed' 'mean'],WOE_['AMT_DRAWINGS_CURRENT' 'mean'],WOE_['AMT_PAYMENT_CURRENT' 'count'],WOE_['CNT_DRAWINGS_ATM_CURRENT' 'mean'],WOE_['CNT_DRAWINGS_CURRENT' 'max'],WOE_['AMT_DOWN_PAYMENT' 'sum'],WOE_['DAYS_LAST_DUE_1ST_VERSION' 'sum'],WOE_['NAME_CONTRACT_STATUS_Approved' 'mean'],WOE_['CODE_REJECT_REASON_XAP' 'mean'],WOE_['DAYS_INSTALMENT' 'min']
0,0.249766,0.163212,-0.187342,0.436057,-0.304800,-0.391887,0.251763,0.021248,0.156295,-0.466422,0.599615,0.523564,0.248168,0.118735,0.441435,0.020782,0.344199,0.246110,0.133783,0.085685,0.395371,0.034488,0.034488,0.069919,0.034488,0.468999,0.087712,0.136890,0.146435,0.032716
1,0.296124,0.380928,0.078430,-0.110769,0.002260,-0.083280,0.266238,0.021248,-0.161475,-0.054113,0.761515,0.202892,0.092652,-0.123846,-0.034063,-0.177724,0.171377,-0.134059,0.133783,-0.304004,-0.258740,0.034488,0.034488,0.069919,0.034488,0.166176,-0.042089,-0.164042,0.146435,-0.164961
2,-0.358265,-0.554948,-0.187342,-0.110769,0.042607,-0.368778,-0.197619,0.021248,0.085583,0.473007,0.012740,0.652092,-0.003175,-0.081418,-0.233657,0.020782,-0.172625,-0.244246,0.133783,0.226656,0.395371,0.034488,0.034488,0.069919,0.034488,-0.079710,-0.042089,0.053093,-0.254040,-0.002635
3,0.047504,-0.124632,0.078430,0.436057,-0.367292,0.120151,0.560757,0.021248,-0.158223,0.808814,0.486644,-1.250475,0.204977,0.106260,-0.447632,-0.132811,0.106109,-0.201340,0.133783,-0.574296,-0.380719,0.034488,0.034488,0.069919,0.034488,0.312118,0.124375,0.136890,0.146435,0.106336
4,0.144611,-0.124632,-0.187342,0.436057,-0.304800,0.336621,-0.295216,0.021248,-0.161475,0.073998,0.110608,-0.215943,-0.509828,-0.078997,-0.318931,-0.269810,-0.284409,-0.201340,0.133783,0.085685,-0.380719,0.034488,0.034488,0.069919,0.034488,-0.176258,0.025519,0.197280,-0.110980,-0.162799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276754,-0.138857,-0.124632,-0.187342,-0.110769,0.002260,-0.253243,-0.302363,-0.380451,-0.261872,-0.054113,0.319177,-0.157430,-0.244364,-0.244364,-0.244364,-0.247774,-0.284409,-0.190260,-0.172810,-0.244364,-0.244364,0.034488,0.034488,0.069919,0.034488,-0.176258,-0.104164,-0.164042,0.146435,-0.281480
276755,-0.102815,0.084100,0.078430,-0.110769,0.042607,-0.368778,-0.197619,-0.380451,-0.172309,0.180759,0.960892,-0.215943,0.204977,0.322985,0.441791,0.185365,0.106109,-0.201340,0.133783,0.085685,0.361354,0.034488,0.034488,0.069919,0.034488,-0.176258,-0.196066,-0.323232,-0.016294,-0.162799
276756,0.249766,0.380928,-0.187342,0.436057,-0.054081,0.447980,0.266238,0.021248,-0.261872,-0.187715,0.319177,0.387844,0.294302,0.118735,-0.245814,-0.132811,0.353909,0.120641,0.133783,-0.150448,0.275789,0.413021,0.387199,0.069919,0.407189,0.438441,0.109442,0.197280,0.146435,0.106336
276757,0.121238,0.039867,-0.187342,-0.110769,-0.205246,-0.083280,-0.367161,0.539581,-0.066373,0.473007,0.199532,-0.572538,0.016543,0.106260,0.130961,0.407180,-0.284409,0.120641,0.133783,-0.150448,-0.380719,-0.523765,-0.689717,0.131858,-0.360229,-0.176258,-0.364481,0.136890,0.146435,-0.288992


In [217]:
xtrain.head()

,WOE_['AMT_CREDIT'],WOE_['AMT_GOODS_PRICE'],WOE_['NAME_INCOME_TYPE'],WOE_['NAME_EDUCATION_TYPE'],WOE_['DAYS_BIRTH'],WOE_['DAYS_EMPLOYED'],WOE_['OCCUPATION_TYPE'],WOE_['REGION_RATING_CLIENT_W_CITY'],WOE_['ORGANIZATION_TYPE'],WOE_['EXT_SOURCE_1'],WOE_['EXT_SOURCE_2'],WOE_['EXT_SOURCE_3'],WOE_['DAYS_CREDIT' 'sum'],WOE_['DAYS_CREDIT' 'min'],WOE_['DAYS_CREDIT' 'max'],WOE_['DAYS_CREDIT' 'mean'],WOE_['DAYS_CREDIT_ENDDATE' 'sum'],WOE_['DAYS_CREDIT_ENDDATE' 'min'],WOE_['DAYS_CREDIT_ENDDATE' 'max'],WOE_['DAYS_CREDIT_ENDDATE' 'mean'],WOE_['DAYS_ENDDATE_FACT' 'sum'],WOE_['DAYS_ENDDATE_FACT' 'min'],WOE_['DAYS_ENDDATE_FACT' 'mean'],WOE_['AMT_CREDIT_SUM_DEBT' 'mean'],WOE_['AMT_CREDIT_SUM_LIMIT' 'max'],WOE_['DAYS_CREDIT_UPDATE' 'sum'],WOE_['DAYS_CREDIT_UPDATE' 'min'],WOE_['DAYS_CREDIT_UPDATE' 'mean'],WOE_['CREDIT_ACTIVE_Active' 'sum'],WOE_['CREDIT_ACTIVE_Active' 'mean'],WOE_['CREDIT_ACTIVE_Closed' 'mean'],WOE_['AMT_DRAWINGS_ATM_CURRENT' 'count'],WOE_['AMT_DRAWINGS_CURRENT' 'mean'],WOE_['AMT_DRAWINGS_OTHER_CURRENT' 'count'],WOE_['AMT_DRAWINGS_POS_CURRENT' 'count'],WOE_['AMT_PAYMENT_CURRENT' 'count'],WOE_['CNT_DRAWINGS_ATM_CURRENT' 'count'],WOE_['CNT_DRAWINGS_ATM_CURRENT' 'mean'],WOE_['CNT_DRAWINGS_CURRENT' 'max'],WOE_['CNT_DRAWINGS_CURRENT' 'mean'],WOE_['CNT_DRAWINGS_OTHER_CURRENT' 'count'],WOE_['CNT_DRAWINGS_POS_CURRENT' 'count'],WOE_['AMT_DOWN_PAYMENT' 'sum'],WOE_['DAYS_LAST_DUE_1ST_VERSION' 'sum'],WOE_['NAME_CONTRACT_STATUS_Approved' 'mean'],WOE_['NAME_CONTRACT_STATUS_Refused' 'mean'],WOE_['CODE_REJECT_REASON_XAP' 'mean'],WOE_['DAYS_INSTALMENT' 'min'],WOE_['DAYS_ENTRY_PAYMENT' 'min']
0,0.249766,0.163212,-0.187342,0.436057,-0.304800,-0.391887,0.251763,0.021248,0.156295,-0.466422,0.599615,0.523564,0.248168,0.333144,0.118735,0.364417,0.441435,0.332041,0.020782,0.511075,0.298887,0.393971,0.344199,0.246110,0.133783,0.281010,0.365687,0.549580,0.085685,0.360033,0.395371,0.034488,0.034488,0.034488,0.034488,0.034488,0.034488,0.069919,0.034488,0.034488,0.034488,0.034488,0.468999,0.087712,0.136890,0.139476,0.146435,0.032716,-0.003404
1,0.296124,0.380928,0.078430,-0.110769,0.002260,-0.083280,0.266238,0.021248,-0.161475,-0.054113,0.761515,0.202892,0.092652,0.034755,-0.123846,-0.106635,-0.034063,0.107708,-0.177724,-0.239588,0.182058,0.170246,0.171377,-0.134059,0.133783,0.186259,0.232031,0.023426,-0.304004,-0.411651,-0.258740,0.034488,0.034488,0.034488,0.034488,0.034488,0.034488,0.069919,0.034488,0.034488,0.034488,0.034488,0.166176,-0.042089,-0.164042,0.139476,0.146435,-0.164961,-0.163835
2,-0.358265,-0.554948,-0.187342,-0.110769,0.042607,-0.368778,-0.197619,0.021248,0.085583,0.473007,0.012740,0.652092,-0.003175,-0.167312,-0.081418,-0.269498,-0.233657,0.001154,0.020782,-0.271042,0.014292,-0.035837,-0.172625,-0.244246,0.133783,0.031720,0.026772,-0.095437,0.226656,0.477236,0.395371,0.034488,0.034488,0.034488,0.034488,0.034488,0.034488,0.069919,0.034488,0.034488,0.034488,0.034488,-0.079710,-0.042089,0.053093,-0.262131,-0.254040,-0.002635,-0.003404
3,0.047504,-0.124632,0.078430,0.436057,-0.367292,0.120151,0.560757,0.021248,-0.158223,0.808814,0.486644,-1.250475,0.204977,-0.079371,0.106260,-0.051801,-0.447632,-0.010142,-0.132811,-0.368336,0.005931,0.063001,0.106109,-0.201340,0.133783,0.113255,0.077346,-0.237972,-0.574296,-0.379437,-0.380719,0.034488,0.034488,0.034488,0.034488,0.034488,0.034488,0.069919,0.034488,0.034488,0.034488,0.034488,0.312118,0.124375,0.136890,0.139476,0.146435,0.106336,0.109711
4,0.144611,-0.124632,-0.187342,0.436057,-0.304800,0.336621,-0.295216,0.021248,-0.161475,0.073998,0.110608,-0.215943,-0.509828,-0.564015,-0.078997,-0.653885,-0.318931,-0.288809,-0.269810,-0.129616,-0.327070,-0.284409,-0.284409,-0.201340,0.133783,-0.348276,-0.391633,-0.432981,0.085685,-0.379437,-0.380719,0.034488,0.034488,0.034488,0.034488,0.034488,0.034488,0.069919,0.034488,0.034488,0.034488,0.034488,-0.176258,0.025519,0.197280,-0.137508,-0.110980,-0.162799,-0.170266


In [222]:
xtrain1=xtrain.loc[:,['WOE_[\'AMT_CREDIT\']', 'WOE_[\'AMT_GOODS_PRICE\']',
       'WOE_[\'NAME_INCOME_TYPE\']', 'WOE_[\'NAME_EDUCATION_TYPE\']',
       'WOE_[\'DAYS_BIRTH\']', 'WOE_[\'DAYS_EMPLOYED\']',
       'WOE_[\'OCCUPATION_TYPE\']', 'WOE_[\'REGION_RATING_CLIENT_W_CITY\']',
       'WOE_[\'ORGANIZATION_TYPE\']', 'WOE_[\'EXT_SOURCE_1\']',
       'WOE_[\'EXT_SOURCE_2\']', 'WOE_[\'EXT_SOURCE_3\']',
       'WOE_[\'DAYS_CREDIT\' \'sum\']', 'WOE_[\'DAYS_CREDIT\' \'max\']',
       'WOE_[\'DAYS_CREDIT_ENDDATE\' \'sum\']',
       'WOE_[\'DAYS_CREDIT_ENDDATE\' \'max\']', 'WOE_[\'DAYS_ENDDATE_FACT\' \'mean\']',
       'WOE_[\'AMT_CREDIT_SUM_DEBT\' \'mean\']',
       'WOE_[\'AMT_CREDIT_SUM_LIMIT\' \'max\']',
       'WOE_[\'CREDIT_ACTIVE_Active\' \'sum\']',
       'WOE_[\'CREDIT_ACTIVE_Closed\' \'mean\']',
       'WOE_[\'AMT_DRAWINGS_CURRENT\' \'mean\']',
       'WOE_[\'AMT_PAYMENT_CURRENT\' \'count\']',
       'WOE_[\'CNT_DRAWINGS_ATM_CURRENT\' \'mean\']',
       'WOE_[\'CNT_DRAWINGS_CURRENT\' \'max\']', 'WOE_[\'AMT_DOWN_PAYMENT\' \'sum\']',
       'WOE_[\'DAYS_LAST_DUE_1ST_VERSION\' \'sum\']',
       'WOE_[\'NAME_CONTRACT_STATUS_Approved\' \'mean\']',
       'WOE_[\'CODE_REJECT_REASON_XAP\' \'mean\']',
       'WOE_[\'DAYS_INSTALMENT\' \'min\']']]

In [223]:
xtrain1.head()

,WOE_['AMT_CREDIT'],WOE_['AMT_GOODS_PRICE'],WOE_['NAME_INCOME_TYPE'],WOE_['NAME_EDUCATION_TYPE'],WOE_['DAYS_BIRTH'],WOE_['DAYS_EMPLOYED'],WOE_['OCCUPATION_TYPE'],WOE_['REGION_RATING_CLIENT_W_CITY'],WOE_['ORGANIZATION_TYPE'],WOE_['EXT_SOURCE_1'],WOE_['EXT_SOURCE_2'],WOE_['EXT_SOURCE_3'],WOE_['DAYS_CREDIT' 'sum'],WOE_['DAYS_CREDIT' 'max'],WOE_['DAYS_CREDIT_ENDDATE' 'sum'],WOE_['DAYS_CREDIT_ENDDATE' 'max'],WOE_['DAYS_ENDDATE_FACT' 'mean'],WOE_['AMT_CREDIT_SUM_DEBT' 'mean'],WOE_['AMT_CREDIT_SUM_LIMIT' 'max'],WOE_['CREDIT_ACTIVE_Active' 'sum'],WOE_['CREDIT_ACTIVE_Closed' 'mean'],WOE_['AMT_DRAWINGS_CURRENT' 'mean'],WOE_['AMT_PAYMENT_CURRENT' 'count'],WOE_['CNT_DRAWINGS_ATM_CURRENT' 'mean'],WOE_['CNT_DRAWINGS_CURRENT' 'max'],WOE_['AMT_DOWN_PAYMENT' 'sum'],WOE_['DAYS_LAST_DUE_1ST_VERSION' 'sum'],WOE_['NAME_CONTRACT_STATUS_Approved' 'mean'],WOE_['CODE_REJECT_REASON_XAP' 'mean'],WOE_['DAYS_INSTALMENT' 'min']
0,0.249766,0.163212,-0.187342,0.436057,-0.304800,-0.391887,0.251763,0.021248,0.156295,-0.466422,0.599615,0.523564,0.248168,0.118735,0.441435,0.020782,0.344199,0.246110,0.133783,0.085685,0.395371,0.034488,0.034488,0.069919,0.034488,0.468999,0.087712,0.136890,0.146435,0.032716
1,0.296124,0.380928,0.078430,-0.110769,0.002260,-0.083280,0.266238,0.021248,-0.161475,-0.054113,0.761515,0.202892,0.092652,-0.123846,-0.034063,-0.177724,0.171377,-0.134059,0.133783,-0.304004,-0.258740,0.034488,0.034488,0.069919,0.034488,0.166176,-0.042089,-0.164042,0.146435,-0.164961
2,-0.358265,-0.554948,-0.187342,-0.110769,0.042607,-0.368778,-0.197619,0.021248,0.085583,0.473007,0.012740,0.652092,-0.003175,-0.081418,-0.233657,0.020782,-0.172625,-0.244246,0.133783,0.226656,0.395371,0.034488,0.034488,0.069919,0.034488,-0.079710,-0.042089,0.053093,-0.254040,-0.002635
3,0.047504,-0.124632,0.078430,0.436057,-0.367292,0.120151,0.560757,0.021248,-0.158223,0.808814,0.486644,-1.250475,0.204977,0.106260,-0.447632,-0.132811,0.106109,-0.201340,0.133783,-0.574296,-0.380719,0.034488,0.034488,0.069919,0.034488,0.312118,0.124375,0.136890,0.146435,0.106336
4,0.144611,-0.124632,-0.187342,0.436057,-0.304800,0.336621,-0.295216,0.021248,-0.161475,0.073998,0.110608,-0.215943,-0.509828,-0.078997,-0.318931,-0.269810,-0.284409,-0.201340,0.133783,0.085685,-0.380719,0.034488,0.034488,0.069919,0.034488,-0.176258,0.025519,0.197280,-0.110980,-0.162799


In [225]:
xtexst1=xtest.loc[:,['WOE_[\'AMT_CREDIT\']', 'WOE_[\'AMT_GOODS_PRICE\']',
       'WOE_[\'NAME_INCOME_TYPE\']', 'WOE_[\'NAME_EDUCATION_TYPE\']',
       'WOE_[\'DAYS_BIRTH\']', 'WOE_[\'DAYS_EMPLOYED\']',
       'WOE_[\'OCCUPATION_TYPE\']', 'WOE_[\'REGION_RATING_CLIENT_W_CITY\']',
       'WOE_[\'ORGANIZATION_TYPE\']', 'WOE_[\'EXT_SOURCE_1\']',
       'WOE_[\'EXT_SOURCE_2\']', 'WOE_[\'EXT_SOURCE_3\']',
       'WOE_[\'DAYS_CREDIT\' \'sum\']', 'WOE_[\'DAYS_CREDIT\' \'max\']',
       'WOE_[\'DAYS_CREDIT_ENDDATE\' \'sum\']',
       'WOE_[\'DAYS_CREDIT_ENDDATE\' \'max\']', 'WOE_[\'DAYS_ENDDATE_FACT\' \'mean\']',
       'WOE_[\'AMT_CREDIT_SUM_DEBT\' \'mean\']',
       'WOE_[\'AMT_CREDIT_SUM_LIMIT\' \'max\']',
       'WOE_[\'CREDIT_ACTIVE_Active\' \'sum\']',
       'WOE_[\'CREDIT_ACTIVE_Closed\' \'mean\']',
       'WOE_[\'AMT_DRAWINGS_CURRENT\' \'mean\']',
       'WOE_[\'AMT_PAYMENT_CURRENT\' \'count\']',
       'WOE_[\'CNT_DRAWINGS_ATM_CURRENT\' \'mean\']',
       'WOE_[\'CNT_DRAWINGS_CURRENT\' \'max\']', 'WOE_[\'AMT_DOWN_PAYMENT\' \'sum\']',
       'WOE_[\'DAYS_LAST_DUE_1ST_VERSION\' \'sum\']',
       'WOE_[\'NAME_CONTRACT_STATUS_Approved\' \'mean\']',
       'WOE_[\'CODE_REJECT_REASON_XAP\' \'mean\']',
       'WOE_[\'DAYS_INSTALMENT\' \'min\']']]

In [226]:
logit1=LogisticRegression(class_weight='balanced',penalty='none',n_jobs=-1)

In [227]:
logit1.fit(xtrain1,ytrain)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=-1, penalty='none',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [230]:
xtrain1.shape

(276759, 30)

In [231]:
pred=logit1.predict(xtest1)

In [232]:
print("Accuracy:",metrics.accuracy_score(ytest,pred))
print('Confusion matrix:\n',metrics.confusion_matrix(ytest,pred))
print('Precision score:',metrics.precision_score(ytest,pred))
print('Recall score:',metrics.recall_score(ytest,pred))
print('ROC:',metrics.roc_auc_score(ytest,pred))

Accuracy: 0.6869471904266389
Confusion matrix:
 [[19401  8832]
 [  795  1724]]
Precision score: 0.16331943918150815
Recall score: 0.684398570861453
ROC: 0.6857865768981583


In [233]:
kagtest

,AMT_CREDIT,AMT_GOODS_PRICE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,OCCUPATION_TYPE,REGION_RATING_CLIENT_W_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,SK_ID_CURR,"(DAYS_CREDIT, sum)","(DAYS_CREDIT, min)","(DAYS_CREDIT, max)","(DAYS_CREDIT, mean)","(DAYS_CREDIT_ENDDATE, sum)","(DAYS_CREDIT_ENDDATE, min)","(DAYS_CREDIT_ENDDATE, max)","(DAYS_CREDIT_ENDDATE, mean)","(DAYS_ENDDATE_FACT, sum)","(DAYS_ENDDATE_FACT, min)","(DAYS_ENDDATE_FACT, mean)","(AMT_CREDIT_SUM_DEBT, mean)","(AMT_CREDIT_SUM_LIMIT, max)","(DAYS_CREDIT_UPDATE, sum)","(DAYS_CREDIT_UPDATE, min)","(DAYS_CREDIT_UPDATE, mean)","(CREDIT_ACTIVE_Active, sum)","(CREDIT_ACTIVE_Active, mean)","(CREDIT_ACTIVE_Closed, mean)","(AMT_DRAWINGS_ATM_CURRENT, count)","(AMT_DRAWINGS_CURRENT, mean)","(AMT_DRAWINGS_OTHER_CURRENT, count)","(AMT_DRAWINGS_POS_CURRENT, count)","(AMT_PAYMENT_CURRENT, count)","(CNT_DRAWINGS_ATM_CURRENT, count)","(CNT_DRAWINGS_ATM_CURRENT, mean)","(CNT_DRAWINGS_CURRENT, max)","(CNT_DRAWINGS_CURRENT, mean)","(CNT_DRAWINGS_OTHER_CURRENT, count)","(CNT_DRAWINGS_POS_CURRENT, count)","(DAYS_INSTALMENT, min)","(DAYS_ENTRY_PAYMENT, min)","(AMT_DOWN_PAYMENT, sum)","(DAYS_LAST_DUE_1ST_VERSION, sum)","(NAME_CONTRACT_STATUS_Approved, mean)","(NAME_CONTRACT_STATUS_Refused, mean)","(CODE_REJECT_REASON_XAP, mean)"
0,568800.0,450000.0,Working,Higher education,-19241,-2329,NaN,2,Kindergarten,0.752614,0.789654,0.159520,100001,-5145.0,-1572.0,-49.0,-735.000000,577.0,-1329.0,1778.0,82.428571,-3302.0,-1328.0,-825.500000,85240.928571,0.000,-652.0,-155.0,-93.142857,3.0,0.428571,0.571429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2916.0,-2916.0,2520.0,-1499.0,1.00,0.000,1.000
1,222768.0,180000.0,Working,Secondary / secondary special,-18064,-4469,Low-skill Laborers,2,Self-employed,0.564990,0.291656,0.432962,100005,-572.0,-373.0,-62.0,-190.666667,1318.0,-128.0,1324.0,439.333333,-123.0,-123.0,-123.000000,189469.500000,0.000,-163.0,-121.0,-54.333333,2.0,0.666667,0.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-706.0,-736.0,4464.0,-376.0,0.50,0.000,1.000
2,663264.0,630000.0,Working,Higher education,-20038,-4458,Drivers,2,Transport: type 3,NaN,0.699787,0.610991,100013,-6950.0,-2070.0,-1210.0,-1737.500000,-4272.0,-1707.0,-567.0,-1068.000000,-4219.0,-1334.0,-1054.750000,0.000000,NaN,-3102.0,-1334.0,-775.500000,0.0,0.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2705.0,-2705.0,6750.0,-1547.0,0.75,0.000,1.000
3,1575000.0,1575000.0,Working,Secondary / secondary special,-13976,-1866,Sales staff,2,Business Entity Type 3,0.525734,0.509677,0.612704,100028,-16821.0,-2105.0,-269.0,-1401.750000,23877.0,-1862.0,30885.0,2387.700000,-8668.0,-1862.0,-1238.285714,18630.450000,101390.760,-7818.0,-1564.0,-651.500000,5.0,0.416667,0.583333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1773.0,-1785.0,11250.0,363664.0,0.60,0.000,0.800
4,625500.0,625500.0,Working,Secondary / secondary special,-13040,-2191,NaN,2,Business Entity Type 3,0.202145,0.425687,NaN,100038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-787.0,-802.0,8095.5,-457.0,0.50,0.000,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,412560.0,270000.0,Working,Secondary / secondary special,-19970,-5169,NaN,3,Other,NaN,0.648575,0.643026,456221,-3636.0,-1337.0,-113.0,-727.200000,35.0,-604.0,793.0,7.000000,-1373.0,-670.0,-457.666667,111294.000000,0.000,-1411.0,-665.0,-282.200000,2.0,0.400000,0.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-653.0,-661.0,0.0,37.0,1.00,0.000,1.000
48740,622413.0,495000.0,Commercial associate,Secondary / secondary special,-11186,-1149,Sales staff,2,Trade: type 7,NaN,0.684596,NaN,456222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2420.0,-2428.0,3600.0,

In [ ]:
woekag2.drop('SK_ID_CURR',axis=1)

In [234]:
woekag21=woekag2.loc[:,['WOE_[\'AMT_CREDIT\']', 'WOE_[\'AMT_GOODS_PRICE\']',
       'WOE_[\'NAME_INCOME_TYPE\']', 'WOE_[\'NAME_EDUCATION_TYPE\']',
       'WOE_[\'DAYS_BIRTH\']', 'WOE_[\'DAYS_EMPLOYED\']',
       'WOE_[\'OCCUPATION_TYPE\']', 'WOE_[\'REGION_RATING_CLIENT_W_CITY\']',
       'WOE_[\'ORGANIZATION_TYPE\']', 'WOE_[\'EXT_SOURCE_1\']',
       'WOE_[\'EXT_SOURCE_2\']', 'WOE_[\'EXT_SOURCE_3\']',
       'WOE_[\'DAYS_CREDIT\' \'sum\']', 'WOE_[\'DAYS_CREDIT\' \'max\']',
       'WOE_[\'DAYS_CREDIT_ENDDATE\' \'sum\']',
       'WOE_[\'DAYS_CREDIT_ENDDATE\' \'max\']', 'WOE_[\'DAYS_ENDDATE_FACT\' \'mean\']',
       'WOE_[\'AMT_CREDIT_SUM_DEBT\' \'mean\']',
       'WOE_[\'AMT_CREDIT_SUM_LIMIT\' \'max\']',
       'WOE_[\'CREDIT_ACTIVE_Active\' \'sum\']',
       'WOE_[\'CREDIT_ACTIVE_Closed\' \'mean\']',
       'WOE_[\'AMT_DRAWINGS_CURRENT\' \'mean\']',
       'WOE_[\'AMT_PAYMENT_CURRENT\' \'count\']',
       'WOE_[\'CNT_DRAWINGS_ATM_CURRENT\' \'mean\']',
       'WOE_[\'CNT_DRAWINGS_CURRENT\' \'max\']', 'WOE_[\'AMT_DOWN_PAYMENT\' \'sum\']',
       'WOE_[\'DAYS_LAST_DUE_1ST_VERSION\' \'sum\']',
       'WOE_[\'NAME_CONTRACT_STATUS_Approved\' \'mean\']',
       'WOE_[\'CODE_REJECT_REASON_XAP\' \'mean\']',
       'WOE_[\'DAYS_INSTALMENT\' \'min\']']]

In [236]:
kagpred1=logit1.predict_proba(woekag21)[:,1]
submission = woekag2[['SK_ID_CURR']]
submission['TARGET'] = kagpred1

D:\Software\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [237]:
submission.to_csv(r'D:\DS\BSFA\home-credit-default-risk\submission1.csv')

In [240]:
xtrain1.columns

Index(['WOE_['AMT_CREDIT']', 'WOE_['AMT_GOODS_PRICE']',
       'WOE_['NAME_INCOME_TYPE']', 'WOE_['NAME_EDUCATION_TYPE']',
       'WOE_['DAYS_BIRTH']', 'WOE_['DAYS_EMPLOYED']',
       'WOE_['OCCUPATION_TYPE']', 'WOE_['REGION_RATING_CLIENT_W_CITY']',
       'WOE_['ORGANIZATION_TYPE']', 'WOE_['EXT_SOURCE_1']',
       'WOE_['EXT_SOURCE_2']', 'WOE_['EXT_SOURCE_3']',
       'WOE_['DAYS_CREDIT' 'sum']', 'WOE_['DAYS_CREDIT' 'max']',
       'WOE_['DAYS_CREDIT_ENDDATE' 'sum']',
       'WOE_['DAYS_CREDIT_ENDDATE' 'max']', 'WOE_['DAYS_ENDDATE_FACT' 'mean']',
       'WOE_['AMT_CREDIT_SUM_DEBT' 'mean']',
       'WOE_['AMT_CREDIT_SUM_LIMIT' 'max']',
       'WOE_['CREDIT_ACTIVE_Active' 'sum']',
       'WOE_['CREDIT_ACTIVE_Closed' 'mean']',
       'WOE_['AMT_DRAWINGS_CURRENT' 'mean']',
       'WOE_['AMT_PAYMENT_CURRENT' 'count']',
       'WOE_['CNT_DRAWINGS_ATM_CURRENT' 'mean']',
       'WOE_['CNT_DRAWINGS_CURRENT' 'max']', 'WOE_['AMT_DOWN_PAYMENT' 'sum']',
       'WOE_['DAYS_LAST_DUE_1ST_VERSION' 'sum